#### Setup:

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#### Import Berkeley Dataset:

In [ ]:
# Open connection to database
conn = sqlite3.connect("kickstarter.db")
c = conn.cursor()

In [ ]:
# Show all available tables in database
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('urls_to_scrape',), ('all_files',), ('category',), ('creator',), ('funding_trend',), ('item',), ('livestream',), ('location',), ('project',), ('reward',), ('update',), ('comments',), ('creator_external_url',), ('reward_item',), ('creator_bio',)]


In [ ]:
# Retrieve project table from DB and store it as the main dataset
kickstarter_berkeley_df = pd.read_sql_query("SELECT * FROM project", conn)
kickstarter_berkeley_df.set_index("id", inplace=True)
print(kickstarter_berkeley_df.shape)

(408637, 56)


#### Preliminary Data Cleaning - Table: Project

Note: a preliminary data cleaning was necessary, as otherwise the merged datasets would be too big and too messy

In [ ]:
kickstarter_berkeley_df.head()

,state,url_project,url_project_short,name,country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,backers_count,comments_count,updates_count,spotlight,staff_pick,blurb,currency_trailing_code,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22807353,canceled,https://www.kickstarter.com/projects/mikewood/vipersharp-the-best-precision-knife-sharpener,http://kck.st/1UNDF6C,"ViperSharp, The Best Precision Knife Sharpener! (Canceled)",US,1750652465,23418188.0,28,1453170931,1462402800,1472168774,1461010001,NaN,1458918033,10000.0,31131.00,USD,$,31131.00,1.0,255,53,22,False,False,The ViperSharp offers infinite angles for sharpening your kitchen or EDC pocket knife while addressing the flaws of existing systems.,True,False,https://ksr-ugc.imgix.net/assets/012/357/781/223ca878a5f7fe8ac340f0ab734bdf9d_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463756591&auto=format&q=92&s=27b350de3f14d4954fb9c692ce03b659,NaN,0.8,NaN,2328882,NaN,NaN,NaN,NaN,NaN,2328882,True,False,inactive,1453170931,NaN,vipersharp-the-best-precision-knife-sharpener,https://www.kickstarter.com/projects/mikewood/vipersharp-the-best-precision-knife-sharpener/rewards,https://www.kickstarter.com/projects/mikewood/vipersharp-the-best-precision-knife-sharpener/posts,657517.0,https://ksr-video.imgix.net/projects/2289308/video-657517-h264_high.mp4,https://ksr-video.imgix.net/projects/2289308/video-657517-webm.webm,360.0,640.0,successful,www.kickstarter.com/projects/mikewood/vipersharp-the-best-precision-knife-sharpener,2018-05-24 02:33:34,0.0
22810934,successful,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,http://kck.st/17xiZX3,"Lights, Coma, Action!",US,1157451606,2357536.0,48,1380594589,1383413613,1488327585,1383413613,1.383414e+09,1380821613,15000.0,18167.88,USD,$,18167.88,1.0,284,4,40,True,False,"A book to tell the story of who I was (LIGHTS), how I sustained a serious brain injury (COMA), and my strong recovery (ACTION!).",True,False,https://ksr-ugc.imgix.net/assets/011/584/530/2c3616a07f99927f568a2964701a8cd7_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463684821&auto=format&q=92&s=da0a23409fc54db476fe6eb9630e786c,NaN,0.8,"A book to tell the story of how I sustained a serious brain injury and made a strong recovery (LIGHTS, COMA, ACTION!).",716409,NaN,Adventures in Brain Injury,NaN,http://www.adventuresinbraininjury.com,"Lights, Coma, Action!",716409,True,False,active,1432157366,NaN,lights-coma-action,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action/rewards,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action/posts,297295.0,https://ksr-video.imgix.net/projects/700251/video-297295-h264_high.mp4,https://ksr-video.imgix.net/projects/700251/video-297295-webm.webm,360.0,640.0,successful,www.kickstarter.com/projects/cavinbounce/lights-coma-action,2018-05-24 02:26:07,0.0
22821161,failed,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,Sentio Golf Putters: Feel IS the difference,US,684178251,2351810.0,28,1433725049,1446429290,1463739059,1446429290,NaN,1442537690,50000.0,9707.00,USD,$,9707.00,1.0,41,0,4,False,False,"Choose the feel YOU want with our patented floating-face technology. Modern design, proven performance, and a splash of color.",True,False,h

##### State:

In [ ]:
kickstarter_berkeley_df.state.isna().sum()

0

In [ ]:
kickstarter_berkeley_df.state.value_counts()

failed        211783
successful    155130
canceled       36659
live            3163
suspended       1700
purged           202
Name: state, dtype: int64

Result:
- Remove canceled, suspended, and purged projects, because they could be cancelled for any reason (no clear outcome)
- Remove live projects, since no clear outcome

In [ ]:
# Remove projects with state != failed or successful, since no clear outcome available
print("before: {}".format(kickstarter_berkeley_df.shape))
remove_indices = kickstarter_berkeley_df[(kickstarter_berkeley_df.state=="live") | 
                                        (kickstarter_berkeley_df.state=="canceled") |
                                        (kickstarter_berkeley_df.state=="suspended") |
                                        (kickstarter_berkeley_df.state=="purged")
                                       ].index
kickstarter_berkeley_df.drop(index=remove_indices, inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))
print(kickstarter_berkeley_df.state.value_counts())

before: (408637, 56)
after: (366913, 56)
failed        211783
successful    155130
Name: state, dtype: int64


##### url_project:

Result:
- keep for now, maybe useful as reference for validating the project
- but must be removed for final dataset

##### url_project_short:

In [ ]:
# Drop attribute url_project_short, since redundant to url_project
print("before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["url_project_short"], inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366913, 56)
after: (366913, 55)


##### name:

In [ ]:
kickstarter_berkeley_df.name.isna().sum()

3

In [ ]:
kickstarter_berkeley_df[kickstarter_berkeley_df.name.isna()==True]

,state,url_project,name,country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,backers_count,comments_count,updates_count,spotlight,staff_pick,blurb,currency_trailing_code,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
634871725,failed,https://www.kickstarter.com/projects/177117814/unlamentia-straima-maybe-more,NaN,GB,177117814,40442.0,35,1355858653,1357509600,1397805790,1357509632,NaN,1355957868,2000.0,196.0,GBP,£,317.728436,1.621063,12,7,0,False,False,...,False,False,https://ksr-ugc.imgix.net/assets/011/461/495/0c0596b59df08feb0e26af7536861677_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463683040&auto=format&q=92&s=3d62d688a5fa94d12609184d643a5fc3,NaN,0.8,NaN,425839,NaN,NaN,NaN,NaN,NaN,425839,True,False,inactive,1425915823,NaN,unlamentia-straima-maybe-more,https://www.kickstarter.com/projects/177117814/unlamentia-straima-maybe-more/rewards,https://www.kickstarter.com/projects/177117814/unlamentia-straima-maybe-more/posts,NaN,NaN,NaN,NaN,NaN,NaN,www.kickstarter.com/projects/177117814/unlamentia-straima-maybe-more,2018-05-24 22:36:52,0.0
796533179,failed,https://www.kickstarter.com/projects/1751437608/you-have-the-power-to-put-our-film-in-theaters,NaN,US,1751437608,2450851.0,23,1320612894,1323061140,1397767520,1323061142,NaN,1320620155,35000.0,220.0,USD,$,220.000000,1.000000,5,0,0,False,False,NaN,True,False,https://ksr-ugc.imgix.net/assets/011/303/534/17c1d0c881ba722cf065b961b97beb1a_original.jpeg?w=1536&h=864&fit=fill&bg=000000&v=1463680703&auto=format&q=92&s=21c3393082bbf2831a51a01c467c61ed,NaN,0.8,NaN,57777,NaN,NaN,NaN,NaN,NaN,57777,True,False,inactive,1425915803,NaN,you-have-the-power-to-put-our-film-in-theaters,https://www.kickstarter.com/projects/1751437608/you-have-the-power-to-put-our-film-in-theaters/rewards,https://www.kickstarter.com/projects/1751437608/you-have-the-power-to-put-our-film-in-theaters/posts,NaN,NaN,NaN,NaN,NaN,NaN,www.kickstarter.com/projects/1751437608/you-have-the-power-to-put-our-film-in-theaters,2017-03-11 11:12:54,NaN
1848699072,failed,https://www.kickstarter.com/projects/madegoke/bullied-to-triumph,NaN,US,887042339,2373295.0,31,1325021401,1330524240,1463680785,1330524241,NaN,1325417731,200000.0,100.0,USD,$,100.000000,1.000000,1,0,0,False,False,NaN,True,False,https://ksr-ugc.imgix.net/assets/011/309/626/f7abab6b89d2f3c440624455372338b0_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463680785&auto=format&q=92&s=8cff585dd07f1484008296eab525f129,NaN,0.8,NaN,66497,NaN,NaN,NaN,NaN,NaN,66497,True,False,inactive,1425915804,NaN,bullied-to-triumph,https://www.kickstarter.com/projects/madegoke/bullied-to-triumph/rewards,https://www.kickstarter.com/projects/madegoke/bullied-to-triumph/posts,NaN,NaN,NaN,NaN,NaN,NaN,www.kickstarter.com/projects/madegoke/bullied-to-triumph,2017-03-11 20:25:17,NaN


In [ ]:
# Drop projects with name = NaN, since they don't look like serious attempts
print("before: {}".format(kickstarter_berkeley_df.shape))
remove_indices = kickstarter_berkeley_df[(kickstarter_berkeley_df.name.isna()==True)].index
kickstarter_berkeley_df.drop(index=remove_indices, inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366913, 55)
after: (366910, 55)


##### country:

In [ ]:
kickstarter_berkeley_df.country.isna().sum()

0

In [ ]:
kickstarter_berkeley_df.country.value_counts()

US    282990
GB     33632
CA     14094
AU      7418
DE      4399
FR      3326
IT      3129
NL      2696
ES      2532
MX      2442
SE      1843
NZ      1449
DK      1112
HK      1015
CH       839
IE       823
SG       815
BE       693
NO       670
AT       645
JP       279
LU        69
Name: country, dtype: int64

In [ ]:
# Replace country codes with full name
import pycountry
def replace_country_code(x):
    country = pycountry.countries.get(alpha_2=x)
    if country is None:
        return x
    else:
        return country.name

kickstarter_berkeley_df.country = kickstarter_berkeley_df.country.apply(lambda x: replace_country_code(x))
kickstarter_berkeley_df.country.value_counts()

United States     282990
United Kingdom     33632
Canada             14094
Australia           7418
Germany             4399
France              3326
Italy               3129
Netherlands         2696
Spain               2532
Mexico              2442
Sweden              1843
New Zealand         1449
Denmark             1112
Hong Kong           1015
Switzerland          839
Ireland              823
Singapore            815
Belgium              693
Norway               670
Austria              645
Japan                279
Luxembourg            69
Name: country, dtype: int64

In [ ]:
# Rename column, since this column is not the location of the project, but the country of the Kickstarter website
kickstarter_berkeley_df.rename(columns={"country" : "kickstarter_country"}, inplace=True)
kickstarter_berkeley_df.head()

,state,url_project,name,kickstarter_country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,backers_count,comments_count,updates_count,spotlight,staff_pick,blurb,currency_trailing_code,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22810934,successful,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,"Lights, Coma, Action!",United States,1157451606,2357536.0,48,1380594589,1383413613,1488327585,1383413613,1.383414e+09,1380821613,15000.0,18167.88,USD,$,18167.88,1.0,284,4,40,True,False,"A book to tell the story of who I was (LIGHTS), how I sustained a serious brain injury (COMA), and my strong recovery (ACTION!).",True,False,https://ksr-ugc.imgix.net/assets/011/584/530/2c3616a07f99927f568a2964701a8cd7_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463684821&auto=format&q=92&s=da0a23409fc54db476fe6eb9630e786c,NaN,0.8,"A book to tell the story of how I sustained a serious brain injury and made a strong recovery (LIGHTS, COMA, ACTION!).",716409,NaN,Adventures in Brain Injury,NaN,http://www.adventuresinbraininjury.com,"Lights, Coma, Action!",716409,True,False,active,1432157366,NaN,lights-coma-action,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action/rewards,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action/posts,297295.0,https://ksr-video.imgix.net/projects/700251/video-297295-h264_high.mp4,https://ksr-video.imgix.net/projects/700251/video-297295-webm.webm,360.0,640.0,successful,www.kickstarter.com/projects/cavinbounce/lights-coma-action,2018-05-24 02:26:07,0.0
22821161,failed,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,Sentio Golf Putters: Feel IS the difference,United States,684178251,2351810.0,28,1433725049,1446429290,1463739059,1446429290,NaN,1442537690,50000.0,9707.00,USD,$,9707.00,1.0,41,0,4,False,False,"Choose the feel YOU want with our patented floating-face technology. Modern design, proven performance, and a splash of color.",True,False,https://ksr-ugc.imgix.net/assets/012/161/151/b936e88b77de917c166855d0e28521d3_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463739059&auto=format&q=92&s=38b2db3a7a98a4d010d08e234e281b0c,NaN,0.8,NaN,1945377,NaN,NaN,NaN,NaN,NaN,1945377,True,False,inactive,1433725049,NaN,sentio-golf-putters-feel-is-the-difference,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference/rewards,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference/posts,579025.0,https://ksr-video.imgix.net/projects/1905803/video-579025-h264_high.mp4,https://ksr-video.imgix.net/projects/1905803/video-579025-webm.webm,480.0,640.0,successful,www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,2017-03-11 01:20:37,NaN
22823613,failed,https://www.kickstarter.com/projects/1237543205/brainade,Brainade,United States,1237543205,2497409.0,342,1486073091,1489778686,1489778687,1489778687,NaN,1487190286,750.0,1.00,USD,$,1.00,1.0,1,0,0,False,False,We intend to create an online platform for individuals to teach or learn a skill via online classrooms and forums.,True,False,https://ksr-ugc.imgix.net/assets/015/387/055/f2a58f4adb49d3b4436d5b07cc8efb56_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1486148160&auto=format&q=92&s=fb83670e48f2b76c933f2a96314cf081,NaN,0.8,NaN,2862898,NaN,NaN,NaN,NaN,NaN,2862898,True

##### creator_id:

In [ ]:
kickstarter_berkeley_df.creator_id.isna().sum()

0

In [ ]:
kickstarter_berkeley_df.creator_id.value_counts()

957058942     100
2104052526     81
1388978138     80
802626862      80
978911160      50
             ... 
230138961       1
443055450       1
1075797070      1
603419723       1
1221978673      1
Name: creator_id, Length: 306238, dtype: int64

In [ ]:
kickstarter_berkeley_df[kickstarter_berkeley_df.creator_id==2104052526].head()

,state,url_project,name,kickstarter_country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,backers_count,comments_count,updates_count,spotlight,staff_pick,blurb,currency_trailing_code,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
165963723,failed,https://www.kickstarter.com/projects/2104052526/bicycle-red-castle-playing-cards,Bicycle Red Castle Playing Cards,United States,2104052526,2425938.0,273,1462295570,1464972909,1469725292,1464972918,NaN,1462380909,4000.0,2123.0,USD,$,2123.0,1.0,105,3,1,False,False,Welcome to the Bicycle Red Castle the highest ranking Fortress in all the land.,True,False,https://ksr-ugc.imgix.net/assets/012/443/963/f94a66faf9c886f22787438aa45b26ee_original.jpg?w=1536&h=864&fit=fill&bg=F6D584&v=1463761470&auto=format&q=92&s=9c6e895eeda5d0f9102a0892a39f57a8,NaN,0.8,NaN,2498967,NaN,NaN,NaN,NaN,NaN,2498967,True,False,inactive,1462295570,NaN,bicycle-red-castle-playing-cards,https://www.kickstarter.com/projects/2104052526/bicycle-red-castle-playing-cards/rewards,https://www.kickstarter.com/projects/2104052526/bicycle-red-castle-playing-cards/posts,NaN,NaN,NaN,NaN,NaN,NaN,www.kickstarter.com/projects/2104052526/bicycle-red-castle-playing-cards,2018-05-24 03:55:55,0.0
181551138,successful,https://www.kickstarter.com/projects/2104052526/bicycle-us-presidents-playing-cards,Bicycle US Presidents Playing Cards,United States,2104052526,2425938.0,273,1421727084,1425832151,1463723447,1425832152,1.425832e+09,1422725351,15000.0,19587.0,USD,$,19587.0,1.0,518,52,9,True,False,"“America”, synonymous with “liberty” and young enough to fit all 44 of its leaders in one deck of playing cards. Printed by USPCC",True,False,https://ksr-ugc.imgix.net/assets/012/006/159/b2bcb8b25d83ddb24d44a746a47cf487_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463723441&auto=format&q=92&s=2c0754ab9f1a6f00644db8aaf7ebbd83,NaN,0.8,NaN,1642708,NaN,NaN,NaN,NaN,NaN,1642708,True,False,inactive,1425915890,NaN,bicycle-us-presidents-playing-cards,https://www.kickstarter.com/projects/2104052526/bicycle-us-presidents-playing-cards/rewards,https://www.kickstarter.com/projects/2104052526/bicycle-us-presidents-playing-cards/posts,NaN,NaN,NaN,NaN,NaN,NaN,www.kickstarter.com/projects/2104052526/bicycle-us-presidents-playing-cards,2018-05-24 04:04:15,0.0
276137966,successful,https://www.kickstarter.com/projects/2104052526/bicycle-starlight-shooting-star-playing-cards,Bicycle Starlight Shooting Star Playing Cards,United States,2104052526,2425938.0,273,1454612992,1458412070,1463757572,1458412071,1.458412e+09,1455823670,5000.0,8416.0,USD,$,8416.0,1.0,307,11,2,True,False,Look up! Bicycle Starlight Shooting Star is here!,True,False,https://ksr-ugc.imgix.net/assets/012/372/329/39585e1bda2a167e8154fb9a113ffaa6_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463757572&auto=format&q=92&s=6ae1a5f43e801af706647f6467a241c3,NaN,0.8,NaN,2357424,NaN,NaN,NaN,NaN,NaN,2357424,True,False,inactive,1454612992,NaN,bicycle-starlight-shooting-star-playing-cards,https://www.kickstarter.com/projects/2104052526/bicycle-starlight-shooting-star-playing-cards/rewards,https://www.kickstarter.com/projects/2104052526/bicycle-starlight-shooting-star-playing-cards/posts,NaN,NaN,NaN,NaN,NaN,NaN,www.kickstarter.com/projects/2104052526/bicycle-starlight-shooting-star-playing-cards,2018-05-24 05:11:09,0.0
305127426,successful,https://www.kickstarter.com/proj

Result:
- leave attribute for now, since I need this to merge it later with the creator table

##### location_id:

In [ ]:
kickstarter_berkeley_df.location_id.isna().sum()

1537

In [ ]:
kickstarter_berkeley_df[kickstarter_berkeley_df.location_id.isna()==True].head()

,state,url_project,name,kickstarter_country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,backers_count,comments_count,updates_count,spotlight,staff_pick,blurb,currency_trailing_code,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
23244535,successful,https://www.kickstarter.com/projects/965936878/the-second-funny-little-fella-album-still-funny,"The second Funny Little Fella album (still funny, slightly bigger)",United States,965936878,NaN,14,1267741198,1270442640,1397754489,1270442708,1.270443e+09,1267858664,3000.0,3079.00,USD,$,3079.00,1.0,45,0,0,True,False,"My second children's album, exploring the toddler years. From tantrums to band-aid obsessions.",True,False,https://ksr-ugc.imgix.net/assets/011/260/185/a54f0719c84466dbd70cee971cae84d5_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463673786&auto=format&q=92&s=b9ed2951c202f0011272b5a0d4bfe928,NaN,0.8,NaN,3755,NaN,NaN,NaN,NaN,NaN,3755,True,False,inactive,1425915800,NaN,the-second-funny-little-fella-album-still-funny,https://www.kickstarter.com/projects/965936878/the-second-funny-little-fella-album-still-funny/rewards,https://www.kickstarter.com/projects/965936878/the-second-funny-little-fella-album-still-funny/posts,NaN,NaN,NaN,NaN,NaN,NaN,www.kickstarter.com/projects/965936878/the-second-funny-little-fella-album-still-funny,2017-03-11 01:20:37,NaN
28409399,failed,https://www.kickstarter.com/projects/angrybrownguy/diamond-destruction-how-greed-media-and-steroids-r,"Diamond Destruction: How Greed, Media And Steroids Ruined Our Game",United States,1672049148,NaN,13,1274231424,1280030340,1397755943,1280030413,NaN,1274244252,1000.0,145.00,USD,$,145.00,1.0,6,0,0,False,False,Diamond Destruction is a book about how the game of baseball is slowly deteriorated and what we might be able to do restore our national pastime.,True,False,https://ksr-ugc.imgix.net/assets/011/262/339/08d38a13dab637fdc3efe7f3f4a7b476_original.png?w=1536&h=864&fit=fill&bg=000000&v=1463673802&auto=format&q=92&s=f3bb5741d7456aff750fd45cc4463ba5,NaN,0.8,NaN,6373,NaN,NaN,NaN,NaN,NaN,6373,True,False,inactive,1425915800,NaN,diamond-destruction-how-greed-media-and-steroids-r,https://www.kickstarter.com/projects/angrybrownguy/diamond-destruction-how-greed-media-and-steroids-r/rewards,https://www.kickstarter.com/projects/angrybrownguy/diamond-destruction-how-greed-media-and-steroids-r/posts,NaN,NaN,NaN,NaN,NaN,NaN,www.kickstarter.com/projects/angrybrownguy/diamond-destruction-how-greed-media-and-steroids-r,2017-03-11 01:20:37,NaN
29465183,failed,https://www.kickstarter.com/projects/2038563099/paddy-mirage-metropolis-2010-an-irish-pub-serving,Paddy Mirage Metropolis 2010 > An Irish pub serving the community of Burning Man 2010,United States,2038563099,NaN,10,1279399239,1282158000,1397757645,1282158008,NaN,1279403847,3000.0,355.00,USD,$,355.00,1.0,8,3,1,False,False,Paddy Mirage is a pub in the desert built for the participants of Burning Man and will serve as a gathering place to bond while sharing the experience,True,False,https://ksr-ugc.imgix.net/assets/011/264/819/3abe48f82409cb2149bcc5d755fd4fac_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463673820&auto=format&q=92&s=8de2943cd06510f922573f9d7fd9f16d,NaN,0.8,NaN,9320,NaN,NaN,NaN,NaN,NaN,9320,True,False,inactive,1425915801,NaN,paddy-mirage-metropolis-2010-an-irish-pub-serving,https://www.kickstarter.com/projects/20385630

Result:
- keep for now, since I need it later to merge it with the location table
- plus: there are 1500 projects that don't have a location specified (need to replace value with "No Location")

##### category_id:

In [ ]:
kickstarter_berkeley_df.category_id.isna().sum()

0

In [ ]:
kickstarter_berkeley_df.category_id.value_counts()

28     21146
14     16011
30     14628
34     14518
32     11692
       ...  
389       62
349       54
317       38
386       37
354       10
Name: category_id, Length: 169, dtype: int64

Result:
- keep for now, since I need it later to merge it with the category table

##### DateTime-Related Attributes:

In [ ]:
print("created_at: {}".format(kickstarter_berkeley_df.created_at.isna().sum()))
print("deadline: {}".format(kickstarter_berkeley_df.deadline.isna().sum()))
print("updated_at: {}".format(kickstarter_berkeley_df.updated_at.isna().sum()))
print("state_changed_at: {}".format(kickstarter_berkeley_df.state_changed_at.isna().sum()))
print("successful_at: {}".format(kickstarter_berkeley_df.successful_at.isna().sum()))
print("launched_at: {}".format(kickstarter_berkeley_df.launched_at.isna().sum()))
print("profile_state_changed_at: {}".format(kickstarter_berkeley_df.profile_state_changed_at.isna().sum()))
print("last_modification: {}".format(kickstarter_berkeley_df.last_modification.isna().sum()))

created_at: 0
deadline: 0
updated_at: 0
state_changed_at: 0
successful_at: 211780
launched_at: 0
profile_state_changed_at: 0
last_modification: 0


Result: is ok! successful_at is allowed to be missing for failed projects

In [ ]:
# Convert attributes from unix timestamp to normal format
from datetime import datetime
kickstarter_berkeley_df["created_at"] = kickstarter_berkeley_df.created_at.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
kickstarter_berkeley_df["deadline"] = kickstarter_berkeley_df.deadline.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
kickstarter_berkeley_df["updated_at"] = kickstarter_berkeley_df.updated_at.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
kickstarter_berkeley_df["state_changed_at"] = kickstarter_berkeley_df.state_changed_at.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
kickstarter_berkeley_df["successful_at"] = kickstarter_berkeley_df.successful_at.apply(lambda x: x if np.isnan(x) else datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
kickstarter_berkeley_df["launched_at"] = kickstarter_berkeley_df.launched_at.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
kickstarter_berkeley_df["profile_state_changed_at"] = kickstarter_berkeley_df.profile_state_changed_at.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
kickstarter_berkeley_df["last_modification"] = kickstarter_berkeley_df.last_modification.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d'))
kickstarter_berkeley_df.head()

,state,url_project,name,kickstarter_country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,backers_count,comments_count,updates_count,spotlight,staff_pick,blurb,currency_trailing_code,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22810934,successful,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,"Lights, Coma, Action!",United States,1157451606,2357536.0,48,2013-10-01,2013-11-02,2017-03-01,2013-11-02,2013-11-02,2013-10-03,15000.0,18167.88,USD,$,18167.88,1.0,284,4,40,True,False,"A book to tell the story of who I was (LIGHTS), how I sustained a serious brain injury (COMA), and my strong recovery (ACTION!).",True,False,https://ksr-ugc.imgix.net/assets/011/584/530/2c3616a07f99927f568a2964701a8cd7_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463684821&auto=format&q=92&s=da0a23409fc54db476fe6eb9630e786c,NaN,0.8,"A book to tell the story of how I sustained a serious brain injury and made a strong recovery (LIGHTS, COMA, ACTION!).",716409,NaN,Adventures in Brain Injury,NaN,http://www.adventuresinbraininjury.com,"Lights, Coma, Action!",716409,True,False,active,2015-05-20,NaN,lights-coma-action,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action/rewards,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action/posts,297295.0,https://ksr-video.imgix.net/projects/700251/video-297295-h264_high.mp4,https://ksr-video.imgix.net/projects/700251/video-297295-webm.webm,360.0,640.0,successful,www.kickstarter.com/projects/cavinbounce/lights-coma-action,2018-05-24,0.0
22821161,failed,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,Sentio Golf Putters: Feel IS the difference,United States,684178251,2351810.0,28,2015-06-08,2015-11-02,2016-05-20,2015-11-02,NaN,2015-09-18,50000.0,9707.00,USD,$,9707.00,1.0,41,0,4,False,False,"Choose the feel YOU want with our patented floating-face technology. Modern design, proven performance, and a splash of color.",True,False,https://ksr-ugc.imgix.net/assets/012/161/151/b936e88b77de917c166855d0e28521d3_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463739059&auto=format&q=92&s=38b2db3a7a98a4d010d08e234e281b0c,NaN,0.8,NaN,1945377,NaN,NaN,NaN,NaN,NaN,1945377,True,False,inactive,2015-06-08,NaN,sentio-golf-putters-feel-is-the-difference,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference/rewards,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference/posts,579025.0,https://ksr-video.imgix.net/projects/1905803/video-579025-h264_high.mp4,https://ksr-video.imgix.net/projects/1905803/video-579025-webm.webm,480.0,640.0,successful,www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,2017-03-11,NaN
22823613,failed,https://www.kickstarter.com/projects/1237543205/brainade,Brainade,United States,1237543205,2497409.0,342,2017-02-02,2017-03-17,2017-03-17,2017-03-17,NaN,2017-02-15,750.0,1.00,USD,$,1.00,1.0,1,0,0,False,False,We intend to create an online platform for individuals to teach or learn a skill via online classrooms and forums.,True,False,https://ksr-ugc.imgix.net/assets/015/387/055/f2a58f4adb49d3b4436d5b07cc8efb56_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1486148160&auto=format&q=92&s=fb83670e48f2b76c933f2a96314cf081,NaN,0.8,NaN,2862898,NaN,NaN,NaN,NaN,NaN,2862898,True,False,inactive,2017

In [ ]:
years = kickstarter_berkeley_df["deadline"].apply(lambda x: x[0:4])
years.value_counts()

2015    61065
2014    59663
2016    47601
2017    45728
2013    41421
2012    39021
2018    38136
2011    23104
2010     8358
2019     2007
2009      806
Name: deadline, dtype: int64

Result: 
- in comparison to the Webrobots dataset, mostly projects from 2019 and 2020 are missing (makes sense, since this dataset was collected in 2019)

##### Currency-Related Attributes:

In [ ]:
currency_df = kickstarter_berkeley_df[["name", "url_project", "goal", "pledged", "currency", "currency_symbol", "usd_pledged", "static_usd_rate",
                       "currency_trailing_code"]]
currency_df.head()

,name,url_project,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,currency_trailing_code
id,,,,,,,,,
22810934,"Lights, Coma, Action!",https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,15000.0,18167.88,USD,$,18167.88,1.0,True
22821161,Sentio Golf Putters: Feel IS the difference,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,50000.0,9707.00,USD,$,9707.00,1.0,True
22823613,Brainade,https://www.kickstarter.com/projects/1237543205/brainade,750.0,1.00,USD,$,1.00,1.0,True
22826244,Horizon Anthology,https://www.kickstarter.com/projects/impkingcomics/horizon-anthology,7000.0,8955.41,USD,$,8955.41,1.0,True
22835897,The Alphabet Story Children's Book,https://www.kickstarter.com/projects/alphabetstory/the-alphabet-story-childrens-book,6000.0,6643.11,USD,$,6643.11,1.0,True


In [ ]:
currency_df[currency_df.currency!="USD"].head()

,name,url_project,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,currency_trailing_code
id,,,,,,,,,
22954941,Material Design Templates for the Ionic Framework,https://www.kickstarter.com/projects/1201567024/material-design-templates-for-ionic-framework,2000.0,479.0,GBP,£,745.740016,1.556869,False
22969799,Funding for my first studio album,https://www.kickstarter.com/projects/angiejones/funding-for-my-first-studio-album,2600.0,0.0,CAD,$,0.000000,0.805413,True
22994870,"BOAR-D Magnetic wakeboard, wakeskate, kitesurf",https://www.kickstarter.com/projects/2087972523/boar-d-a-new-esperience-wakeboard-wake-skate-kites,50000.0,1047.0,EUR,€,1159.983560,1.107912,False
23027769,God Will Fall,https://www.kickstarter.com/projects/610778520/god-will-fall,1000.0,1105.0,GBP,£,1716.731525,1.553603,False
23061036,Cold Corpse - Virtual Reality Horror Shooter Game,https://www.kickstarter.com/projects/346917533/cold-corpse-virtual-reality-horror-shooter-game,1000.0,1.0,GBP,£,1.429989,1.429989,False


In [ ]:
currency_df.isna().sum()

name                      0
url_project               0
goal                      0
pledged                   0
currency                  0
currency_symbol           0
usd_pledged               0
static_usd_rate           0
currency_trailing_code    0
dtype: int64

In [ ]:
# Convert goal into USD
kickstarter_berkeley_df["goal"] = kickstarter_berkeley_df["goal"] * kickstarter_berkeley_df["static_usd_rate"]

In [ ]:
# Remove unnecessary currency-related attributes
print("Before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["pledged", "currency", "currency_symbol", "static_usd_rate", "currency_trailing_code"], inplace=True)
kickstarter_berkeley_df.rename(columns={"usd_pledged" : "pledged"}, inplace=True)
print("After: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.head()

Before: (366910, 55)
After: (366910, 50)


,state,url_project,name,kickstarter_country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,backers_count,comments_count,updates_count,spotlight,staff_pick,blurb,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22810934,successful,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,"Lights, Coma, Action!",United States,1157451606,2357536.0,48,2013-10-01,2013-11-02,2017-03-01,2013-11-02,2013-11-02,2013-10-03,15000.0,18167.88,284,4,40,True,False,"A book to tell the story of who I was (LIGHTS), how I sustained a serious brain injury (COMA), and my strong recovery (ACTION!).",False,https://ksr-ugc.imgix.net/assets/011/584/530/2c3616a07f99927f568a2964701a8cd7_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463684821&auto=format&q=92&s=da0a23409fc54db476fe6eb9630e786c,NaN,0.8,"A book to tell the story of how I sustained a serious brain injury and made a strong recovery (LIGHTS, COMA, ACTION!).",716409,NaN,Adventures in Brain Injury,NaN,http://www.adventuresinbraininjury.com,"Lights, Coma, Action!",716409,True,False,active,2015-05-20,NaN,lights-coma-action,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action/rewards,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action/posts,297295.0,https://ksr-video.imgix.net/projects/700251/video-297295-h264_high.mp4,https://ksr-video.imgix.net/projects/700251/video-297295-webm.webm,360.0,640.0,successful,www.kickstarter.com/projects/cavinbounce/lights-coma-action,2018-05-24,0.0
22821161,failed,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,Sentio Golf Putters: Feel IS the difference,United States,684178251,2351810.0,28,2015-06-08,2015-11-02,2016-05-20,2015-11-02,NaN,2015-09-18,50000.0,9707.00,41,0,4,False,False,"Choose the feel YOU want with our patented floating-face technology. Modern design, proven performance, and a splash of color.",False,https://ksr-ugc.imgix.net/assets/012/161/151/b936e88b77de917c166855d0e28521d3_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1463739059&auto=format&q=92&s=38b2db3a7a98a4d010d08e234e281b0c,NaN,0.8,NaN,1945377,NaN,NaN,NaN,NaN,NaN,1945377,True,False,inactive,2015-06-08,NaN,sentio-golf-putters-feel-is-the-difference,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference/rewards,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference/posts,579025.0,https://ksr-video.imgix.net/projects/1905803/video-579025-h264_high.mp4,https://ksr-video.imgix.net/projects/1905803/video-579025-webm.webm,480.0,640.0,successful,www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,2017-03-11,NaN
22823613,failed,https://www.kickstarter.com/projects/1237543205/brainade,Brainade,United States,1237543205,2497409.0,342,2017-02-02,2017-03-17,2017-03-17,2017-03-17,NaN,2017-02-15,750.0,1.00,1,0,0,False,False,We intend to create an online platform for individuals to teach or learn a skill via online classrooms and forums.,False,https://ksr-ugc.imgix.net/assets/015/387/055/f2a58f4adb49d3b4436d5b07cc8efb56_original.jpg?w=1536&h=864&fit=fill&bg=000000&v=1486148160&auto=format&q=92&s=fb83670e48f2b76c933f2a96314cf081,NaN,0.8,NaN,2862898,NaN,NaN,NaN,NaN,NaN,2862898,True,False,inactive,2017-02-02,NaN,brainade,https://www.kickstarter.com/projects/1237543205/brainade/rewards,https://www.kickstarter.com/projects/1237543205/brainade/posts,

##### backers_count:

In [ ]:
kickstarter_berkeley_df.backers_count.isna().sum()

0

Result: 
- keep attribute for now
- however, must be likely removed in final dataset, since it is an information leak

##### comments_count:

In [ ]:
kickstarter_berkeley_df.comments_count.isna().sum()

0

Result: 
- keep attribute for now
- however, must be likely removed in final dataset, since it is an information leak

##### updates_count:

In [ ]:
kickstarter_berkeley_df.updates_count.isna().sum()

0

Result: 
- keep attribute for now
- however, must be likely removed in final dataset, since it is an information leak

##### spotlight:

In [ ]:
kickstarter_berkeley_df.spotlight.isna().sum()

0

In [ ]:
kickstarter_berkeley_df.spotlight.value_counts()

False    201534
True     148992
False      5353
0          4893
True       3270
1          2868
Name: spotlight, dtype: int64

In [ ]:
# Drop attribute spotlight, since spotlight is a feature that is created after a project has been already successfully funded
# i.e. all successful projects are in spotlight
print("before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["spotlight"], inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366910, 50)
after: (366910, 49)


##### staff_pick:

In [ ]:
kickstarter_berkeley_df.staff_pick.isna().sum()

0

In [ ]:
kickstarter_berkeley_df.staff_pick.value_counts()

False    314247
True      36279
False      7759
0          7189
True        864
1           572
Name: staff_pick, dtype: int64

In [ ]:
# Clean up values
kickstarter_berkeley_df["staff_pick"] = kickstarter_berkeley_df.staff_pick.apply(lambda x: True if x=="True" else
                                                                              (True if x=="1" else
                                                                              (False if x=="False" else
                                                                              (False if x=="0" else x))))
kickstarter_berkeley_df.staff_pick.value_counts()

False    329195
True      37715
Name: staff_pick, dtype: int64

Result: keep attribute

##### blurb:

In [ ]:
kickstarter_berkeley_df.blurb.isna().sum()

3

In [ ]:
kickstarter_berkeley_df[kickstarter_berkeley_df.blurb.isna()==True]

,state,url_project,name,kickstarter_country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,backers_count,comments_count,updates_count,staff_pick,blurb,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1484424573,failed,https://www.kickstarter.com/projects/678870507/modern-ceramics-design,Pottery Fun,United States,678870507,2356503.0,1,2015-10-11,2015-12-11,2015-12-11,2015-12-11,NaN,2015-10-12,200.00000,0.000000,0,0,0,False,NaN,False,https://ksr-ugc.imgix.net/assets/012/276/655/7cf782442d65e4104f722ac4aef991c8_original.png?w=1536&h=864&fit=fill&bg=000000&v=1463749649&auto=format&q=92&s=dbe43ac5330e541528a643a0cc5d2d8d,NaN,0.8,NaN,2168447,NaN,NaN,NaN,NaN,NaN,2168447,True,False,inactive,2015-10-11,NaN,modern-ceramics-design,https://www.kickstarter.com/projects/678870507/modern-ceramics-design/rewards,https://www.kickstarter.com/projects/678870507/modern-ceramics-design/posts,NaN,NaN,NaN,NaN,NaN,NaN,www.kickstarter.com/projects/678870507/modern-ceramics-design,2017-03-11,NaN
937524480,failed,https://www.kickstarter.com/projects/1303591875/serious-business-collection,Ready to wear,Germany,1303591875,676757.0,269,2017-09-02,2017-10-10,2017-10-10,2017-10-10,NaN,2017-09-10,18055.49535,8873.674115,39,1,2,False,NaN,0,https://ksr-ugc.imgix.net/assets/018/167/575/383ff9073fee925edb82abb1e6c92c02_original.PNG?crop=faces&w=1552&h=873&fit=crop&v=1506598860&auto=format&q=92&s=84896a7aafb1a9e38d4447952a3e0e0a,NaN,0.8,NaN,3128728,NaN,NaN,NaN,NaN,NaN,3128728,1,0,inactive,2017-09-02,NaN,serious-business-collection,https://www.kickstarter.com/projects/1303591875/serious-business-collection/rewards,https://www.kickstarter.com/projects/1303591875/serious-business-collection/posts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-29,NaN
1077399482,failed,https://www.kickstarter.com/projects/1454907110/charivari,awlsies,United States,1454907110,2442047.0,21,2017-11-03,2018-01-12,2018-01-12,2018-01-12,NaN,2017-11-13,40000.00000,0.000000,0,0,0,False,NaN,0,https://ksr-ugc.imgix.net/assets/019/044/943/d2571966baa2e25603ac45c1d90754ce_original.png?crop=faces&w=1552&h=873&fit=crop&v=1512541342&auto=format&q=92&s=33d931460fa607c40a7e4100409cd2a8,NaN,0.8,NaN,3206374,NaN,NaN,NaN,NaN,NaN,3206374,1,0,inactive,2017-11-03,NaN,charivari,https://www.kickstarter.com/projects/1454907110/charivari/rewards,https://www.kickstarter.com/projects/1454907110/charivari/posts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-29,NaN


In [ ]:
# Delete the 3 projects with blurb=NaN, since they look like outliers
print("before: {}".format(kickstarter_berkeley_df.shape))
remove_indices = kickstarter_berkeley_df[kickstarter_berkeley_df.blurb.isna()==True].index
kickstarter_berkeley_df.drop(index=remove_indices, inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))
print(kickstarter_berkeley_df.blurb.isna().sum())

before: (366910, 49)
after: (366907, 49)
0


Result: keep attribute

##### disable_communication:

In [ ]:
kickstarter_berkeley_df.disable_communication.isna().sum()

0

In [ ]:
kickstarter_berkeley_df.disable_communication.value_counts()

False    350525
False      8623
0          7759
Name: disable_communication, dtype: int64

In [ ]:
# Drop attribute, since contains only the value False
print("before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["disable_communication"], inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366907, 49)
after: (366907, 48)


##### photo_url:

In [ ]:
kickstarter_berkeley_df.photo_url.isna().sum()

94

In [ ]:
kickstarter_berkeley_df[kickstarter_berkeley_df.photo_url.isna()==True].tail()

,state,url_project,name,kickstarter_country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,backers_count,comments_count,updates_count,staff_pick,blurb,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22572931,successful,https://www.kickstarter.com/projects/imcardboard/orbit-360-a-better-way-to-take-spherical-panoramas,Orbit 360: A better way to take spherical panoramas,United States,1040005785,2165352.0,52,2016-07-14,2016-10-17,2018-03-12,2016-10-17,2016-10-17,2016-08-18,20000.000000,60955.000000,1109,424,17,0,Create and share better spherical panoramas on your smartphone with the Orbit 360 and our new super-compact virtual reality viewers.,NaN,NaN,0.8,NaN,2599598,NaN,NaN,NaN,NaN,NaN,2599598,1,0,inactive,2016-07-14,NaN,orbit-360-a-better-way-to-take-spherical-panoramas,https://www.kickstarter.com/projects/imcardboard/orbit-360-a-better-way-to-take-spherical-panoramas/rewards,https://www.kickstarter.com/projects/imcardboard/orbit-360-a-better-way-to-take-spherical-panoramas/posts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-07,424.0
886619370,failed,https://www.kickstarter.com/projects/1977143777/magnetical-educational-toys,Magnetical educational toys,Denmark,1977143777,554015.0,12,2018-04-23,2018-06-02,2018-06-15,2018-06-02,NaN,2018-04-23,49499.928000,1.843047,2,0,0,0,Magnetic educational and playful toys for children from approximately 3-15 years.,NaN,NaN,0.8,NaN,3365002,NaN,NaN,NaN,NaN,NaN,3365002,1,0,inactive,2018-04-23,NaN,magnetical-educational-toys,https://www.kickstarter.com/projects/1977143777/magnetical-educational-toys/rewards,https://www.kickstarter.com/projects/1977143777/magnetical-educational-toys/posts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-07,NaN
320459788,successful,https://www.kickstarter.com/projects/1294073501/modern-space-a-simple-print,Modern Space - A Simple Print,United States,1294073501,2379574.0,27,2018-05-13,2018-05-31,2018-06-14,2018-05-31,2018-05-31,2018-05-21,550.000000,1542.000000,62,0,4,0,Every path of every NASA spacecraft ever launched.,NaN,NaN,0.8,NaN,3380171,NaN,NaN,NaN,NaN,NaN,3380171,1,0,inactive,2018-05-13,NaN,modern-space-a-simple-print,https://www.kickstarter.com/projects/1294073501/modern-space-a-simple-print/rewards,https://www.kickstarter.com/projects/1294073501/modern-space-a-simple-print/posts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-07,NaN
1452966445,failed,https://www.kickstarter.com/projects/1737027734/i-am-working-on-my-new-2nd-cd-album,release of second album,Italy,1737027734,20150549.0,292,2018-04-09,2018-06-12,2018-07-10,2018-06-12,NaN,2018-04-13,12368.461400,0.000000,0,0,9,0,"I am a classical pianist and I am currently working on my second album, in which I would like to include other Chopin's Mazurkas and al",NaN,NaN,0.8,NaN,3353989,NaN,NaN,NaN,NaN,NaN,3353989,1,0,inactive,2018-04-09,NaN,i-am-working-on-my-new-2nd-cd-album,https://www.kickstarter.com/projects/1737027734/i-am-working-on-my-new-2nd-cd-album/rewards,https://www.kickstarter.com/projects/1737027734/i-am-working-on-my-new-2nd-cd-album/posts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-07,NaN
1549739337,failed,https://www.kickstarter.com/projects/loretlail/wizzard-of-ankh-morpork,Wizzard of Ankh-Morpork (discworld pins),France,1935476002,591892.0,262,2018-02-14,2018-06-03,2018-06-28,2018-06-03,NaN,2018-05-04,419.317353,253.986511,13,1,2,0,A pins about a forgotten guy in a distant world…\r\nSome discwolrd's pins. (Un pins du disque monde),NaN,NaN,0.8,NaN,3309980,NaN,NaN,NaN,NaN,NaN

Result:
- delete this attribute, since my models will not consider image data
- the only option would be to see if an image is available at all; however, only 94 projects don't have an available image and they were disabled by Kickstarter (privacy request from creator, intellectual property issues, etc.); so, I assume, each project has an image 

In [ ]:
# Drop attribute photo_url
print("before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["photo_url"], inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366907, 48)
after: (366907, 47)


##### Profile-Related Attributes:

In [ ]:
# Drop attribute, since it's only style-related
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_background_color.isna()==False].state.value_counts()

successful    12336
Name: state, dtype: int64

In [ ]:
# Drop attribute, since it's only style-related
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_background_image_opacity.isna()==False].state.value_counts()

failed        211777
successful    155130
Name: state, dtype: int64

In [ ]:
# Drop attribute, since profile_blurb is only non-empty if the blurb was changed after the campaign has been successful
# Plus, mostly the blurbs are anyway very similar
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_blurb.isna()==False].state.value_counts()

successful    36602
Name: state, dtype: int64

In [ ]:
tmp = kickstarter_berkeley_df[(kickstarter_berkeley_df.profile_blurb.isna()==False) &
                            (kickstarter_berkeley_df.profile_blurb != kickstarter_berkeley_df.blurb)]
print(len(tmp))
print(tmp.profile_state.value_counts())
tmp[["name", "url_project", "blurb", "profile_blurb", "profile_state"]].head()

8153
active    8153
Name: profile_state, dtype: int64


,name,url_project,blurb,profile_blurb,profile_state
id,,,,,
22810934,"Lights, Coma, Action!",https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,"A book to tell the story of who I was (LIGHTS), how I sustained a serious brain injury (COMA), and my strong recovery (ACTION!).","A book to tell the story of how I sustained a serious brain injury and made a strong recovery (LIGHTS, COMA, ACTION!).",active
23046745,Survival,https://www.kickstarter.com/projects/1898454501/survival,Help me and friends make a survival horror short film about 3 campers and 3 cons fighting one another for survival in the Oregon woods.,Help me and friends make a survival horror short film about a camper and 3 cons fighting one another for survival in the Oregon woods.,active
23073872,Rubbee - The electric drive for bicycles,https://www.kickstarter.com/projects/1732543648/rubbee-the-electric-drive-for-bicycles,Rubbee is a revolutionary electric drive which can be mounted on any bicycle in just a few seconds.,Rubbee is a revolutionary electric drive which can be mounted on any bicycle in just a few seconds.\nMore info www.rubbee.co.uk,active
23510026,Buried Treasure - My First Film,https://www.kickstarter.com/projects/662087238/buried-treasure-my-first-film,Fortunes shift for a man stricken by tragedy when he unwittingly stumbles upon hidden treasure.,"A short film directed by Leslie Hope and written by Jeff Galfer. Starring Eloise Mumford, Gregg Henry, Crista Flanagan, Scott Klace, and Jeff Galfer.",active
24672857,MORE THAN 4 HOURS... a short film about lasting too long,https://www.kickstarter.com/projects/771136645/more-than-4-hours-a-short-film-about-lasting-too-l,"On his first day on the job, a geometry teacher mixes up his medications to potentially disastrous results.","On his first day on the job, a high school algebra teacher mixes up his medications to disastrous results.",active


In [ ]:
# Drop attribute since it does not give any explanatory value
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_id.isna()==False].state.value_counts()

failed        211777
successful    155130
Name: state, dtype: int64

In [ ]:
# Drop attribute, since it's only style-related
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_link_background_color.isna()==False].state.value_counts()

successful    14631
Name: state, dtype: int64

In [ ]:
# Drop attribute, since the external link is only available after the campaign has been already successful
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_link_text.isna()==False].state.value_counts()

successful    35872
Name: state, dtype: int64

In [ ]:
kickstarter_berkeley_df.profile_link_text.value_counts()

Follow along!                13001
Buy Now                        481
Order Now                      435
Buy Now!                       370
Order Now!                     333
                             ...  
PRE-ORDER POPPIES!               1
More about us!                   1
tundragear.com                   1
You Late? Go Here NOW <==        1
LBM Available Here!              1
Name: profile_link_text, Length: 10880, dtype: int64

In [ ]:
# Drop attribute, since it's only style-related
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_link_text_color.isna()==False].state.value_counts()

successful    2629
Name: state, dtype: int64

In [ ]:
# Drop attribute, since the external link is only available after the campaign has been already successful
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_link_url.isna()==False].state.value_counts()

successful    33755
Name: state, dtype: int64

In [ ]:
# Drop attribute, since profile_name is only non-empty if the name was changed after the campaign has been successful
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_name.isna()==False].state.value_counts()

successful    36809
Name: state, dtype: int64

In [ ]:
tmp = kickstarter_berkeley_df[(kickstarter_berkeley_df.profile_name.isna()==False) &
                            (kickstarter_berkeley_df.profile_name != kickstarter_berkeley_df.name)]
print(len(tmp))
print(tmp.profile_state.value_counts())
tmp[["name", "url_project", "name", "profile_name", "profile_state"]].head()

4932
active    4932
Name: profile_state, dtype: int64


,name,url_project,name,profile_name,profile_state
id,,,,,
23073872,Rubbee - The electric drive for bicycles,https://www.kickstarter.com/projects/1732543648/rubbee-the-electric-drive-for-bicycles,Rubbee - The electric drive for bicycles,Rubbee - The electric drive for bicycles.,active
23510026,Buried Treasure - My First Film,https://www.kickstarter.com/projects/662087238/buried-treasure-my-first-film,Buried Treasure - My First Film,Buried Treasure,active
25220776,AXED,https://www.kickstarter.com/projects/jking/axed,AXED,BLUE (Formerly AXED),active
27123691,Mages - FINAL DAY!,https://www.kickstarter.com/projects/normalgamesco/mages-0,Mages - FINAL DAY!,Mages by Normal Games Co.,active
28499509,Sean's Northern Light Photography Tours,https://www.kickstarter.com/projects/seannorman/seans-northern-light-photography-tours,Sean's Northern Light Photography Tours,Sean's Northern Light Photo Tours,active


In [ ]:
# Drop attribute, since redundant to profile_id
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_project_id.isna()==False].state.value_counts()

failed        211777
successful    155130
Name: state, dtype: int64

In [ ]:
len(kickstarter_berkeley_df[kickstarter_berkeley_df.profile_id != kickstarter_berkeley_df.profile_project_id])

0

In [ ]:
# Drop attribute, since it is style-related
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_should_show_feature_image_section.isna()==False].state.value_counts()

failed        210872
successful    154387
Name: state, dtype: int64

In [ ]:
# Clean up values
kickstarter_berkeley_df["profile_should_show_feature_image_section"] = kickstarter_berkeley_df.profile_should_show_feature_image_section.apply(
                                                                        lambda x: True if x=="True" else
                                                                        (True if x=="1" else
                                                                        (False if x=="False" else
                                                                        (False if x=="0" else x))))
kickstarter_berkeley_df.profile_should_show_feature_image_section.value_counts()

True     362887
False      2372
Name: profile_should_show_feature_image_section, dtype: int64

In [ ]:
kickstarter_berkeley_df.profile_should_show_feature_image_section.isna().sum()

1648

In [ ]:
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_should_show_feature_image_section==False].state.value_counts()

successful    2372
Name: state, dtype: int64

In [ ]:
# Drop feature, since it's only style-related after the project was successful
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_show_feature_image.isna()==False].state.value_counts()

failed        211777
successful    155130
Name: state, dtype: int64

In [ ]:
# Clean up values
kickstarter_berkeley_df["profile_show_feature_image"] = kickstarter_berkeley_df.profile_show_feature_image.apply(
                                                                        lambda x: True if x=="True" else
                                                                        (True if x=="1" else
                                                                        (False if x=="False" else
                                                                        (False if x=="0" else x))))
kickstarter_berkeley_df.profile_show_feature_image.value_counts()

False    356970
True       9937
Name: profile_show_feature_image, dtype: int64

In [ ]:
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_show_feature_image==True].state.value_counts()

successful    9937
Name: state, dtype: int64

In [ ]:
# Keep feature, since it shows if a campaign has been changed after success
# Maybe useful to identify which projects have been edited afterwards (to avoid information leak)
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_state.isna()==False].state.value_counts()

failed        211777
successful    155130
Name: state, dtype: int64

In [ ]:
kickstarter_berkeley_df.profile_state.value_counts()

inactive    330094
active       36813
Name: profile_state, dtype: int64

In [ ]:
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_state=="active"].state.value_counts()

successful    36813
Name: state, dtype: int64

In [ ]:
# Drop attribute, since it only shows when profile has been activated/deactivated
# This happened typically either after the project has been created (i.e. profile=active) or shortly after the 
# campaign has been ended (either active when project was used as spotlight or inactive)
# We only care if campaign has been changed after success (so feature profile_state is sufficient)
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_state_changed_at.isna()==False].state.value_counts()

failed        211777
successful    155130
Name: state, dtype: int64

In [ ]:
# Drop attribute, since it's only style-related
kickstarter_berkeley_df[kickstarter_berkeley_df.profile_text_color.isna()==False].state.value_counts()

successful    11950
Name: state, dtype: int64

Result:
- the profile features are only available after the project has been successful
- then the creator can edit its description, title, blurb, etc.
- thus, profile-related attributes are an information leak and cannot be used
- the only use-case for these attributes is to determine, whether a successful project has been edited afterwards (since a project can also be successful and can contain NaN profile-features)

In [ ]:
# Drop style-related profile attributes, since they don't give any explanatory value
print("before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["profile_background_color", "profile_background_image_opacity", 
                                     "profile_link_background_color", "profile_link_text_color", 
                                     "profile_text_color", "profile_should_show_feature_image_section",
                                     "profile_show_feature_image"], inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366907, 47)
after: (366907, 40)


In [ ]:
# Drop remaining profile-related attributes
print("before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["profile_blurb", "profile_link_text", "profile_link_url", "profile_name",
                                     "profile_project_id", "profile_state_changed_at"], inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366907, 40)
after: (366907, 34)


In [ ]:
# Rename profile_state to profile_has_been_changed_after_success
kickstarter_berkeley_df.profile_state.replace("active", True, inplace=True)
kickstarter_berkeley_df.profile_state.replace("inactive", False, inplace=True)
kickstarter_berkeley_df.rename(columns={"profile_state" : "profile_changed_after_success"}, inplace=True)
kickstarter_berkeley_df.profile_changed_after_success.value_counts()

False    330094
True      36813
Name: profile_changed_after_success, dtype: int64

##### slug:

In [ ]:
kickstarter_berkeley_df.slug.isna().sum()

0

In [ ]:
print(len(kickstarter_berkeley_df.slug.value_counts()))
print(kickstarter_berkeley_df.shape)

366907
(366907, 34)


In [ ]:
# Delete attribute, since I already have the project id as unique identifier
print("before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["slug"], inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366907, 34)
after: (366907, 33)


##### url_rewards / url_updates:

In [ ]:
# Delete both attributes, since I only need to project id to merge it with update and reward tables
print("before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["url_rewards", "url_updates"], inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366907, 33)
after: (366907, 31)


##### video-related features:

In [ ]:
video_df = kickstarter_berkeley_df[["video_id", "video_url_high", "video_url_webm", "video_height", "video_width", "video_status"]]
video_df.isna().sum()

video_id          103695
video_url_high    103695
video_url_webm    224859
video_height      103699
video_width       103699
video_status      103695
dtype: int64

In [ ]:
video_df.video_status.value_counts()

successful    263198
failed            12
processing         2
Name: video_status, dtype: int64

Result:
- my NN will not consider video-data
- only possibility: create one custom feature campaign_has_video and drop remaining video-related features

In [ ]:
# Create a custom feature "campaign_has_video"
kickstarter_berkeley_df["campaign_has_video"] = kickstarter_berkeley_df.video_id.apply(lambda x: False if np.isnan(x) else True)
kickstarter_berkeley_df.campaign_has_video.value_counts()

True     263212
False    103695
Name: campaign_has_video, dtype: int64

In [ ]:
# Drop all remaining video-related features
print("before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["video_id", "video_url_high", "video_url_webm", "video_height",
                                    "video_width", "video_status"], inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366907, 32)
after: (366907, 26)


In [ ]:
kickstarter_berkeley_df[kickstarter_berkeley_df.campaign_has_video==False].head()

,state,url_project,name,kickstarter_country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,backers_count,comments_count,updates_count,staff_pick,blurb,profile_id,profile_changed_after_success,file_name,last_modification,deleted_comments,campaign_has_video
id,,,,,,,,,,,,,,,,,,,,,,,,,,
22823613,failed,https://www.kickstarter.com/projects/1237543205/brainade,Brainade,United States,1237543205,2497409.0,342,2017-02-02,2017-03-17,2017-03-17,2017-03-17,NaN,2017-02-15,750.00000,1.000000,1,0,0,False,We intend to create an online platform for individuals to teach or learn a skill via online classrooms and forums.,2862898,False,www.kickstarter.com/projects/1237543205/brainade,2017-04-12,NaN,False
22850049,failed,https://www.kickstarter.com/projects/912297962/masquerading-masks,Masquerading Masks,United States,912297962,2391446.0,1,2014-08-04,2014-09-04,2014-09-04,2014-09-04,NaN,2014-08-05,1500.00000,0.000000,0,0,0,False,I love to make masks. They can be used for Halloween costumes or wall art. All are hand painted and decorated by me to order.,1257468,False,www.kickstarter.com/projects/912297962/masquerading-masks,2017-03-11,NaN,False
22885939,failed,https://www.kickstarter.com/projects/2067689029/african-american-male-fashion,African American Male Fashion,United States,2067689029,2503418.0,15,2012-01-17,2012-04-30,2016-05-19,2012-04-30,NaN,2012-03-21,6000.00000,100.000000,1,0,2,False,Senior Photo Project about the representation of African American men through clothing,71205,False,www.kickstarter.com/projects/2067689029/african-american-male-fashion,2017-03-11,NaN,False
22954941,failed,https://www.kickstarter.com/projects/1201567024/material-design-templates-for-ionic-framework,Material Design Templates for the Ionic Framework,United Kingdom,1201567024,12602158.0,51,2015-07-27,2015-08-31,2016-05-20,2015-08-31,NaN,2015-07-29,3113.73702,745.740016,11,3,2,False,Help create a series of application templates for the Ionic Framework using Material Design.,2036663,False,www.kickstarter.com/projects/1201567024/material-design-templates-for-ionic-framework,2018-05-24,0.0,False
22965058,failed,https://www.kickstarter.com/projects/614494142/wooden-creations,wooden creations,United States,614494142,2379772.0,356,2014-07-07,2014-08-07,2014-08-07,2014-08-07,NaN,2014-07-08,1000.00000,5.000000,1,0,0,False,I want to build some wooden crafts for the yard and inside the home.,1107414,False,www.kickstarter.com/projects/614494142/wooden-creations,2017-03-11,NaN,False


##### file_name:

In [ ]:
kickstarter_berkeley_df.file_name.isna().sum()

96141

In [ ]:
tmp = kickstarter_berkeley_df[kickstarter_berkeley_df.url_project!=kickstarter_berkeley_df.file_name]
tmp[["url_project", "file_name"]].head()

,url_project,file_name
id,,
22810934,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,www.kickstarter.com/projects/cavinbounce/lights-coma-action
22821161,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference
22823613,https://www.kickstarter.com/projects/1237543205/brainade,www.kickstarter.com/projects/1237543205/brainade
22826244,https://www.kickstarter.com/projects/impkingcomics/horizon-anthology,www.kickstarter.com/projects/impkingcomics/horizon-anthology
22835897,https://www.kickstarter.com/projects/alphabetstory/the-alphabet-story-childrens-book,www.kickstarter.com/projects/alphabetstory/the-alphabet-story-childrens-book


In [ ]:
# Drop attribute, since it is redundant to url_project (just without the https)
print("before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["file_name"], inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366907, 26)
after: (366907, 25)


##### last_modification:

In [ ]:
kickstarter_berkeley_df.last_modification.isna().sum()

0

Result: keep for now

##### deleted_comments:

In [ ]:
kickstarter_berkeley_df.deleted_comments.isna().sum()

234796

In [ ]:
kickstarter_berkeley_df.deleted_comments.value_counts()

 0.0       120197
 1.0         2252
 2.0         1592
-1.0          802
 3.0          781
            ...  
 646.0          1
 1686.0         1
 660.0          1
 419.0          1
 739.0          1
Name: deleted_comments, Length: 552, dtype: int64

In [ ]:
kickstarter_berkeley_df[(kickstarter_berkeley_df.deleted_comments.isna()==False) &
                      (kickstarter_berkeley_df.deleted_comments != 0.0)].head()

,state,url_project,name,kickstarter_country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,backers_count,comments_count,updates_count,staff_pick,blurb,profile_id,profile_changed_after_success,last_modification,deleted_comments,campaign_has_video
id,,,,,,,,,,,,,,,,,,,,,,,,,
26212166,successful,https://www.kickstarter.com/projects/1429072448/lost-ages-of-civilization-art-photobook,Lost Ages of Civilization - ART/PhotoBook,United States,1429072448,2475687.0,15,2010-11-03,2010-12-03,2014-07-21,2010-12-03,2010-12-03,2010-11-03,1700.00000,1945.000000,19,8,19,False,I've wanted to make a book of my art for a long time now....owners of my art have often suggested it. WITH YOUR HELP...I CAN DO IT!!,15219,False,2018-05-24,1.0,True
47519781,failed,https://www.kickstarter.com/projects/newgrimmcity/mother-mary,Mother Mary,United States,738417779,2358820.0,32,2012-02-14,2012-05-07,2016-05-19,2012-05-07,NaN,2012-04-05,8000.00000,260.000000,4,1,0,False,Mother Mary is a short film based on the work of A. N. Calaway.,78217,False,2018-05-29,1.0,False
69267875,failed,https://www.kickstarter.com/projects/1068269572/trans-zion-trek-the-documentary,Trans-Zion Trek: The Documentary,United States,1068269572,2498210.0,11,2013-05-06,2013-06-12,2016-02-14,2013-06-12,NaN,2013-05-13,2500.00000,231.000000,15,3,1,False,Video-documenting every single step of the 48.3 miles that make up the Trans-Zion Trek in Zion National Park.,559312,False,2018-05-29,3.0,True
104818455,failed,https://www.kickstarter.com/projects/1520079396/channel-19-in-hd,Channel 19 in HD,Sweden,1520079396,889278.0,292,2014-09-24,2014-11-19,2016-05-20,2014-11-19,NaN,2014-10-20,1532.11729,497.938119,19,3,4,True,A webseries about a TV-station in decline with an unusual manager.,1377211,False,2018-05-29,3.0,True
115620934,successful,https://www.kickstarter.com/projects/maughangl/create-a-home-alternative-comedy-in-philly,Create a Home for Alternative Comedy in Philly,United States,676135798,2471217.0,24,2010-04-27,2010-06-27,2016-10-11,2010-06-27,2010-06-27,2010-05-13,10000.00000,15420.510000,233,11,1,False,"Philly is the 5th largest city in the US but lacks a permanent home for its exploding alternative comedy scene. With your help, PHIT will change that.",5547,False,2018-05-24,1.0,True


In [ ]:
# Drop attribute, since the number of deleted comments by Kickstarter cannot be determined at the start of the campaign
print("before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["deleted_comments"], inplace=True)
print("after: {}".format(kickstarter_berkeley_df.shape))

before: (366907, 25)
after: (366907, 24)


##### Finalize data cleaning of table Project:

In [ ]:
kickstarter_berkeley_df.head(1)

,state,url_project,name,kickstarter_country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,backers_count,comments_count,updates_count,staff_pick,blurb,profile_id,profile_changed_after_success,last_modification,campaign_has_video
id,,,,,,,,,,,,,,,,,,,,,,,,
22810934,successful,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,"Lights, Coma, Action!",United States,1157451606,2357536.0,48,2013-10-01,2013-11-02,2017-03-01,2013-11-02,2013-11-02,2013-10-03,15000.0,18167.88,284,4,40,False,"A book to tell the story of who I was (LIGHTS), how I sustained a serious brain injury (COMA), and my strong recovery (ACTION!).",716409,True,2018-05-24,True


In [ ]:
# Rearrange columns
print("Before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df = kickstarter_berkeley_df[["url_project", "name", "blurb", "state", "goal", "pledged", 
                                                 "backers_count", "comments_count", "updates_count", "staff_pick",
                                                "campaign_has_video","kickstarter_country", "created_at", "launched_at", 
                                                 "state_changed_at", "successful_at", "deadline", "updated_at", "last_modification",
                                                 "profile_changed_after_success", "creator_id", "location_id",
                                                 "category_id", "profile_id"]]
print("After: {}".format(kickstarter_berkeley_df.shape))

Before: (366907, 24)
After: (366907, 24)


In [ ]:
kickstarter_berkeley_df.staff_pick.value_counts()

False    299032
True      35107
0         15214
False     14946
True       1435
1          1173
Name: staff_pick, dtype: int64

In [ ]:
# Clean up values for staff_pick
kickstarter_berkeley_df["staff_pick"] = kickstarter_berkeley_df.staff_pick.apply(
                                                                        lambda x: "True" if x==True else
                                                                        ("True" if x=="1" else
                                                                        ("False" if x==False else
                                                                        ("False" if x=="0" else x))))
kickstarter_berkeley_df.staff_pick.value_counts()

False    329192
True      37715
Name: staff_pick, dtype: int64

#### Preliminary Data Cleaning - Table: Category

In [ ]:
# Retrieve category table from Berkeley database
category_df = pd.read_sql_query("SELECT * FROM category", conn)
category_df.set_index("id", inplace=True)
print(category_df.shape)
category_df.head()

(169, 6)


,name,parent_id,position,color,slug,urls_web_discover
id,,,,,,
354,Taxidermy,26.0,12,16744876,crafts/taxidermy,http://www.kickstarter.com/discover/categories/crafts/taxidermy
347,Glass,26.0,5,16744876,crafts/glass,http://www.kickstarter.com/discover/categories/crafts/glass
352,Quilts,26.0,10,16744876,crafts/quilts,http://www.kickstarter.com/discover/categories/crafts/quilts
336,Flight,16.0,6,6526716,technology/flight,http://www.kickstarter.com/discover/categories/technology/flight
255,Residencies,6.0,2,10917369,dance/residencies,http://www.kickstarter.com/discover/categories/dance/residencies


Result:
- name = keep
- parent_id = relevant for retrieving the over-category; delete afterwards
- color = delete
- slug = delete, since id is already a unique identifier
- urls_web_discover = delete, since url to the category discover page is not relevant

In [ ]:
# Keep only relevant attributes
category_df = category_df[["name", "parent_id"]]

In [ ]:
# Create custom column with parent-category
category_df.rename(columns={"name" : "subcategory"}, inplace=True)
category_df["category"] = category_df.parent_id.apply(lambda x: x if np.isnan(x) else category_df.loc[x, "subcategory"])
category_df.head()

,subcategory,parent_id,category
id,,,
354,Taxidermy,26.0,Crafts
347,Glass,26.0,Crafts
352,Quilts,26.0,Crafts
336,Flight,16.0,Technology
255,Residencies,6.0,Dance


In [ ]:
# For categories, shift value from column "subcategory" to column "category"
category_mask = category_df.category.isna()==True
category_df.loc[category_mask, "category"] = category_df.loc[category_mask, "subcategory"]
category_df.loc[category_mask, "subcategory"] = "No Subcategory"

In [ ]:
category_df.subcategory.isna().sum()

0

In [ ]:
category_df.category.isna().sum()

0

In [ ]:
category_df[category_df.subcategory == "No Subcategory"]

,subcategory,parent_id,category
id,,,
6,No Subcategory,NaN,Dance
3,No Subcategory,NaN,Comics
13,No Subcategory,NaN,Journalism
15,No Subcategory,NaN,Photography
11,No Subcategory,NaN,Film & Video
12,No Subcategory,NaN,Games
17,No Subcategory,NaN,Theater
10,No Subcategory,NaN,Food
9,No Subcategory,NaN,Fashion


In [ ]:
# Delete parent_id, since not relevant anymore
print("Before: {}".format(category_df.shape))
category_df.drop(columns=["parent_id"], inplace=True)
print("After: {}".format(category_df.shape))
category_df.tail()

Before: (169, 3)
After: (169, 2)


,subcategory,category
id,,
337,Gadgets,Technology
355,Weaving,Crafts
22,Illustration,Art
35,Video Games,Games
16,No Subcategory,Technology


In [ ]:
# Merge category_df with kickstarter_berkeley_df
kickstarter_berkeley_df = kickstarter_berkeley_df.join(category_df, on="category_id")

In [ ]:
kickstarter_berkeley_df.head()

,url_project,name,blurb,state,goal,pledged,backers_count,comments_count,updates_count,staff_pick,campaign_has_video,kickstarter_country,created_at,launched_at,state_changed_at,successful_at,deadline,updated_at,last_modification,profile_changed_after_success,creator_id,location_id,category_id,profile_id,subcategory,category
id,,,,,,,,,,,,,,,,,,,,,,,,,,
22810934,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,"Lights, Coma, Action!","A book to tell the story of who I was (LIGHTS), how I sustained a serious brain injury (COMA), and my strong recovery (ACTION!).",successful,15000.0,18167.88,284,4,40,False,True,United States,2013-10-01,2013-10-03,2013-11-02,2013-11-02,2013-11-02,2017-03-01,2018-05-24,True,1157451606,2357536.0,48,716409,Nonfiction,Publishing
22821161,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,Sentio Golf Putters: Feel IS the difference,"Choose the feel YOU want with our patented floating-face technology. Modern design, proven performance, and a splash of color.",failed,50000.0,9707.00,41,0,4,False,True,United States,2015-06-08,2015-09-18,2015-11-02,NaN,2015-11-02,2016-05-20,2017-03-11,False,684178251,2351810.0,28,1945377,Product Design,Design
22823613,https://www.kickstarter.com/projects/1237543205/brainade,Brainade,We intend to create an online platform for individuals to teach or learn a skill via online classrooms and forums.,failed,750.0,1.00,1,0,0,False,False,United States,2017-02-02,2017-02-15,2017-03-17,NaN,2017-03-17,2017-03-17,2017-04-12,False,1237543205,2497409.0,342,2862898,Web,Technology
22826244,https://www.kickstarter.com/projects/impkingcomics/horizon-anthology,Horizon Anthology,HORIZON is a 144-page collection of fantasy comics by fifteen artists exploring what happens after the hero has won...,successful,7000.0,8955.41,286,20,16,True,True,United States,2013-09-13,2014-05-05,2014-06-04,2014-06-04,2014-06-04,2016-05-19,2018-05-24,False,2122725209,2489314.0,249,696107,Anthologies,Comics
22835897,https://www.kickstarter.com/projects/alphabetstory/the-alphabet-story-childrens-book,The Alphabet Story Children's Book,A richly illustrated children's book following a story about a boy lost in a magical dream land who meets and learns the ABCs.,successful,6000.0,6643.11,66,0,8,False,True,United States,2016-08-21,2016-08-31,2016-09-28,2016-09-28,2016-09-28,2017-02-28,2017-03-11,False,1566612915,2475687.0,46,2647051,Children's Books,Publishing


In [ ]:
# Drop attribute category_id, since not relevant anymore
print("Before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["category_id"], inplace=True)
print("After: {}".format(kickstarter_berkeley_df.shape))

Before: (366907, 29)
After: (366907, 28)


#### Preliminary Data Cleaning - Table: Location

In [ ]:
location_df = pd.read_sql_query("SELECT * FROM location", conn)
location_df.set_index("id", inplace=True)
print(location_df.shape)
location_df.head()

(23252, 12)


,displayable_name,type,name,state,short_name,is_root,country,slug,urls_api_nearby_projects,urls_web_discover,urls_web_location,localized_name
id,,,,,,,,,,,,
33,"Aurora, Canada",Town,Aurora,ON,"Aurora, Canada",False,CA,aurora-york-on,https://api.kickstarter.com/v1/discover?signature=1489096865.728a8b1a03ba39aba5aa937255b561b2580e9a77&woe_id=33,https://www.kickstarter.com/discover/places/aurora-york-on,https://www.kickstarter.com/locations/aurora-york-on,None
70,"Battleford, Canada",Town,Battleford,SK,"Battleford, Canada",False,CA,battleford-ca,https://api.kickstarter.com/v1/discover?signature=1488929708.620b6c650558c60ed950bb82e68920b38ffd623d&woe_id=70,https://www.kickstarter.com/discover/places/battleford-ca,https://www.kickstarter.com/locations/battleford-ca,None
78,"Beaconsfield, Canada",Suburb,Beaconsfield,QC,"Beaconsfield, Canada",False,CA,beaconsfield-montreal-ca,https://api.kickstarter.com/v1/discover?signature=1489068138.d8f6fcc652209d25f4c2b7250786b4ad4a003d5a&woe_id=78,https://www.kickstarter.com/discover/places/beaconsfield-montreal-ca,https://www.kickstarter.com/locations/beaconsfield-montreal-ca,None
79,"Beamsville, Canada",Town,Beamsville,ON,"Beamsville, Canada",False,CA,beamsville-ca,https://api.kickstarter.com/v1/discover?signature=1489274111.50598ccc006947de3a24a16e7bea53ddee9ef388&woe_id=79,https://www.kickstarter.com/discover/places/beamsville-ca,https://www.kickstarter.com/locations/beamsville-ca,None
90,"Beaverton, Canada",Town,Beaverton,ON,"Beaverton, Canada",False,CA,beaverton-on-ca,https://api.kickstarter.com/v1/discover?signature=1489010614.157442227244d283ba8c493c290fc2ab11ed5237&woe_id=90,https://www.kickstarter.com/discover/places/beaverton-on-ca,https://www.kickstarter.com/locations/beaverton-on-ca,None


##### displayable_name:

In [ ]:
location_df.displayable_name.isna().sum()

0

In [ ]:
location_df.displayable_name.value_counts()

Los Angeles, CA           16
Manhattan, NY             16
Brooklyn, NY              14
Chicago, IL                8
San Diego, CA              7
                          ..
Newport, RI                1
Dalmine, Italy             1
Kolbjørnsvik, Norway       1
Nelson, NH                 1
Hoogezand, Netherlands     1
Name: displayable_name, Length: 22037, dtype: int64

Result: keep, since that is the attribute, how it is displayed on the Kickstarter website; however, probably too granular for the final dataset (over 20,000 different values)

In [ ]:
# Rename column "displayable_name" to "location_name"
location_df.rename(columns={"displayable_name" : "location_name"}, inplace=True)
location_df.head()

,location_name,type,name,state,short_name,is_root,country,slug,urls_api_nearby_projects,urls_web_discover,urls_web_location,localized_name
id,,,,,,,,,,,,
33,"Aurora, Canada",Town,Aurora,ON,"Aurora, Canada",False,CA,aurora-york-on,https://api.kickstarter.com/v1/discover?signature=1489096865.728a8b1a03ba39aba5aa937255b561b2580e9a77&woe_id=33,https://www.kickstarter.com/discover/places/aurora-york-on,https://www.kickstarter.com/locations/aurora-york-on,None
70,"Battleford, Canada",Town,Battleford,SK,"Battleford, Canada",False,CA,battleford-ca,https://api.kickstarter.com/v1/discover?signature=1488929708.620b6c650558c60ed950bb82e68920b38ffd623d&woe_id=70,https://www.kickstarter.com/discover/places/battleford-ca,https://www.kickstarter.com/locations/battleford-ca,None
78,"Beaconsfield, Canada",Suburb,Beaconsfield,QC,"Beaconsfield, Canada",False,CA,beaconsfield-montreal-ca,https://api.kickstarter.com/v1/discover?signature=1489068138.d8f6fcc652209d25f4c2b7250786b4ad4a003d5a&woe_id=78,https://www.kickstarter.com/discover/places/beaconsfield-montreal-ca,https://www.kickstarter.com/locations/beaconsfield-montreal-ca,None
79,"Beamsville, Canada",Town,Beamsville,ON,"Beamsville, Canada",False,CA,beamsville-ca,https://api.kickstarter.com/v1/discover?signature=1489274111.50598ccc006947de3a24a16e7bea53ddee9ef388&woe_id=79,https://www.kickstarter.com/discover/places/beamsville-ca,https://www.kickstarter.com/locations/beamsville-ca,None
90,"Beaverton, Canada",Town,Beaverton,ON,"Beaverton, Canada",False,CA,beaverton-on-ca,https://api.kickstarter.com/v1/discover?signature=1489010614.157442227244d283ba8c493c290fc2ab11ed5237&woe_id=90,https://www.kickstarter.com/discover/places/beaverton-on-ca,https://www.kickstarter.com/locations/beaverton-on-ca,None


##### type:

In [ ]:
location_df.type.isna().sum()

0

In [ ]:
location_df.type.value_counts()

Town             19623
Suburb            2088
LocalAdmin         756
Zip                688
County              81
Miscellaneous        9
Estate               5
Island               1
Country              1
Name: type, dtype: int64

In [ ]:
location_df[location_df.type=="Zip"].head(1)

,location_name,type,name,state,short_name,is_root,country,slug,urls_api_nearby_projects,urls_web_discover,urls_web_location,localized_name
id,,,,,,,,,,,,
12758466,"Westminster, MA",Zip,01473,MA,"Westminster, MA",False,US,None,https://api.kickstarter.com/v1/discover?signature=1489012209.8afda97da769f95bf8cb8c8f1c03df969e461cb0&woe_id=12758466,https://www.kickstarter.com/discover/places/12758466,https://www.kickstarter.com/locations/12758466,None


Result: delete; attribute only shows how the attribute "name" is stored; since I am using displayable_name anyway, this feature is not relevant for me

In [ ]:
# Drop attribute "type"
print("Before: {}".format(location_df.shape))
location_df.drop(columns=["type"], inplace=True)
print("After: {}".format(location_df.shape))

Before: (23252, 12)
After: (23252, 11)


##### name:

In [ ]:
location_df.name.isna().sum()

0

In [ ]:
location_df.name.value_counts()

Clinton            17
Franklin           16
Marion             15
Madison            15
Monroe             15
                   ..
Le Teich            1
Puerto Morelos      1
Humlebæk            1
Nishinomiya-shi     1
Wylie               1
Name: name, Length: 18856, dtype: int64

In [ ]:
# Drop attribute "name", since it is redundant to displayable_name
print("Before: {}".format(location_df.shape))
location_df.drop(columns=["name"], inplace=True)
print("After: {}".format(location_df.shape))

Before: (23252, 11)
After: (23252, 10)


In [ ]:
location_df.head()

,location_name,state,short_name,is_root,country,slug,urls_api_nearby_projects,urls_web_discover,urls_web_location,localized_name
id,,,,,,,,,,
33,"Aurora, Canada",ON,"Aurora, Canada",False,CA,aurora-york-on,https://api.kickstarter.com/v1/discover?signature=1489096865.728a8b1a03ba39aba5aa937255b561b2580e9a77&woe_id=33,https://www.kickstarter.com/discover/places/aurora-york-on,https://www.kickstarter.com/locations/aurora-york-on,None
70,"Battleford, Canada",SK,"Battleford, Canada",False,CA,battleford-ca,https://api.kickstarter.com/v1/discover?signature=1488929708.620b6c650558c60ed950bb82e68920b38ffd623d&woe_id=70,https://www.kickstarter.com/discover/places/battleford-ca,https://www.kickstarter.com/locations/battleford-ca,None
78,"Beaconsfield, Canada",QC,"Beaconsfield, Canada",False,CA,beaconsfield-montreal-ca,https://api.kickstarter.com/v1/discover?signature=1489068138.d8f6fcc652209d25f4c2b7250786b4ad4a003d5a&woe_id=78,https://www.kickstarter.com/discover/places/beaconsfield-montreal-ca,https://www.kickstarter.com/locations/beaconsfield-montreal-ca,None
79,"Beamsville, Canada",ON,"Beamsville, Canada",False,CA,beamsville-ca,https://api.kickstarter.com/v1/discover?signature=1489274111.50598ccc006947de3a24a16e7bea53ddee9ef388&woe_id=79,https://www.kickstarter.com/discover/places/beamsville-ca,https://www.kickstarter.com/locations/beamsville-ca,None
90,"Beaverton, Canada",ON,"Beaverton, Canada",False,CA,beaverton-on-ca,https://api.kickstarter.com/v1/discover?signature=1489010614.157442227244d283ba8c493c290fc2ab11ed5237&woe_id=90,https://www.kickstarter.com/discover/places/beaverton-on-ca,https://www.kickstarter.com/locations/beaverton-on-ca,None


##### state:

In [ ]:
location_df.state.isna().sum()

27

In [ ]:
location_df[location_df.state.isna()==True].head()

,location_name,state,short_name,is_root,country,slug,urls_api_nearby_projects,urls_web_discover,urls_web_location,localized_name
id,,,,,,,,,,
151844,"Australie, Curacao",None,"Australie, Curacao",False,CW,australie-cw,https://api.kickstarter.com/v1/discover?signature=1489069798.8514c8452bcb6ba0474f7e6d7757698861b11c90&woe_id=151844,https://www.kickstarter.com/discover/places/australie-cw,https://www.kickstarter.com/locations/australie-cw,None
1887901,"Macau, Macao",None,"Macau, Macao",False,MO,macau-mo,https://api.kickstarter.com/v1/discover?signature=1488971228.1eae51843d77773f7645648da7e2911927a66df1&woe_id=1887901,https://www.kickstarter.com/discover/places/macau-mo,https://www.kickstarter.com/locations/macau-mo,None
2344254,"Marmaris, Turkey",None,"Marmaris, Turkey",False,TR,marmaris-tr,https://api.kickstarter.com/v1/discover?signature=1489073844.c3350707c8d0b975cb64795bd4e3483b1af86aef&woe_id=2344254,https://www.kickstarter.com/discover/places/marmaris-tr,https://www.kickstarter.com/locations/marmaris-tr,None
2348434,"Douglas, NZ",None,"Douglas, NZ",False,NZ,douglas-nz,https://api.kickstarter.com/v1/discover?signature=1489097171.c5feb4bc68a08858885f79e79db69ac8ab849cf6&woe_id=2348434,https://www.kickstarter.com/discover/places/douglas-nz,https://www.kickstarter.com/locations/douglas-nz,None
2350400,"Reporoa, NZ",None,"Reporoa, NZ",False,NZ,reporoa-nz,https://api.kickstarter.com/v1/discover?signature=1489284445.dd43d1eb1046ac964c69e6eddd84a28d2f95ff03&woe_id=2350400,https://www.kickstarter.com/discover/places/reporoa-nz,https://www.kickstarter.com/locations/reporoa-nz,None


In [ ]:
# Fill in the value "No State" for locations where a state is not specified
print("Before: {}".format(location_df.state.isna().sum()))
location_df.loc[location_df.state.isna()==True, "state"] = "No State"
print("After: {}".format(location_df.state.isna().sum()))

Before: 27
After: 0


In [ ]:
location_df[location_df.state.str.len()<=3].state.value_counts()

Sud    4
Tak    2
Imo    1
Nan    1
Goa    1
Vas    1
Name: state, dtype: int64

In [ ]:
# Replace US State Codes with Full Names
import us
def replace_us_state_code(x):
    us_state = us.states.lookup(x)
    if us_state is None:
        return x
    else:
        return us_state.name


location_df.state = location_df.state.apply(lambda x: replace_us_state_code(x))

In [ ]:
# Replace state codes for Canada and Australia
location_df.loc[location_df.state=="ON", "state"] = "Ontario"
location_df.loc[location_df.state=="QC", "state"] = "Quebec"
location_df.loc[location_df.state=="BC", "state"] = "British Columbia"
location_df.loc[location_df.state=="AB", "state"] = "Alberta"
location_df.loc[location_df.state=="NS", "state"] = "Nova Scotia"
location_df.loc[location_df.state=="MB", "state"] = "Manitoba"
location_df.loc[location_df.state=="SA", "state"] = "South Australia"
location_df.loc[location_df.state=="NSW", "state"] = "New South Wales"
location_df.loc[location_df.state=="VIC", "state"] = "Victoria"
location_df.loc[location_df.state=="QLD", "state"] = "Queensland"
location_df.loc[location_df.state=="ACT", "state"] = "Australian Capital Territory"
location_df.loc[location_df.state=="SK", "state"] = "Saskatchewan"
location_df.loc[location_df.state=="NB", "state"] = "New Brunswick"
location_df.loc[location_df.state=="TAS", "state"] = "Tasmania"
location_df.loc[location_df.state=="PE", "state"] = "Prince Edward Island"
location_df.loc[location_df.state=="NL", "state"] = "Newfoundland and Labrador"
location_df.loc[location_df.state=="NT", "state"] = "Northwest Territories"
location_df.loc[location_df.state=="NU", "state"] = "Nunavut"

In [ ]:
location_df.rename(columns={"state" : "location_state"}, inplace=True)

In [ ]:
location_df.head()

,location_name,location_state,country,slug,urls_api_nearby_projects,urls_web_discover,urls_web_location,localized_name
id,,,,,,,,
33,"Aurora, Canada",Ontario,CA,aurora-york-on,https://api.kickstarter.com/v1/discover?signature=1489096865.728a8b1a03ba39aba5aa937255b561b2580e9a77&woe_id=33,https://www.kickstarter.com/discover/places/aurora-york-on,https://www.kickstarter.com/locations/aurora-york-on,None
70,"Battleford, Canada",Saskatchewan,CA,battleford-ca,https://api.kickstarter.com/v1/discover?signature=1488929708.620b6c650558c60ed950bb82e68920b38ffd623d&woe_id=70,https://www.kickstarter.com/discover/places/battleford-ca,https://www.kickstarter.com/locations/battleford-ca,None
78,"Beaconsfield, Canada",Quebec,CA,beaconsfield-montreal-ca,https://api.kickstarter.com/v1/discover?signature=1489068138.d8f6fcc652209d25f4c2b7250786b4ad4a003d5a&woe_id=78,https://www.kickstarter.com/discover/places/beaconsfield-montreal-ca,https://www.kickstarter.com/locations/beaconsfield-montreal-ca,None
79,"Beamsville, Canada",Ontario,CA,beamsville-ca,https://api.kickstarter.com/v1/discover?signature=1489274111.50598ccc006947de3a24a16e7bea53ddee9ef388&woe_id=79,https://www.kickstarter.com/discover/places/beamsville-ca,https://www.kickstarter.com/locations/beamsville-ca,None
90,"Beaverton, Canada",Ontario,CA,beaverton-on-ca,https://api.kickstarter.com/v1/discover?signature=1489010614.157442227244d283ba8c493c290fc2ab11ed5237&woe_id=90,https://www.kickstarter.com/discover/places/beaverton-on-ca,https://www.kickstarter.com/locations/beaverton-on-ca,None


##### short_name:

In [ ]:
# Drop attribute, since it is redundant to displayable_name
print("Before: {}".format(location_df.shape))
location_df.drop(columns=["short_name"], inplace=True)
print("After: {}".format(location_df.shape))

Before: (23252, 10)
After: (23252, 9)


##### is_root:

In [ ]:
location_df.is_root.isna().sum()

0

In [ ]:
location_df.is_root.value_counts()

False    12898
0        10354
Name: is_root, dtype: int64

In [ ]:
# Drop attribute, since it contains only the value False
print("Before: {}".format(location_df.shape))
location_df.drop(columns=["is_root"], inplace=True)
print("After: {}".format(location_df.shape))

Before: (23252, 9)
After: (23252, 8)


##### country:

In [ ]:
location_df.country.isna().sum()

0

In [ ]:
location_df.country.value_counts()

US    13574
GB     1677
CA      964
DE      866
AU      661
      ...  
AD        1
CG        1
XK        1
KP        1
BM        1
Name: country, Length: 215, dtype: int64

In [ ]:
# Replace country codes with full name
import pycountry
def replace_country_code(x):
    country = pycountry.countries.get(alpha_2=x)
    if country is None:
        return x
    else:
        return country.name

location_df.country = location_df.country.apply(lambda x: replace_country_code(x))
location_df.country.value_counts()

United States             13574
United Kingdom             1677
Canada                      964
Germany                     866
Australia                   661
                          ...  
French Guiana                 1
Seychelles                    1
Solomon Islands               1
Svalbard and Jan Mayen        1
Cayman Islands                1
Name: country, Length: 215, dtype: int64

In [ ]:
# Replace country code for Kosovo
location_df.loc[location_df.country=="XK", "country"] = "Kosovo"

In [ ]:
# Rename column "country" to "location_country"
location_df.rename(columns={"country" : "location_country"}, inplace=True)

In [ ]:
location_df.head()

,location_name,location_state,location_country,slug,urls_api_nearby_projects,urls_web_discover,urls_web_location,localized_name
id,,,,,,,,
33,"Aurora, Canada",Ontario,Canada,aurora-york-on,https://api.kickstarter.com/v1/discover?signature=1489096865.728a8b1a03ba39aba5aa937255b561b2580e9a77&woe_id=33,https://www.kickstarter.com/discover/places/aurora-york-on,https://www.kickstarter.com/locations/aurora-york-on,None
70,"Battleford, Canada",Saskatchewan,Canada,battleford-ca,https://api.kickstarter.com/v1/discover?signature=1488929708.620b6c650558c60ed950bb82e68920b38ffd623d&woe_id=70,https://www.kickstarter.com/discover/places/battleford-ca,https://www.kickstarter.com/locations/battleford-ca,None
78,"Beaconsfield, Canada",Quebec,Canada,beaconsfield-montreal-ca,https://api.kickstarter.com/v1/discover?signature=1489068138.d8f6fcc652209d25f4c2b7250786b4ad4a003d5a&woe_id=78,https://www.kickstarter.com/discover/places/beaconsfield-montreal-ca,https://www.kickstarter.com/locations/beaconsfield-montreal-ca,None
79,"Beamsville, Canada",Ontario,Canada,beamsville-ca,https://api.kickstarter.com/v1/discover?signature=1489274111.50598ccc006947de3a24a16e7bea53ddee9ef388&woe_id=79,https://www.kickstarter.com/discover/places/beamsville-ca,https://www.kickstarter.com/locations/beamsville-ca,None
90,"Beaverton, Canada",Ontario,Canada,beaverton-on-ca,https://api.kickstarter.com/v1/discover?signature=1489010614.157442227244d283ba8c493c290fc2ab11ed5237&woe_id=90,https://www.kickstarter.com/discover/places/beaverton-on-ca,https://www.kickstarter.com/locations/beaverton-on-ca,None


##### slug:

In [ ]:
# Delete attribute, since id is already a unique identifier
print("Before: {}".format(location_df.shape))
location_df.drop(columns=["slug"], inplace=True)
print("After: {}".format(location_df.shape))

Before: (23252, 8)
After: (23252, 7)


##### urls:

In [ ]:
# Delete attributes, since url to the location is not relevant for my purpose
print("Before: {}".format(location_df.shape))
location_df.drop(columns=["urls_api_nearby_projects", "urls_web_discover", "urls_web_location"], inplace=True)
print("After: {}".format(location_df.shape))

Before: (23252, 7)
After: (23252, 4)


##### localized_name:

In [ ]:
location_df.localized_name.isna().sum()

12898

In [ ]:
location_df[location_df.localized_name.isna()==False].head()

,location_name,location_state,location_country,localized_name
id,,,,
23415324,"Vigo, Spain",Galicia,Spain,Vigo
10288,"Abertillery, UK",Wales,United Kingdom,Abertillery
552759,"Christiansfeld, Denmark",Syddanmark,Denmark,Christiansfeld
948092,"Corfu, Greece",Peloponnese Western Greece and the Ionian Islands,Greece,Corfu
45082,"Portaferry, UK",Northern Ireland,United Kingdom,Portaferry


In [ ]:
# Drop attribute, since redundant with displayable_name
print("Before: {}".format(location_df.shape))
location_df.drop(columns=["localized_name"], inplace=True)
print("After: {}".format(location_df.shape))

Before: (23252, 4)
After: (23252, 3)


In [ ]:
location_df.head()

,location_name,location_state,location_country
id,,,
33,"Aurora, Canada",Ontario,Canada
70,"Battleford, Canada",Saskatchewan,Canada
78,"Beaconsfield, Canada",Quebec,Canada
79,"Beamsville, Canada",Ontario,Canada
90,"Beaverton, Canada",Ontario,Canada


##### Merge location_df with kickstarter_berkeley_df:

In [ ]:
kickstarter_berkeley_df = kickstarter_berkeley_df.join(location_df, on="location_id")
kickstarter_berkeley_df.head()

,url_project,name,blurb,state,goal,pledged,backers_count,comments_count,updates_count,staff_pick,campaign_has_video,kickstarter_country,created_at,launched_at,state_changed_at,successful_at,deadline,updated_at,last_modification,profile_changed_after_success,creator_id,location_id,category_id,profile_id,subcategory,category,location_name,location_state,location_country
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22810934,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,"Lights, Coma, Action!","A book to tell the story of who I was (LIGHTS), how I sustained a serious brain injury (COMA), and my strong recovery (ACTION!).",successful,15000.0,18167.88,284,4,40,False,True,United States,2013-10-01,2013-10-03,2013-11-02,2013-11-02,2013-11-02,2017-03-01,2018-05-24,True,1157451606,2357536.0,48,716409,Nonfiction,Publishing,"Austin, TX",Texas,United States
22821161,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,Sentio Golf Putters: Feel IS the difference,"Choose the feel YOU want with our patented floating-face technology. Modern design, proven performance, and a splash of color.",failed,50000.0,9707.00,41,0,4,False,True,United States,2015-06-08,2015-09-18,2015-11-02,NaN,2015-11-02,2016-05-20,2017-03-11,False,684178251,2351810.0,28,1945377,Product Design,Design,"Acton, MA",Massachusetts,United States
22823613,https://www.kickstarter.com/projects/1237543205/brainade,Brainade,We intend to create an online platform for individuals to teach or learn a skill via online classrooms and forums.,failed,750.0,1.00,1,0,0,False,False,United States,2017-02-02,2017-02-15,2017-03-17,NaN,2017-03-17,2017-03-17,2017-04-12,False,1237543205,2497409.0,342,2862898,Web,Technology,"Spearfish, SD",South Dakota,United States
22826244,https://www.kickstarter.com/projects/impkingcomics/horizon-anthology,Horizon Anthology,HORIZON is a 144-page collection of fantasy comics by fifteen artists exploring what happens after the hero has won...,successful,7000.0,8955.41,286,20,16,True,True,United States,2013-09-13,2014-05-05,2014-06-04,2014-06-04,2014-06-04,2016-05-19,2018-05-24,False,2122725209,2489314.0,249,696107,Anthologies,Comics,"Savannah, GA",Georgia,United States
22835897,https://www.kickstarter.com/projects/alphabetstory/the-alphabet-story-childrens-book,The Alphabet Story Children's Book,A richly illustrated children's book following a story about a boy lost in a magical dream land who meets and learns the ABCs.,successful,6000.0,6643.11,66,0,8,False,True,United States,2016-08-21,2016-08-31,2016-09-28,2016-09-28,2016-09-28,2017-02-28,2017-03-11,False,1566612915,2475687.0,46,2647051,Children's Books,Publishing,"Portland, OR",Oregon,United States


In [ ]:
# Drop attribute location_id, since not relevant anymore
print("Before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["location_id"], inplace=True)
print("After: {}".format(kickstarter_berkeley_df.shape))

Before: (366907, 28)
After: (366907, 27)


#### Preliminary Data Cleaning - Table: Creator

In [ ]:
creator_df = pd.read_sql_query("SELECT * FROM creator", conn)
creator_df.set_index("id", inplace=True)
print(creator_df.shape)
creator_df.head()

(334841, 9)


,name,slug,avatar_thumb,avatar_small,urls_web_user,avatar_medium,is_registered,urls_api_user,chosen_currency
id,,,,,,,,,
3,Charles Adler,cadler,https://ksr-ugc.imgix.net/assets/005/760/138/4ad23108e38b1926d2de79f22fb7dcd6_original.jpeg?w=40&h=40&fit=crop&v=1461076651&auto=format&q=92&s=5e0e5e140d3933b0b2a9775acccdde1b,https://ksr-ugc.imgix.net/assets/005/760/138/4ad23108e38b1926d2de79f22fb7dcd6_original.jpeg?w=160&h=160&fit=crop&v=1461076651&auto=format&q=92&s=1892e2e78aed1e8ec0158876f0b56370,https://www.kickstarter.com/profile/cadler,https://ksr-ugc.imgix.net/assets/005/760/138/4ad23108e38b1926d2de79f22fb7dcd6_original.jpeg?w=160&h=160&fit=crop&v=1461076651&auto=format&q=92&s=1892e2e78aed1e8ec0158876f0b56370,None,https://api.kickstarter.com/v1/users/3?signature=1489093035.d5512439536e228de601f337e695968468094966,None
4,Perry Chen,perry,https://ksr-ugc.imgix.net/assets/005/760/139/e301cbd260d40057dfe38bd2927020e4_original.jpeg?w=40&h=40&fit=crop&v=1461076651&auto=format&q=92&s=8a7fb49bdedcef1afdbf11af0c1668bb,https://ksr-ugc.imgix.net/assets/005/760/139/e301cbd260d40057dfe38bd2927020e4_original.jpeg?w=160&h=160&fit=crop&v=1461076651&auto=format&q=92&s=3e5d350ff7cdcbd20b6cb03478c47760,https://www.kickstarter.com/profile/perry,https://ksr-ugc.imgix.net/assets/005/760/139/e301cbd260d40057dfe38bd2927020e4_original.jpeg?w=160&h=160&fit=crop&v=1461076651&auto=format&q=92&s=3e5d350ff7cdcbd20b6cb03478c47760,None,https://api.kickstarter.com/v1/users/4?signature=1489132788.87409d06cabef5c5e2a1e55aefe20087c1c449ec,None
11,Earl Scioneaux,Samplefreq,https://ksr-ugc.imgix.net/assets/005/760/144/041f5b5bd724d07791ea5a118433337b_original.png?w=40&h=40&fit=crop&v=1461076651&auto=format&q=92&s=71f4d2d056f0b983770cc4ee522788b6,https://ksr-ugc.imgix.net/assets/005/760/144/041f5b5bd724d07791ea5a118433337b_original.png?w=160&h=160&fit=crop&v=1461076651&auto=format&q=92&s=85995c499dbf43e61cc7d0b34830fc1c,https://www.kickstarter.com/profile/Samplefreq,https://ksr-ugc.imgix.net/assets/005/760/144/041f5b5bd724d07791ea5a118433337b_original.png?w=160&h=160&fit=crop&v=1461076651&auto=format&q=92&s=85995c499dbf43e61cc7d0b34830fc1c,None,https://api.kickstarter.com/v1/users/11?signature=1489083113.2ecacaf1f14b6cd7c1616838a3f6f1d976d6f89a,None
12806,Timothy Plain,None,https://ksr-ugc.imgix.net/assets/006/618/091/fb53d236c0bb902fb73ff020a3e586f9_original.jpg?w=40&h=40&fit=crop&v=1461400052&auto=format&q=92&s=2a2917328704aee12d35705c04616dda,https://ksr-ugc.imgix.net/assets/006/618/091/fb53d236c0bb902fb73ff020a3e586f9_original.jpg?w=160&h=160&fit=crop&v=1461400052&auto=format&q=92&s=c42c820e12790888a96512d6c6226823,https://www.kickstarter.com/profile/12806,https://ksr-ugc.imgix.net/assets/006/618/091/fb53d236c0bb902fb73ff020a3e586f9_original.jpg?w=160&h=160&fit=crop&v=1461400052&auto=format&q=92&s=c42c820e12790888a96512d6c6226823,None,https://api.kickstarter.com/v1/users/12806?signature=1488941923.0640fd1cad056a2e32516e185d412c44bb925a3a,None
13061,Sam G. Hershman,None,https://ksr-ugc.imgix.net/assets/006/925/264/cde1dfc5121179d4808d33bf6c05a071_original.jpg?w=40&h=40&fit=crop&v=1461421622&auto=format&q=92&s=ab2915acf8a8d4a47fa805f1d6a097b1,https://ksr-ugc.imgix.net/assets/006/925/264/cde1dfc5121179d4808d33bf6c05a071_original.jpg?w=160&h=160&fit=crop&v=1461421622&auto=format&q=92&s=8d2f57148e620a753a42ecea0cf3a19b,https://www.kickstarter.com/profile/13061,https://ksr-ugc.imgix.net/assets/006/925/264/cde1dfc5121179d4808d33bf6c05a071_original.jpg?w=160&h=160&fit=crop&v=1461421622&auto=format&q=92&s=8d2f57148e620a753a42ecea0cf3a19b,None,https://api.kickstarter.com/v1/users/13061?signature=1488944362.5f8fdd1d05693a2339fa10ac5807c3fb7d0d5906,None


In [ ]:
creator_df.isna().sum()

name                    0
slug               247798
avatar_thumb            0
avatar_small            0
urls_web_user           0
avatar_medium           0
is_registered      227563
urls_api_user           0
chosen_currency    334109
dtype: int64

##### name:

In [ ]:
creator_df.name.isna().sum()

0

Result: keep for now for better identification of the creator; however, not useful for the final dataset

##### slug:

In [ ]:
creator_df.slug.isna().sum()

247798

In [ ]:
# Drop attribute, since it contains too many NaN values + id is already the unique identifier
print("Before: {}".format(creator_df.shape))
creator_df.drop(columns=["slug"], inplace=True)
print("After: {}".format(creator_df.shape))

Before: (334841, 9)
After: (334841, 8)


##### avatar-related attributes:

In [ ]:
# Drop attributes, since I will not consider image data
# Plus: there are no NaN values, so I can also not check, whether a profile picture is available or not
print("Before: {}".format(creator_df.shape))
creator_df.drop(columns=["avatar_thumb", "avatar_small", "avatar_medium"], inplace=True)
print("After: {}".format(creator_df.shape))

Before: (334841, 8)
After: (334841, 5)


##### urls:

In [ ]:
# Drop attribute "urls_api_user", since redundant to "urls_web_user"
print("Before: {}".format(creator_df.shape))
creator_df.drop(columns=["urls_api_user"], inplace=True)
print("After: {}".format(creator_df.shape))

Before: (334841, 5)
After: (334841, 4)


##### is_registered:

In [ ]:
creator_df.is_registered.isna().sum()

227563

In [ ]:
creator_df.is_registered.value_counts()

1       76905
True    30373
Name: is_registered, dtype: int64

In [ ]:
# Drop attribute, since it contains only NaN and the value True
# Also, it cannot be inferred that NaN==False
print("Before: {}".format(creator_df.shape))
creator_df.drop(columns=["is_registered"], inplace=True)
print("After: {}".format(creator_df.shape))

Before: (334841, 4)
After: (334841, 3)


##### chosen_currency:

In [ ]:
creator_df.chosen_currency.isna().sum()

334109

In [ ]:
creator_df.chosen_currency.value_counts()

USD    420
EUR    134
GBP     42
CAD     33
MXN     25
JPY     18
HKD     14
AUD     14
CHF      9
NZD      7
SEK      6
DKK      6
SGD      3
NOK      1
Name: chosen_currency, dtype: int64

In [ ]:
# Drop attribute, since it contains mostly NaN values
print("Before: {}".format(creator_df.shape))
creator_df.drop(columns=["chosen_currency"], inplace=True)
print("After: {}".format(creator_df.shape))

Before: (334841, 3)
After: (334841, 2)


In [ ]:
creator_df.rename(columns={"name":"creator_name", "urls_web_user":"creator_url"}, inplace=True)

In [ ]:
creator_df.head()

,creator_name,creator_url
id,,
3,Charles Adler,https://www.kickstarter.com/profile/cadler
4,Perry Chen,https://www.kickstarter.com/profile/perry
11,Earl Scioneaux,https://www.kickstarter.com/profile/Samplefreq
12806,Timothy Plain,https://www.kickstarter.com/profile/12806
13061,Sam G. Hershman,https://www.kickstarter.com/profile/13061


#### Preliminary Data Cleaning - Table: Creator External URL

In [ ]:
creator_external_url_df = pd.read_sql_query("SELECT * FROM creator_external_url", conn)
print(creator_external_url_df.shape)
creator_external_url_df.head()

(484338, 4)


,creator_id,href,target,link_text
0,3,http://charles-adler.com,_blank,charles-adler.com
1,3,http://twitter.com/cadler,_blank,twitter.com
2,3,http://subsystence.net,_blank,subsystence.net
3,3,http://lostarts.co,_blank,lostarts.co
4,4,http://perrychen.tumblr.com,_blank,perrychen.tumblr.com


In [ ]:
# Delete target and link_text, since they don't give us any explanatory value (only the actual link is relevant)
print("Before: {}".format(creator_external_url_df.shape))
creator_external_url_df.drop(columns=["target", "link_text"], inplace=True)
print("After: {}".format(creator_external_url_df.shape))

Before: (484338, 4)
After: (484338, 2)


In [ ]:
# Drop all links that point to generic websites (e.g. facebook.com) without actually linking to the profile/website of the creator
remove_urls = ["http://kickingitforward.org", "http://kickingitforward.org/", "https://twitter.com/",
              "http://www.facebook.com", "http://facebook.com", "https://www.linkedin.com/profile/public-profile-settings?trk=prof-edit-edit-public_profile",
              "http://www.linkedin.com", "https://www.linkedin.com/profile/preview?locale=en_US&trk=prof-0-sb-preview-primary-button",
              "http://www.twitter.com", "http://www.linkedin.com/profile/edit?trk=hb_tab_pro_top", "http://twitter.com",
              "http://www.kickingitforward.org", "https://www.facebook.com/", "http://www.youtube.com", "http://Facebook.com",
              "https://www.instagram.com/", "https://vimeo.com/home/myvideos", "http://linkedin.com", "https://mobile.twitter.com/account", 
              "http://www.kickstarter.com", "https://www.linkedin.com/home?trk=nav_responsive_tab_home", "https://www.instagram.com/?hl=en",
              "http://www.amazon.com", "http://youtube.com", "http://www.instagram.com", "https://twitter.com", "http://instagram.com",
              "http://www.imdb.com", "https://www.linkedin.com/profile/preview?vpa=pub&locale=en_US", "https://twitter.com/?lang=en",
              "http://www.reverbnation.com", "http://kickstarter.com", "https://www.instagram.com", "http://Twitter.com",
              "https://www.linkedin.com/feed/", "https://plus.google.com/u/0/", "https://soundcloud.com/stream", "https://www.youtube.com/my_videos?o=U",
              "https://www.linkedin.com/nhome/?trk=", "http://www.kickingitforward.org/", "http://Youtube.com", "http://amazon.com",
              "http://Amazon.com", "http://www.linkedin.com/home?trk=nav_responsive_tab_home", "http://KickingItForward.org",
              "https://www.linkedin.com/nhome/", "http://www.etsy.com", "https://www.facebook.com/settings?tab=account&section=username",
              "https://twitter.com/?lang=en-gb", "https://m.facebook.com/profile.php?ref=bookmarks", "http://www.pinterest.com",
              "https://www.facebook.com", "http://www.google.com", "http://www.linkedin.com/", "http://kickingitforward.org/add-project",
              "http://www.google.com/webfonts", "http://www.linkedin.com/profile/edit?trk=tab_pro", "https://www.facebook.com/home.php",
              "http://Instagram.com", "http://Linkedin.com", "http://www.soundcloud.com", "https://mobile.twitter.com/home",
              "https://www.linkedin.com", "http://LinkedIn.com", "https://m.facebook.com/profile.php?ref=bookmark", "http://spotify.com",
              "http://etsy.com", "http://reverbnation.com", "http://www.Facebook.com", "http://soundcloud.com", "https://www.linkedin.com/", 
              "http://imdb.com", "http://www.itunes.com", "http://www.barnesandnoble.com", "https://kickingitforward.org",
              "https://twitter.com/?lang=it", "https://vimeo.com/home/myvideos/page:1/sort:date/format:video", "http://soundcloud.com/you/tracks"]

print("Before: {}".format(creator_external_url_df.shape))
creator_external_url_df.drop(index= creator_external_url_df[creator_external_url_df.href.isin(remove_urls)].index, inplace=True)
print("After: {}".format(creator_external_url_df.shape))

Before: (484338, 2)
After: (482138, 2)


In [ ]:
creator_external_url_df.href.value_counts()

http://ludibooster.com/                 7
http://www.comeandlive.com              5
http://www.thearcticcircle.org/#        4
http://www.ludibooster.com/             4
http://www.giochix.it                   4
                                       ..
http://www.johnlund.com                 1
http://www.chibistudiovancouver.com     1
http://etsy.com/shop/haleyvray          1
http://www.getgpskin.com                1
http://followingthebox.wordpress.com    1
Name: href, Length: 479502, dtype: int64

In [ ]:
# Create custom column in creator_df "number_of_creator_links"
print("Before: {}".format(creator_df.shape))
link_counts = creator_external_url_df.groupby(["creator_id"], as_index=False)[["href"]].count().rename(columns={"href":"number_of_creator_links"})
link_counts.set_index("creator_id", inplace=True)
creator_df = creator_df.join(other=link_counts, how="left")
print("After: {}".format(creator_df.shape))
creator_df.head()

Before: (334841, 2)
After: (334841, 3)


,creator_name,creator_url,number_of_creator_links
id,,,
3,Charles Adler,https://www.kickstarter.com/profile/cadler,4.0
4,Perry Chen,https://www.kickstarter.com/profile/perry,1.0
11,Earl Scioneaux,https://www.kickstarter.com/profile/Samplefreq,8.0
12806,Timothy Plain,https://www.kickstarter.com/profile/12806,3.0
13061,Sam G. Hershman,https://www.kickstarter.com/profile/13061,NaN


In [ ]:
creator_df.number_of_creator_links.isna().sum()

106457

In [ ]:
# Replace NaN value through 0
creator_df.number_of_creator_links.fillna(value=0, inplace=True)
print(creator_df.number_of_creator_links.isna().sum())

# Convert number_of_creator_links to int
creator_df["number_of_creator_links"] = creator_df.number_of_creator_links.astype("int64")
print(creator_df.number_of_creator_links.dtype)

0
int64


In [ ]:
# Identify interesting external websites
print("Facebook: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("facebook", case=False)])))
print("Twitter: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("twitter", case=False)])))
print("Instagram: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("instagram", case=False)])))
print("iTunes: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("itunes", case=False)])))
print("Spotify: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("spotify", case=False)])))
print("Tumblr: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("tumblr", case=False)])))
print("Pinterest: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("pinterest", case=False)])))
print("SoundCloud: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("soundcloud", case=False)])))
print("Reverbnation: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("reverbnation", case=False)])))
print("Google: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("google", case=False)])))
print("IMDB: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("imdb", case=False)])))
print("Etsy: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("etsy", case=False)])))
print("LinkedIn: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("linkedin", case=False)])))
print("YouTube: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("youtube", case=False)])))
print("Vimeo: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("vimeo", case=False)])))
print("Amazon: {}".format(len(creator_external_url_df[creator_external_url_df.href.str.contains("amazon", case=False)])))

Facebook: 83643
Twitter: 44619
Instagram: 23525
iTunes: 855
Spotify: 187
Tumblr: 6372
Pinterest: 1916
SoundCloud: 5129
Reverbnation: 4109
Google: 2061
IMDB: 3272
Etsy: 2866
LinkedIn: 7285
YouTube: 19626
Vimeo: 4002
Amazon: 1546


Result:
- chosen links: Facebook, Twitter, Instagram, Youtube, and LinkedIn
- since these websites are agnostic to specific categories and contain most of the values

In [ ]:
# Create custom column "creator_facebook_linked" and "creator_facebook_link"
print("Before: {}".format(creator_df.shape))
facebook_links = creator_external_url_df[creator_external_url_df.href.str.contains("facebook", case=False)].rename(columns={"href":"creator_facebook_links"})
facebook_links = facebook_links.groupby('creator_id').agg(lambda x: list(x))
creator_df = creator_df.join(other=facebook_links, how="left")
creator_df["creator_facebook_linked"] = creator_df.creator_facebook_links.apply(lambda x: True if isinstance(x, list) else False)
print("After: {}".format(creator_df.shape))
creator_df.head()

Before: (334841, 3)
After: (334841, 5)


,creator_name,creator_url,number_of_creator_links,creator_facebook_links,creator_facebook_linked
id,,,,,
3,Charles Adler,https://www.kickstarter.com/profile/cadler,4,NaN,False
4,Perry Chen,https://www.kickstarter.com/profile/perry,1,NaN,False
11,Earl Scioneaux,https://www.kickstarter.com/profile/Samplefreq,8,[http://www.facebook.com/maddwikkid],True
12806,Timothy Plain,https://www.kickstarter.com/profile/12806,3,[http://www.facebook.com/spindryproductions],True
13061,Sam G. Hershman,https://www.kickstarter.com/profile/13061,0,NaN,False


In [ ]:
creator_df.creator_facebook_linked.value_counts()

False    257738
True      77103
Name: creator_facebook_linked, dtype: int64

In [ ]:
# Create custom column "creator_twitter_linked" and "creator_twitter_links"
print("Before: {}".format(creator_df.shape))
twitter_links = creator_external_url_df[creator_external_url_df.href.str.contains("twitter", case=False)].rename(columns={"href":"creator_twitter_links"})
twitter_links = twitter_links.groupby('creator_id').agg(lambda x: list(x))
creator_df = creator_df.join(other=twitter_links, how="left")
creator_df["creator_twitter_linked"] = creator_df.creator_twitter_links.apply(lambda x: True if isinstance(x, list) else False)
print("After: {}".format(creator_df.shape))
creator_df.head()

Before: (334841, 5)
After: (334841, 7)


,creator_name,creator_url,number_of_creator_links,creator_facebook_links,creator_facebook_linked,creator_twitter_links,creator_twitter_linked
id,,,,,,,
3,Charles Adler,https://www.kickstarter.com/profile/cadler,4,NaN,False,[http://twitter.com/cadler],True
4,Perry Chen,https://www.kickstarter.com/profile/perry,1,NaN,False,NaN,False
11,Earl Scioneaux,https://www.kickstarter.com/profile/Samplefreq,8,[http://www.facebook.com/maddwikkid],True,[https://twitter.com/#!/mwikkid],True
12806,Timothy Plain,https://www.kickstarter.com/profile/12806,3,[http://www.facebook.com/spindryproductions],True,NaN,False
13061,Sam G. Hershman,https://www.kickstarter.com/profile/13061,0,NaN,False,NaN,False


In [ ]:
creator_df.creator_twitter_linked.value_counts()

False    292125
True      42716
Name: creator_twitter_linked, dtype: int64

In [ ]:
# Create custom column "creator_instagram_linked" and "creator_instagram_links"
print("Before: {}".format(creator_df.shape))
instagram_links = creator_external_url_df[creator_external_url_df.href.str.contains("instagram", case=False)].rename(columns={"href":"creator_instagram_links"})
instagram_links = instagram_links.groupby('creator_id').agg(lambda x: list(x))
creator_df = creator_df.join(other=instagram_links, how="left")
creator_df["creator_instagram_linked"] = creator_df.creator_instagram_links.apply(lambda x: True if isinstance(x, list) else False)
print("After: {}".format(creator_df.shape))
creator_df.head()

Before: (334841, 7)
After: (334841, 9)


,creator_name,creator_url,number_of_creator_links,creator_facebook_links,creator_facebook_linked,creator_twitter_links,creator_twitter_linked,creator_instagram_links,creator_instagram_linked
id,,,,,,,,,
3,Charles Adler,https://www.kickstarter.com/profile/cadler,4,NaN,False,[http://twitter.com/cadler],True,NaN,False
4,Perry Chen,https://www.kickstarter.com/profile/perry,1,NaN,False,NaN,False,NaN,False
11,Earl Scioneaux,https://www.kickstarter.com/profile/Samplefreq,8,[http://www.facebook.com/maddwikkid],True,[https://twitter.com/#!/mwikkid],True,NaN,False
12806,Timothy Plain,https://www.kickstarter.com/profile/12806,3,[http://www.facebook.com/spindryproductions],True,NaN,False,NaN,False
13061,Sam G. Hershman,https://www.kickstarter.com/profile/13061,0,NaN,False,NaN,False,NaN,False


In [ ]:
creator_df.creator_instagram_linked.value_counts()

False    312410
True      22431
Name: creator_instagram_linked, dtype: int64

In [ ]:
# Create custom column "creator_youtube_linked" and "creator_youtube_links"
print("Before: {}".format(creator_df.shape))
youtube_links = creator_external_url_df[creator_external_url_df.href.str.contains("youtube", case=False)].rename(columns={"href":"creator_youtube_links"})
youtube_links = youtube_links.groupby('creator_id').agg(lambda x: list(x))
creator_df = creator_df.join(other=youtube_links, how="left")
creator_df["creator_youtube_linked"] = creator_df.creator_youtube_links.apply(lambda x: True if isinstance(x, list) else False)
print("After: {}".format(creator_df.shape))
creator_df.head()

Before: (334841, 9)
After: (334841, 11)


,creator_name,creator_url,number_of_creator_links,creator_facebook_links,creator_facebook_linked,creator_twitter_links,creator_twitter_linked,creator_instagram_links,creator_instagram_linked,creator_youtube_links,creator_youtube_linked
id,,,,,,,,,,,
3,Charles Adler,https://www.kickstarter.com/profile/cadler,4,NaN,False,[http://twitter.com/cadler],True,NaN,False,NaN,False
4,Perry Chen,https://www.kickstarter.com/profile/perry,1,NaN,False,NaN,False,NaN,False,NaN,False
11,Earl Scioneaux,https://www.kickstarter.com/profile/Samplefreq,8,[http://www.facebook.com/maddwikkid],True,[https://twitter.com/#!/mwikkid],True,NaN,False,NaN,False
12806,Timothy Plain,https://www.kickstarter.com/profile/12806,3,[http://www.facebook.com/spindryproductions],True,NaN,False,NaN,False,NaN,False
13061,Sam G. Hershman,https://www.kickstarter.com/profile/13061,0,NaN,False,NaN,False,NaN,False,NaN,False


In [ ]:
creator_df.creator_youtube_linked.value_counts()

False    317200
True      17641
Name: creator_youtube_linked, dtype: int64

In [ ]:
# Create custom column "creator_linkedin_linked" and "creator_linkedin_links"
print("Before: {}".format(creator_df.shape))
linkedin_links = creator_external_url_df[creator_external_url_df.href.str.contains("linkedin", case=False)].rename(columns={"href":"creator_linkedin_links"})
linkedin_links = linkedin_links.groupby('creator_id').agg(lambda x: list(x))
creator_df = creator_df.join(other=linkedin_links, how="left")
creator_df["creator_linkedin_linked"] = creator_df.creator_linkedin_links.apply(lambda x: True if isinstance(x, list) else False)
print("After: {}".format(creator_df.shape))
creator_df.head()

Before: (334841, 11)
After: (334841, 13)


,creator_name,creator_url,number_of_creator_links,creator_facebook_links,creator_facebook_linked,creator_twitter_links,creator_twitter_linked,creator_instagram_links,creator_instagram_linked,creator_youtube_links,creator_youtube_linked,creator_linkedin_links,creator_linkedin_linked
id,,,,,,,,,,,,,
3,Charles Adler,https://www.kickstarter.com/profile/cadler,4,NaN,False,[http://twitter.com/cadler],True,NaN,False,NaN,False,NaN,False
4,Perry Chen,https://www.kickstarter.com/profile/perry,1,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False
11,Earl Scioneaux,https://www.kickstarter.com/profile/Samplefreq,8,[http://www.facebook.com/maddwikkid],True,[https://twitter.com/#!/mwikkid],True,NaN,False,NaN,False,NaN,False
12806,Timothy Plain,https://www.kickstarter.com/profile/12806,3,[http://www.facebook.com/spindryproductions],True,NaN,False,NaN,False,NaN,False,[http://www.linkedin.com/pub/timothy-plain/7/9ab/497],True
13061,Sam G. Hershman,https://www.kickstarter.com/profile/13061,0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False


In [ ]:
creator_df.creator_linkedin_linked.value_counts()

False    327966
True       6875
Name: creator_linkedin_linked, dtype: int64

#### Preliminary Data Cleaning - Table: Creator Bio

In [ ]:
creator_bio_df = pd.read_sql_query("SELECT * FROM creator_bio", conn)
creator_bio_df.set_index("creator_id", inplace=True)
print(creator_bio_df.shape)
creator_bio_df.head()

(315396, 2)


,location_name,location_id
creator_id,,
13061,"Middletown, CT",2450408.0
19289,"Post Falls, ID",2475813.0
26057,"Atlanta, GA",2357024.0
34301,"Valdosta, GA",2510659.0
38996,"Brian Head, UT",2368680.0


In [ ]:
creator_bio_df.location_name.isna().sum()

11715

In [ ]:
creator_bio_df.location_id.isna().sum()

17744

In [ ]:
creator_bio_df[creator_bio_df.location_id.isna()==True].head()

,location_name,location_id
creator_id,,
48480,None,NaN
143529,None,NaN
196446,"Concord, VA",NaN
383340,None,NaN
479324,"Balwyn North, AU",NaN


Result: 
- may be useful to get the location of the creator
- for now get location_name, location_state, location_country (however, probably only country will be used for final dataset)

In [ ]:
location_df = pd.read_sql_query("SELECT * FROM location", conn)
location_df.set_index("id", inplace=True)
print(location_df.shape)
location_df.head()

(23252, 12)


,displayable_name,type,name,state,short_name,is_root,country,slug,urls_api_nearby_projects,urls_web_discover,urls_web_location,localized_name
id,,,,,,,,,,,,
33,"Aurora, Canada",Town,Aurora,ON,"Aurora, Canada",False,CA,aurora-york-on,https://api.kickstarter.com/v1/discover?signature=1489096865.728a8b1a03ba39aba5aa937255b561b2580e9a77&woe_id=33,https://www.kickstarter.com/discover/places/aurora-york-on,https://www.kickstarter.com/locations/aurora-york-on,None
70,"Battleford, Canada",Town,Battleford,SK,"Battleford, Canada",False,CA,battleford-ca,https://api.kickstarter.com/v1/discover?signature=1488929708.620b6c650558c60ed950bb82e68920b38ffd623d&woe_id=70,https://www.kickstarter.com/discover/places/battleford-ca,https://www.kickstarter.com/locations/battleford-ca,None
78,"Beaconsfield, Canada",Suburb,Beaconsfield,QC,"Beaconsfield, Canada",False,CA,beaconsfield-montreal-ca,https://api.kickstarter.com/v1/discover?signature=1489068138.d8f6fcc652209d25f4c2b7250786b4ad4a003d5a&woe_id=78,https://www.kickstarter.com/discover/places/beaconsfield-montreal-ca,https://www.kickstarter.com/locations/beaconsfield-montreal-ca,None
79,"Beamsville, Canada",Town,Beamsville,ON,"Beamsville, Canada",False,CA,beamsville-ca,https://api.kickstarter.com/v1/discover?signature=1489274111.50598ccc006947de3a24a16e7bea53ddee9ef388&woe_id=79,https://www.kickstarter.com/discover/places/beamsville-ca,https://www.kickstarter.com/locations/beamsville-ca,None
90,"Beaverton, Canada",Town,Beaverton,ON,"Beaverton, Canada",False,CA,beaverton-on-ca,https://api.kickstarter.com/v1/discover?signature=1489010614.157442227244d283ba8c493c290fc2ab11ed5237&woe_id=90,https://www.kickstarter.com/discover/places/beaverton-on-ca,https://www.kickstarter.com/locations/beaverton-on-ca,None


In [ ]:
creator_bio_df = creator_bio_df.join(other=location_df, how="left", on="location_id")
creator_bio_df = creator_bio_df[["location_name", "state", "country"]]
creator_bio_df.rename(columns={"location_name" : "creator_location_name", "state" : "creator_state", "country" : "creator_country"}, inplace=True)
creator_bio_df.head()

,creator_location_name,creator_state,creator_country
creator_id,,,
13061,"Middletown, CT",CT,US
19289,"Post Falls, ID",ID,US
26057,"Atlanta, GA",GA,US
34301,"Valdosta, GA",GA,US
38996,"Brian Head, UT",UT,US


In [ ]:
creator_bio_df.isna().sum()

creator_location_name    11715
creator_state            17789
creator_country          17744
dtype: int64

In [ ]:
creator_bio_df[creator_bio_df.creator_location_name.isna()==True].head()

,creator_location_name,creator_state,creator_country
creator_id,,,
48480,None,NaN,NaN
143529,None,NaN,NaN
383340,None,NaN,NaN
517150,None,NaN,NaN
582581,None,NaN,NaN


In [ ]:
# Fill in the value "No Location" where a location is not specified
print("Before: {}".format(creator_bio_df.creator_location_name.isna().sum()))
print("Before: {}".format(creator_bio_df.creator_state.isna().sum()))
print("Before: {}".format(creator_bio_df.creator_country.isna().sum()))
no_location_index = creator_bio_df[creator_bio_df.creator_location_name.isna()==True].index
creator_bio_df.loc[no_location_index, "creator_location_name"] = "No Location"
creator_bio_df.loc[no_location_index, "creator_state"] = "No Location"
creator_bio_df.loc[no_location_index, "creator_country"] = "No Location"
print("After: {}".format(creator_bio_df.creator_location_name.isna().sum()))
print("After: {}".format(creator_bio_df.creator_state.isna().sum()))
print("After: {}".format(creator_bio_df.creator_country.isna().sum()))

Before: 11715
Before: 17789
Before: 17744
After: 0
After: 6074
After: 6029


Take care of the missing values where a location is actually specified:

In [ ]:
import googlemaps
def convert_missing_location(x):
    gmaps = googlemaps.Client(key='fill in here the Google API key')
    geocode_result = gmaps.geocode(x)
    
    country = None
    state = None
    
    if len(geocode_result)>0:
        for results in geocode_result[0]["address_components"]:
            if "country" in results["types"]:
                country = results["long_name"]

            if "administrative_area_level_1" in results["types"]:
                state = results["long_name"]

    return country, state

In [ ]:
missing_loc_mask = (creator_bio_df.creator_state.isna()==True) | (creator_bio_df.creator_country.isna()==True)

In [ ]:
for row in creator_bio_df[missing_loc_mask].itertuples():
    country, state = convert_missing_location(row.creator_location_name)
    creator_bio_df.loc[row.Index, "creator_country"] = country
    creator_bio_df.loc[row.Index, "creator_state"] = state

In [ ]:
# Fill in missing states for remaining cities
creator_bio_df.loc[creator_bio_df.creator_location_name=="Antarctica", "creator_state"] = "Antarctica"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Gibraltar, Gibraltar", "creator_state"] = "Gibraltar"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Jhongjheng District, Taiwan", "creator_state"] = "Keelung"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Nyíregyháza, Hungary", "creator_state"] = "Szabolcs-Szatmár-Bereg"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Macau, Macao", "creator_state"] = "Macau"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Hualien City, Taiwan", "creator_state"] = "Hualien"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Kos, Greece", "creator_state"] = "Kos"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Sodankylä, Finland", "creator_state"] = "Lapland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Palaion Faliron, Greece", "creator_state"] = "Attica"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Kuusisto, Finland", "creator_state"] = "Kaarina"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Dunaharaszti, Hungary", "creator_state"] = "Pest"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Hamrun, Malta", "creator_state"] = "Malta Xlokk"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Siberia, AU", "creator_state"] = "Siberia"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Agana Heights, Guam", "creator_state"] = "Agana"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Oranjestad, Aruba", "creator_state"] = "Oranjestad"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Petroupolis, Greece", "creator_state"] = "Attica"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Poland, IN", "creator_state"] = "Indiana"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Poland, IN", "creator_country"] = "US"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Lovisa, Finland", "creator_state"] = "Uusimaa"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Singapore, Singapore", "creator_state"] = "Central Singapore"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Elizabeth, AU", "creator_state"] = "South Australia"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Elizabeth, AU", "creator_country"] = "Australia"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Khios, Greece", "creator_state"] = "Khios"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Hong Kong, Hong Kong", "creator_state"] = "Hong Kong Island"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Nish, Serbia", "creator_state"] = "Niasava"
creator_bio_df.loc[creator_bio_df.creator_location_name=="La Vendée, France", "creator_state"] = "Pays de la Loire"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Takapau, NZ", "creator_state"] = "Hawke's Bay"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Agios Nikolaos, Greece", "creator_state"] = "Crete"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Józsa, Hungary", "creator_state"] = "Debrecen"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Bujebba, Malta", "creator_state"] = "Malta Majjistral"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Hameenlinna, Finland", "creator_state"] = "Tavastia Proper"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Kópavogur, Iceland", "creator_state"] = "Reykjavik"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Biatorbágy, Hungary", "creator_state"] = "Pest"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Taipei City, Taiwan", "creator_state"] = "Taipei City"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Fgura, Malta", "creator_state"] = "Malta Xlokk"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Jászberény, Hungary", "creator_state"] = "Jász-Nagykun-Szolnok"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Larissa, Greece", "creator_state"] = "Thessaly and Continental Greece"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Miskolcz, Hungary", "creator_state"] = "Borsod-Abaúj-Zemplén"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Saint-Martin, France", "creator_state"] = "Saint-Martin"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Boon Lay, Singapore", "creator_state"] = "West Singapore"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Haukipudas, Finland", "creator_state"] = "Ostrobothnia"

In [ ]:
# Fill in missing states for Danish cities
creator_bio_df.loc[creator_bio_df.creator_location_name=="Greve, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Valby, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Søborg, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Taastrup, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Vanløse, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Vallekilde, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Fredensborg, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Horsholm, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Haderslev, Denmark", "creator_state"] = "Syddanmark"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Ringsted, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Lille Skensved, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Ydre Osterbro, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Hadsund, Denmark", "creator_state"] = "Nordjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Auning, Denmark", "creator_state"] = "Midtjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Aalborg East, Denmark", "creator_state"] = "Nordjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Bredsten, Denmark", "creator_state"] = "Syddanmark"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Løgten, Denmark", "creator_state"] = "Midtjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Kårup, Denmark", "creator_state"] = "Midtjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Albertslund, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Glud, Denmark", "creator_state"] = "Midtjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Snekkersten, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Kibæk, Denmark", "creator_state"] = "Midtjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Nibe, Denmark", "creator_state"] = "Nordjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Bagsværd, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Ronne, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Borup, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Klippinge, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Fodby, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Tune, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Trige, Denmark", "creator_state"] = "Midtjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Pandrup, Denmark", "creator_state"] = "Nordjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Brande, Denmark", "creator_state"] = "Midtjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Græsted, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Rodvig Stevns, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Olstykke, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Hørning, Denmark", "creator_state"] = "Midtjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Vesterbro, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Vallensbaek, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Vordingborg, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Praesto, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Havdrup, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Nyborg, Denmark", "creator_state"] = "Syddanmark"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Nivå, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Nykobing Mors, Denmark", "creator_state"] = "Nordjylland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Dalmose, Denmark", "creator_state"] = "Sjaelland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Smørumnedre, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Charlottenlund, Denmark", "creator_state"] = "Hovedstaden"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Bramming, Denmark", "creator_state"] = "Syddanmark"

In [ ]:
# Fill in missing countries
creator_bio_df.loc[creator_bio_df.creator_location_name=="Vauxhall, UK", "creator_state"] = "England"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Vauxhall, UK", "creator_country"] = "GB"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Springfield, AU", "creator_state"] = "Queensland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Springfield, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Reservoir, AU", "creator_state"] = "Victoria"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Reservoir, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Grange, AU", "creator_state"] = "South Australia"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Grange, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Rundu, Namibia", "creator_state"] = "Okavango"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Rundu, Namibia", "creator_country"] = "Namibia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Pristina, Kosovo", "creator_state"] = "Pristina"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Pristina, Kosovo", "creator_country"] = "Kosovo"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Pemulwuy, AU", "creator_state"] = "New South Wales"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Pemulwuy, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Norway, AU", "creator_state"] = "New South Wales"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Norway, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Chifley, AU", "creator_state"] = "New South Wales"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Chifley, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Norrlandet, Sweden", "creator_state"] = "Västervik"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Norrlandet, Sweden", "creator_country"] = "Sweden"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Gordon, AU", "creator_state"] = "New South Wales"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Gordon, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Hallam, AU", "creator_state"] = "Victoria"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Hallam, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Hendra, AU", "creator_state"] = "Queensland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Hendra, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Dromedary, AU", "creator_state"] = "Tasmania"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Dromedary, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Caterham, UK", "creator_state"] = "Surrey"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Caterham, UK", "creator_country"] = "GB"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Balmain, AU", "creator_state"] = "New South Wales"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Balmain, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Lapithos, Cyprus", "creator_state"] = "Girne"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Lapithos, Cyprus", "creator_country"] = "Cyprus"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Busby, AU", "creator_state"] = "New South Wales"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Busby, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Windhoek, Namibia", "creator_state"] = "Khomas"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Windhoek, Namibia", "creator_country"] = "Namibia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Australie, Curacao", "creator_state"] = "No State"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Australie, Curacao", "creator_country"] = "Curacao"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Fitzgibbon, AU", "creator_state"] = "Queensland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Fitzgibbon, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Clyde, AU", "creator_state"] = "Victoria"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Clyde, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Bow, UK", "creator_state"] = "England"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Bow, UK", "creator_country"] = "GB"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Doreen, AU", "creator_state"] = "Victoria"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Doreen, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Westminster, AU", "creator_state"] = "Western Australia"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Westminster, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="The Gap, AU", "creator_state"] = "Queensland"
creator_bio_df.loc[creator_bio_df.creator_location_name=="The Gap, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Carlisle, AU", "creator_state"] = "Western Australia"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Carlisle, AU", "creator_country"] = "Australia"

creator_bio_df.loc[creator_bio_df.creator_location_name=="Fern Tree, AU", "creator_state"] = "Tasmania"
creator_bio_df.loc[creator_bio_df.creator_location_name=="Fern Tree, AU", "creator_country"] = "Australia"

In [ ]:
creator_bio_df.isna().sum()

creator_location_name    0
creator_state            0
creator_country          0
dtype: int64

In [ ]:
creator_bio_df.creator_country.value_counts()

US             226955
GB              25961
CA              12012
No Location     11715
AU               5754
                ...  
MG                  1
Belize              1
Vietnam             1
YE                  1
Georgia             1
Name: creator_country, Length: 267, dtype: int64

In [ ]:
# Replace country codes with full name
import pycountry
def replace_country_code(x):
    country = pycountry.countries.get(alpha_2=x)
    if country is None:
        return x
    else:
        return country.name

creator_bio_df.creator_country = creator_bio_df.creator_country.apply(lambda x: replace_country_code(x))
creator_bio_df.creator_country.value_counts()

United States               229330
United Kingdom               26653
Canada                       12374
No Location                  11715
Australia                     6419
                             ...  
Turks and Caicos Islands         1
Sint Maarten                     1
Djibouti                         1
Montenegro                       1
Aruba                            1
Name: creator_country, Length: 195, dtype: int64

In [ ]:
creator_bio_df[(creator_bio_df.creator_country.str.len()<=2)]

,creator_location_name,creator_state,creator_country
creator_id,,,


In [ ]:
creator_bio_df[(creator_bio_df.creator_state.str.len()<=2) 
               & (
                    (creator_bio_df.creator_country != "United States") &
                    (creator_bio_df.creator_country != "Canada") &
                    (creator_bio_df.creator_country != "Australia")
                 )]

,creator_location_name,creator_state,creator_country
creator_id,,,


In [ ]:
# Replace US State Codes with Full Names
import us
def replace_us_state_code(x):
    us_state = us.states.lookup(x)
    if us_state is None:
        return x
    else:
        return us_state.name


creator_bio_df.creator_state = creator_bio_df.creator_state.apply(lambda x: replace_us_state_code(x))

In [ ]:
# Replace state codes for Canada and Australia
creator_bio_df.loc[creator_bio_df.creator_state=="ON", "creator_state"] = "Ontario"
creator_bio_df.loc[creator_bio_df.creator_state=="QC", "creator_state"] = "Quebec"
creator_bio_df.loc[creator_bio_df.creator_state=="BC", "creator_state"] = "British Columbia"
creator_bio_df.loc[creator_bio_df.creator_state=="AB", "creator_state"] = "Alberta"
creator_bio_df.loc[creator_bio_df.creator_state=="NS", "creator_state"] = "Nova Scotia"
creator_bio_df.loc[creator_bio_df.creator_state=="MB", "creator_state"] = "Manitoba"
creator_bio_df.loc[creator_bio_df.creator_state=="SA", "creator_state"] = "South Australia"
creator_bio_df.loc[creator_bio_df.creator_state=="NSW", "creator_state"] = "New South Wales"
creator_bio_df.loc[creator_bio_df.creator_state=="VIC", "creator_state"] = "Victoria"
creator_bio_df.loc[creator_bio_df.creator_state=="QLD", "creator_state"] = "Queensland"
creator_bio_df.loc[creator_bio_df.creator_state=="ACT", "creator_state"] = "Australian Capital Territory"
creator_bio_df.loc[creator_bio_df.creator_state=="SK", "creator_state"] = "Saskatchewan"
creator_bio_df.loc[creator_bio_df.creator_state=="NB", "creator_state"] = "New Brunswick"
creator_bio_df.loc[creator_bio_df.creator_state=="TAS", "creator_state"] = "Tasmania"
creator_bio_df.loc[creator_bio_df.creator_state=="PE", "creator_state"] = "Prince Edward Island"
creator_bio_df.loc[creator_bio_df.creator_state=="NL", "creator_state"] = "Newfoundland and Labrador"
creator_bio_df.loc[creator_bio_df.creator_state=="NT", "creator_state"] = "Northwest Territories"
creator_bio_df.loc[creator_bio_df.creator_state=="NU", "creator_state"] = "Nunavut"

In [ ]:
# Replace country code for Kosovo
creator_bio_df.loc[creator_bio_df.creator_country=="XK", "creator_country"] = "Kosovo"

In [ ]:
creator_bio_df.head()

,creator_location_name,creator_state,creator_country
creator_id,,,
13061,"Middletown, CT",Connecticut,United States
19289,"Post Falls, ID",Idaho,United States
26057,"Atlanta, GA",Georgia,United States
34301,"Valdosta, GA",Georgia,United States
38996,"Brian Head, UT",Utah,United States


In [ ]:
# Merge table creator_df with creator_bio_df
print("Before: {}".format(creator_df.shape))
creator_df = creator_df.join(other=creator_bio_df, how="left")
print("After: {}".format(creator_df.shape))

Before: (334841, 13)
After: (334841, 16)


In [ ]:
creator_df.head()

,creator_name,creator_url,number_of_creator_links,creator_facebook_links,creator_facebook_linked,creator_twitter_links,creator_twitter_linked,creator_instagram_links,creator_instagram_linked,creator_youtube_links,creator_youtube_linked,creator_linkedin_links,creator_linkedin_linked,creator_location_name,creator_state,creator_country
id,,,,,,,,,,,,,,,,
3,Charles Adler,https://www.kickstarter.com/profile/cadler,4,NaN,False,[http://twitter.com/cadler],True,NaN,False,NaN,False,NaN,False,"Chicago, IL",Illinois,United States
4,Perry Chen,https://www.kickstarter.com/profile/perry,1,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,"Brooklyn, NY",New York,United States
11,Earl Scioneaux,https://www.kickstarter.com/profile/Samplefreq,8,[http://www.facebook.com/maddwikkid],True,[https://twitter.com/#!/mwikkid],True,NaN,False,NaN,False,NaN,False,"New Orleans, LA",Louisiana,United States
12806,Timothy Plain,https://www.kickstarter.com/profile/12806,3,[http://www.facebook.com/spindryproductions],True,NaN,False,NaN,False,NaN,False,[http://www.linkedin.com/pub/timothy-plain/7/9ab/497],True,"San Francisco, CA",California,United States
13061,Sam G. Hershman,https://www.kickstarter.com/profile/13061,0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,"Middletown, CT",Connecticut,United States


In [ ]:
creator_df[creator_df.creator_location_name.isna()==True].shape

(72697, 16)

In [ ]:
# Merge table creator_df with kickstarter_berkeley_df
print("Before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df = kickstarter_berkeley_df.join(other=creator_df, how="left", on="creator_id")
print("After: {}".format(kickstarter_berkeley_df.shape))

Before: (366907, 27)
After: (366907, 43)


In [ ]:
kickstarter_berkeley_df.head()

,url_project,name,blurb,state,goal,pledged,backers_count,comments_count,updates_count,staff_pick,campaign_has_video,kickstarter_country,created_at,launched_at,state_changed_at,successful_at,deadline,updated_at,last_modification,profile_changed_after_success,creator_id,profile_id,subcategory,category,location_name,location_state,location_country,creator_name,creator_url,number_of_creator_links,creator_facebook_links,creator_facebook_linked,creator_twitter_links,creator_twitter_linked,creator_instagram_links,creator_instagram_linked,creator_youtube_links,creator_youtube_linked,creator_linkedin_links,creator_linkedin_linked,creator_location_name,creator_state,creator_country
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22810934,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,"Lights, Coma, Action!","A book to tell the story of who I was (LIGHTS), how I sustained a serious brain injury (COMA), and my strong recovery (ACTION!).",successful,15000.0,18167.88,284,4,40,False,True,United States,2013-10-01,2013-10-03,2013-11-02,2013-11-02,2013-11-02,2017-03-01,2018-05-24,True,1157451606,716409,Nonfiction,Publishing,"Austin, TX",Texas,United States,Cavin Balaster,https://www.kickstarter.com/profile/cavinbounce,2,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,"Austin, TX",Texas,United States
22821161,https://www.kickstarter.com/projects/684178251/sentio-golf-putters-feel-is-the-difference,Sentio Golf Putters: Feel IS the difference,"Choose the feel YOU want with our patented floating-face technology. Modern design, proven performance, and a splash of color.",failed,50000.0,9707.00,41,0,4,False,True,United States,2015-06-08,2015-09-18,2015-11-02,NaN,2015-11-02,2016-05-20,2017-03-11,False,684178251,1945377,Product Design,Design,"Acton, MA",Massachusetts,United States,Sentio Golf,https://www.kickstarter.com/profile/684178251,2,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,"Acton, MA",Massachusetts,United States
22823613,https://www.kickstarter.com/projects/1237543205/brainade,Brainade,We intend to create an online platform for individuals to teach or learn a skill via online classrooms and forums.,failed,750.0,1.00,1,0,0,False,False,United States,2017-02-02,2017-02-15,2017-03-17,NaN,2017-03-17,2017-03-17,2017-04-12,False,1237543205,2862898,Web,Technology,"Spearfish, SD",South Dakota,United States,Aric Buhler,https://www.kickstarter.com/profile/1237543205,0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,"Spearfish, SD",South Dakota,United States
22826244,https://www.kickstarter.com/projects/impkingcomics/horizon-anthology,Horizon Anthology,HORIZON is a 144-page collection of fantasy comics by fifteen artists exploring what happens after the hero has won...,successful,7000.0,8955.41,286,20,16,True,True,United States,2013-09-13,2014-05-05,2014-06-04,2014-06-04,2014-06-04,2016-05-19,2018-05-24,False,2122725209,696107,Anthologies,Comics,"Savannah, GA",Georgia,United States,Jeremy Lawson,https://www.kickstarter.com/profile/impkingcomics,1,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,NaN,NaN
22835897,https://www.kickstarter.com/projects/alphabetstory/the-alphabet-story-childrens-book,The Alphabet Story Children's Book,A richly illustrated children's book following a story about a boy lost in a magical dream land who meets and learns the ABCs.,successful,6000.0,6643.11,66,0,8,False,True,United States,2016-08-21,2016-08-31,2016-09-28,2016-09-28,2016-09-28,2017-02-28,2017-03-11,False,1566612915,2647051,Children's Books,Publishing,"Portland, OR",Oregon,United States,Nate Jensen - Artist,https://www.kickstarter.com/profile/alphabetstory,3,[https://www.facebook.com/NateJensenArtist/],True,NaN,False,NaN,False,NaN,False,NaN,False,"Portland, OR",Oregon,United States


In [ ]:
print(kickstarter_berkeley_df.creator_location_name.isna().sum())
print(kickstarter_berkeley_df.creator_state.isna().sum())
print(kickstarter_berkeley_df.creator_country.isna().sum())

78241
78241
78241


In [ ]:
print(kickstarter_berkeley_df.creator_facebook_linked.isna().sum())
print(kickstarter_berkeley_df.creator_twitter_linked.isna().sum())
print(kickstarter_berkeley_df.creator_instagram_linked.isna().sum())
print(kickstarter_berkeley_df.creator_youtube_linked.isna().sum())
print(kickstarter_berkeley_df.creator_linkedin_linked.isna().sum())

0
0
0
0
0


Result: 
- around 78,000 creator locations were not available in creator_bio_df
- however, they are actually available in the profiles of the creators
- maybe I need to scrape the remaining 78,000 creator profiles for the location

#### Preliminary Data Cleaning - Table: Reward

In [ ]:
reward_df = pd.read_sql_query("SELECT * FROM reward", conn)
reward_df.set_index("id", inplace=True)
print(reward_df.shape)
reward_df.head()

(3220544, 17)


,project_id,title,reward,title_for_backing_tier,shipping_preference,description,shipping_summary,limit,estimated_delivery_on,ends_at,starts_at,updated_at,shipping_enabled,backers_count,remaining,minimum,urls_api_reward
id,,,,,,,,,,,,,,,,,
3,1860890148,None,Grace Jones T Shirt - Limited Edition. (Delivery Included),$20 reward,None,Grace Jones T Shirt - Limited Edition. (Delivery Included),None,100.0,NaN,0.0,0.0,1488230489,False,30,70.0,20,https://api.kickstarter.com/v1/projects/1860890148/rewards/3?signature=1489132787.eb2e7d53b4909c670101ef665cc2d47a675fa4c4
8,405307432,None,"Get the behind-the-scenes updates AND an exclusive early download of the record as soon as it's done (~1 month before it's out on CDs, iTunes, etc.)",$15 reward,None,"Get the behind-the-scenes updates AND an exclusive early download of the record as soon as it's done (~1 month before it's out on CDs, iTunes, etc.)",None,NaN,NaN,0.0,0.0,1485279122,False,53,NaN,15,https://api.kickstarter.com/v1/projects/405307432/rewards/8?signature=1489083114.4276618852e8c63661a6832bb7fb48a03494eb1f
9,405307432,None,"Get access to the behind-the-scenes video & audio updates that will document the creation of this album. This will include video clips of recording sessions, audio clips of tunes as they progress, and more!",$1 reward,None,"Get access to the behind-the-scenes video & audio updates that will document the creation of this album. This will include video clips of recording sessions, audio clips of tunes as they progress, and more!",None,NaN,NaN,0.0,0.0,1485274207,False,12,NaN,1,https://api.kickstarter.com/v1/projects/405307432/rewards/9?signature=1489083114.708a63ca2f4e4704195102f1a5fdbd66d23f3522
11,1672335313,None,Free LP,$15 reward,None,Free LP,None,NaN,NaN,0.0,0.0,1485283398,False,25,NaN,15,https://api.kickstarter.com/v1/projects/1672335313/rewards/11?signature=1488944834.074f9618767863a2699f8c74bc5ad6f94cfe3fdf
12,1672335313,None,Free LP and tshirt,$40 reward,None,Free LP and tshirt,None,NaN,NaN,0.0,0.0,1485283409,False,16,NaN,40,https://api.kickstarter.com/v1/projects/1672335313/rewards/12?signature=1488944834.9e346577dd5bdf229f2309a80bc0848e70d3a5a1


##### Drop all instances which are not available in the Berkeley dataset:

In [ ]:
print("Before: {}".format(reward_df.shape))
reward_df = reward_df[reward_df.project_id.isin(kickstarter_berkeley_df.index)] 
print("After: {}".format(reward_df.shape))

Before: (3220544, 17)
After: (2914494, 17)


##### title:

In [ ]:
print(reward_df.title.isna().sum())

2083310


In [ ]:
reward_df[reward_df.title.isna()==True].head()

,project_id,title,reward,title_for_backing_tier,shipping_preference,description,shipping_summary,limit,estimated_delivery_on,ends_at,starts_at,updated_at,shipping_enabled,backers_count,remaining,minimum,urls_api_reward
id,,,,,,,,,,,,,,,,,
3,1860890148,None,Grace Jones T Shirt - Limited Edition. (Delivery Included),$20 reward,None,Grace Jones T Shirt - Limited Edition. (Delivery Included),None,100.0,NaN,0.0,0.0,1488230489,False,30,70.0,20,https://api.kickstarter.com/v1/projects/1860890148/rewards/3?signature=1489132787.eb2e7d53b4909c670101ef665cc2d47a675fa4c4
8,405307432,None,"Get the behind-the-scenes updates AND an exclusive early download of the record as soon as it's done (~1 month before it's out on CDs, iTunes, etc.)",$15 reward,None,"Get the behind-the-scenes updates AND an exclusive early download of the record as soon as it's done (~1 month before it's out on CDs, iTunes, etc.)",None,NaN,NaN,0.0,0.0,1485279122,False,53,NaN,15,https://api.kickstarter.com/v1/projects/405307432/rewards/8?signature=1489083114.4276618852e8c63661a6832bb7fb48a03494eb1f
9,405307432,None,"Get access to the behind-the-scenes video & audio updates that will document the creation of this album. This will include video clips of recording sessions, audio clips of tunes as they progress, and more!",$1 reward,None,"Get access to the behind-the-scenes video & audio updates that will document the creation of this album. This will include video clips of recording sessions, audio clips of tunes as they progress, and more!",None,NaN,NaN,0.0,0.0,1485274207,False,12,NaN,1,https://api.kickstarter.com/v1/projects/405307432/rewards/9?signature=1489083114.708a63ca2f4e4704195102f1a5fdbd66d23f3522
11,1672335313,None,Free LP,$15 reward,None,Free LP,None,NaN,NaN,0.0,0.0,1485283398,False,25,NaN,15,https://api.kickstarter.com/v1/projects/1672335313/rewards/11?signature=1488944834.074f9618767863a2699f8c74bc5ad6f94cfe3fdf
12,1672335313,None,Free LP and tshirt,$40 reward,None,Free LP and tshirt,None,NaN,NaN,0.0,0.0,1485283409,False,16,NaN,40,https://api.kickstarter.com/v1/projects/1672335313/rewards/12?signature=1488944834.9e346577dd5bdf229f2309a80bc0848e70d3a5a1


In [ ]:
# Replace NaN values through empty string, since I may merge all string attribute together later
reward_df["title"] = reward_df.title.apply(lambda x: "" if pd.isnull(x) else x)
print(reward_df.title.isna().sum())

0


Result: 
- keep attribute for now
- the instances with the NaN value just don't have a title
- but, most rewards at least have a description

##### reward and description:

In [ ]:
print(reward_df.reward.isna().sum())
print(reward_df.description.isna().sum())

1997
1997


In [ ]:
len(reward_df[reward_df.reward!=reward_df.description])

1997

In [ ]:
reward_df.reward.compare(reward_df.description)

,self,other
id,,


In [ ]:
reward_df[reward_df.reward.isna()!=reward_df.description.isna()].head()

,project_id,title,reward,title_for_backing_tier,shipping_preference,description,shipping_summary,limit,estimated_delivery_on,ends_at,starts_at,updated_at,shipping_enabled,backers_count,remaining,minimum,urls_api_reward
id,,,,,,,,,,,,,,,,,


In [ ]:
# Drop attribute "reward", since it is redundant to "description"
print("Before: {}".format(reward_df.shape))
reward_df.drop(columns=["reward"], inplace=True)
print("After: {}".format(reward_df.shape))

Before: (2914494, 17)
After: (2914494, 16)


##### title_for_backing_tier:

In [ ]:
reward_df.title_for_backing_tier.isna().sum()

0

In [ ]:
reward_df.title_for_backing_tier.value_counts()

$100 reward                                         141503
$50 reward                                          138516
$25 reward                                          135073
$10 reward                                          117314
$500 reward                                          99372
                                                     ...  
The Deluxe Edition (£57)                                 1
Original Commission Collection ($1,000)                  1
Print Book + Essential Styling Package (AU$ 650)         1
Just the New Stuff- Digital ($10)                        1
Nope Webcam Cover Mega 12 Pack ($60)                     1
Name: title_for_backing_tier, Length: 771957, dtype: int64

In [ ]:
reward_df.title.compare(reward_df.title_for_backing_tier).tail(10)

,self,other
id,,
7014636,The Map Itself,The Map Itself (AU$ 500)
7037437,The Reskin Map,The Reskin Map (AU$ 500)
7045719,The Sepia Toned Map,The Sepia Toned Map (AU$ 500)
7047558,The Colour Map!,The Colour Map! (AU$ 500)
7047559,The Kingdom Death Monster Map,The Kingdom Death Monster Map (AU$ 500)
6996217,Digital Copy,Digital Copy ($5)
6996219,Physical Copy,Physical Copy ($10)
6996221,Signed Copy and Sketch,Signed Copy and Sketch ($20)
7009772,Retailer Reward,Retailer Reward ($30)


Result:
- "title_for_backing_tier" is basically the title + the pledge amount
- since I will keep the title and I have the pledge amount in another attribute, this attribute can be dropped

In [ ]:
# Drop attribute "title_for_backing_tier"
print("Before: {}".format(reward_df.shape))
reward_df.drop(columns=["title_for_backing_tier"], inplace=True)
print("After: {}".format(reward_df.shape))

Before: (2914494, 16)
After: (2914494, 15)


##### shipping_enabled, shipping_summary and shipping_preference:

In [ ]:
print(reward_df.shipping_enabled.isna().sum())
print(reward_df.shipping_preference.isna().sum())
print(reward_df.shipping_summary.isna().sum())

0
1225755
1225755


In [ ]:
reward_df.shipping_enabled.value_counts()

True     1099866
False    1003414
1         588873
0         222341
Name: shipping_enabled, dtype: int64

In [ ]:
# Normalize shipping_enabled attribute
reward_df["shipping_enabled"] = reward_df.shipping_enabled.apply(lambda x: True if x=="True" else
                                                                        (True if x=="1" else
                                                                        (False if x=="False" else
                                                                        (False if x=="0" else x))))
reward_df.shipping_enabled.value_counts()

True     1688739
False    1225755
Name: shipping_enabled, dtype: int64

In [ ]:
reward_df.shipping_preference.value_counts()

unrestricted    1207010
restricted       481729
Name: shipping_preference, dtype: int64

In [ ]:
reward_df.shipping_summary.value_counts()

Ships anywhere in the world                                                               751873
Anywhere in the world                                                                     455137
Only Ships to United States                                                               291354
Only United States                                                                         82791
Only certain countries                                                                     37065
                                                                                           ...  
Only Ships to France, Italy                                                                    1
Only Ships to Brazil, El Salvador, Mexico, Puerto Rico, United States                          1
Only Ships to Brazil, United Kingdom                                                           1
Only Ships to Taiwan                                                                           1
Only Ships to United Arab Emir

Result:
- shipping_enabled is unnecessary, since it can also be determined by shipping_preference.isna()==True -> drop
- shipping_summary has too many different values, so that it can be useful -> drop
- i.e. only keep shipping_preference and add another value "No shipping"

In [ ]:
# Drop attributes "shipping_enabled" and "shipping_summary"
print("Before: {}".format(reward_df.shape))
reward_df.drop(columns=["shipping_enabled", "shipping_summary"], inplace=True)
print("After: {}".format(reward_df.shape))

Before: (2914494, 15)
After: (2914494, 13)


In [ ]:
# Add another value "no shipping" to "shipping_preference"
reward_df["shipping_preference"] = reward_df.shipping_preference.apply(lambda x: "no shipping specified" if pd.isnull(x) else x)
reward_df.shipping_preference.value_counts()

no shipping specified    1225755
unrestricted             1207010
restricted                481729
Name: shipping_preference, dtype: int64

In [ ]:
# Rename attribute to "shipping_policy"
reward_df.rename(columns={"shipping_preference" : "shipping_policy"}, inplace=True)

In [ ]:
reward_df.head()

,project_id,title,shipping_policy,description,limit,estimated_delivery_on,ends_at,starts_at,updated_at,backers_count,remaining,minimum,urls_api_reward
id,,,,,,,,,,,,,
3,1860890148,,no shipping specified,Grace Jones T Shirt - Limited Edition. (Delivery Included),100.0,NaN,0.0,0.0,1488230489,30,70.0,20,https://api.kickstarter.com/v1/projects/1860890148/rewards/3?signature=1489132787.eb2e7d53b4909c670101ef665cc2d47a675fa4c4
8,405307432,,no shipping specified,"Get the behind-the-scenes updates AND an exclusive early download of the record as soon as it's done (~1 month before it's out on CDs, iTunes, etc.)",NaN,NaN,0.0,0.0,1485279122,53,NaN,15,https://api.kickstarter.com/v1/projects/405307432/rewards/8?signature=1489083114.4276618852e8c63661a6832bb7fb48a03494eb1f
9,405307432,,no shipping specified,"Get access to the behind-the-scenes video & audio updates that will document the creation of this album. This will include video clips of recording sessions, audio clips of tunes as they progress, and more!",NaN,NaN,0.0,0.0,1485274207,12,NaN,1,https://api.kickstarter.com/v1/projects/405307432/rewards/9?signature=1489083114.708a63ca2f4e4704195102f1a5fdbd66d23f3522
11,1672335313,,no shipping specified,Free LP,NaN,NaN,0.0,0.0,1485283398,25,NaN,15,https://api.kickstarter.com/v1/projects/1672335313/rewards/11?signature=1488944834.074f9618767863a2699f8c74bc5ad6f94cfe3fdf
12,1672335313,,no shipping specified,Free LP and tshirt,NaN,NaN,0.0,0.0,1485283409,16,NaN,40,https://api.kickstarter.com/v1/projects/1672335313/rewards/12?signature=1488944834.9e346577dd5bdf229f2309a80bc0848e70d3a5a1


##### limit, backers_count, and remaining:

In [ ]:
print(reward_df.limit.isna().sum())
print(reward_df.backers_count.isna().sum())
print(reward_df.remaining.isna().sum())

1972544
0
1972544


In [ ]:
reward_df.limit.value_counts()

1.0       114691
10.0      114086
5.0        97747
50.0       70481
100.0      64820
           ...  
2540.0         1
2499.0         1
2498.0         1
2490.0         1
3998.0         1
Name: limit, Length: 1476, dtype: int64

Result:
- "backers_count" and "remaining" won't be anyway available when the campaign starts -> drop
- keep "limit" for now

In [ ]:
# Drop attributes "backers_count" and "remaining"
print("Before: {}".format(reward_df.shape))
reward_df.drop(columns=["backers_count", "remaining"], inplace=True)
print("After: {}".format(reward_df.shape))

Before: (2914494, 13)
After: (2914494, 11)


##### urls_api_reward:

In [ ]:
# Drop attribute, since not useful for my purpose
print("Before: {}".format(reward_df.shape))
reward_df.drop(columns=["urls_api_reward"], inplace=True)
print("After: {}".format(reward_df.shape))

Before: (2914494, 11)
After: (2914494, 10)


##### minimum:

In [ ]:
reward_df.minimum.isna().sum()

0

In [ ]:
reward_df.minimum.value_counts()

100      209778
50       208550
25       196832
10       186232
500      140800
          ...  
3625          1
15050         1
4814          1
1577          1
2047          1
Name: minimum, Length: 4777, dtype: int64

In [ ]:
# Rename attribute to "pledge_amount"
reward_df.rename(columns={"minimum" : "pledge_amount"}, inplace=True)
reward_df.head(1)

,project_id,title,shipping_policy,description,limit,estimated_delivery_on,ends_at,starts_at,updated_at,pledge_amount
id,,,,,,,,,,
3,1860890148,,no shipping specified,Grace Jones T Shirt - Limited Edition. (Delivery Included),100.0,NaN,0.0,0.0,1488230489,20


In [ ]:
# Get the currency and conversion rate to USD per project
currency_df = pd.read_csv("02_Data Collection/Kickstarter_Berkeley_Cleaned.csv", index_col=0)
currency_df = currency_df[["currency", "static_usd_rate"]]
print(currency_df.shape)
print(len(currency_df.index.unique()))
currency_df.head(1)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (24,25,27,28,40,41,50,54) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(408637, 2)
408637


,currency,static_usd_rate
id,,
22807353,USD,1.0


In [ ]:
# Add the currency-related attributes to reward_df
print("Before: {}".format(reward_df.shape))
reward_df = reward_df.join(currency_df, on="project_id", how="left")
print("After: {}".format(reward_df.shape))
reward_df.head()

Before: (2914494, 10)
After: (2914494, 12)


,project_id,title,shipping_policy,description,limit,estimated_delivery_on,ends_at,starts_at,updated_at,pledge_amount,currency,static_usd_rate
id,,,,,,,,,,,,
3,1860890148,,no shipping specified,Grace Jones T Shirt - Limited Edition. (Delivery Included),100.0,NaN,0.0,0.0,1488230489,20,USD,1.0
8,405307432,,no shipping specified,"Get the behind-the-scenes updates AND an exclusive early download of the record as soon as it's done (~1 month before it's out on CDs, iTunes, etc.)",NaN,NaN,0.0,0.0,1485279122,15,USD,1.0
9,405307432,,no shipping specified,"Get access to the behind-the-scenes video & audio updates that will document the creation of this album. This will include video clips of recording sessions, audio clips of tunes as they progress, and more!",NaN,NaN,0.0,0.0,1485274207,1,USD,1.0
11,1672335313,,no shipping specified,Free LP,NaN,NaN,0.0,0.0,1485283398,15,USD,1.0
12,1672335313,,no shipping specified,Free LP and tshirt,NaN,NaN,0.0,0.0,1485283409,40,USD,1.0


In [ ]:
reward_df.currency.value_counts()

USD    2290814
GBP     258475
EUR     137391
CAD     105287
AUD      51604
MXN      15395
SEK      11769
NZD      10222
HKD       7967
DKK       7211
CHF       6428
SGD       5652
NOK       3925
JPY       2354
Name: currency, dtype: int64

In [ ]:
reward_df[reward_df.currency!="USD"].sample()

,project_id,title,shipping_policy,description,limit,estimated_delivery_on,ends_at,starts_at,updated_at,pledge_amount,currency,static_usd_rate
id,,,,,,,,,,,,
5217119,1911138055,Personal add-on,no shipping specified,With this you can add the add-on for your own account.,1000.0,1.472688e+09,0.0,0.0,1487483812,2,EUR,1.135694


In [ ]:
# Convert foreign currency into USD
reward_df["pledge_amount_converted"] = reward_df.apply(lambda x: round(x["pledge_amount"]*x["static_usd_rate"]) if x["currency"]!="USD" else x["pledge_amount"], axis=1)
reward_df[reward_df.currency!="USD"].sample()

,project_id,title,shipping_policy,description,limit,estimated_delivery_on,ends_at,starts_at,updated_at,pledge_amount,currency,static_usd_rate,pledge_amount_converted
id,,,,,,,,,,,,,
6824208,694577442,1 WATCH - THE ULTRAMATIC,unrestricted,You save CHF 510 off retail (CHF 1299)\r\n(You will choose your watch style later)\r\n+ we'll add 1 luxury watch box (CHF 49 value) \r\n+ we'll add 1 extra calf leather band with Deployment Buckle\r\n\r\nFree shipping worldwide,NaN,1.543622e+09,0.0,0.0,1539961972,789,CHF,1.026978,810


In [ ]:
# Remove currency-related attributes, since not needed anymore
reward_df.drop(columns=["pledge_amount", "currency", "static_usd_rate"], inplace=True)
reward_df.head(1)

,project_id,title,shipping_policy,description,limit,estimated_delivery_on,ends_at,starts_at,updated_at,pledge_amount_converted
id,,,,,,,,,,
3,1860890148,,no shipping specified,Grace Jones T Shirt - Limited Edition. (Delivery Included),100.0,NaN,0.0,0.0,1488230489,20


In [ ]:
# Rename converted pledge amount
reward_df.rename(columns={"pledge_amount_converted":"pledge_amount"}, inplace=True)
reward_df.head(1)

,project_id,title,shipping_policy,description,limit,estimated_delivery_on,ends_at,starts_at,updated_at,pledge_amount
id,,,,,,,,,,
3,1860890148,,no shipping specified,Grace Jones T Shirt - Limited Edition. (Delivery Included),100.0,NaN,0.0,0.0,1488230489,20


In [ ]:
# Restructure the columns
reward_df = reward_df[["project_id", "title", "description", "pledge_amount", "limit", "shipping_policy",
                      "estimated_delivery_on", "starts_at", "ends_at", "updated_at"]]

In [ ]:
reward_df.head()

,project_id,title,description,pledge_amount,limit,shipping_policy,estimated_delivery_on,starts_at,ends_at,updated_at
id,,,,,,,,,,
3,1860890148,,Grace Jones T Shirt - Limited Edition. (Delivery Included),20,100.0,no shipping specified,NaN,0.0,0.0,1488230489
8,405307432,,"Get the behind-the-scenes updates AND an exclusive early download of the record as soon as it's done (~1 month before it's out on CDs, iTunes, etc.)",15,NaN,no shipping specified,NaN,0.0,0.0,1485279122
9,405307432,,"Get access to the behind-the-scenes video & audio updates that will document the creation of this album. This will include video clips of recording sessions, audio clips of tunes as they progress, and more!",1,NaN,no shipping specified,NaN,0.0,0.0,1485274207
11,1672335313,,Free LP,15,NaN,no shipping specified,NaN,0.0,0.0,1485283398
12,1672335313,,Free LP and tshirt,40,NaN,no shipping specified,NaN,0.0,0.0,1485283409


##### datetime-related attributes:

In [ ]:
print(reward_df.estimated_delivery_on.isna().sum())
print(reward_df.starts_at.isna().sum())
print(reward_df.ends_at.isna().sum())
print(reward_df.updated_at.isna().sum())

183991
14114
14114
0


In [ ]:
# Convert attributes from unix timestamp to normal format
from datetime import datetime
reward_df["estimated_delivery_on"] = reward_df.estimated_delivery_on.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d') if pd.isnull(x)==False else None)
reward_df["starts_at"] = reward_df.starts_at.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d') if pd.isnull(x)==False else None)
reward_df["ends_at"] = reward_df.ends_at.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d') if pd.isnull(x)==False else None)
reward_df["updated_at"] = reward_df.updated_at.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d') if pd.isnull(x)==False else None)
reward_df.head()

,project_id,title,description,pledge_amount,limit,shipping_policy,estimated_delivery_on,starts_at,ends_at,updated_at
id,,,,,,,,,,
3,1860890148,,Grace Jones T Shirt - Limited Edition. (Delivery Included),20,100.0,no shipping specified,None,1970-01-01,1970-01-01,2017-02-27
8,405307432,,"Get the behind-the-scenes updates AND an exclusive early download of the record as soon as it's done (~1 month before it's out on CDs, iTunes, etc.)",15,NaN,no shipping specified,None,1970-01-01,1970-01-01,2017-01-24
9,405307432,,"Get access to the behind-the-scenes video & audio updates that will document the creation of this album. This will include video clips of recording sessions, audio clips of tunes as they progress, and more!",1,NaN,no shipping specified,None,1970-01-01,1970-01-01,2017-01-24
11,1672335313,,Free LP,15,NaN,no shipping specified,None,1970-01-01,1970-01-01,2017-01-24
12,1672335313,,Free LP and tshirt,40,NaN,no shipping specified,None,1970-01-01,1970-01-01,2017-01-24


In [ ]:
tmp = reward_df.estimated_delivery_on.apply(lambda x: x[0:4] if x is not None else None)
tmp.value_counts().sort_index(ascending=False)

2027         6
2025         2
2023        85
2022       377
2021       581
2020      3856
2019     90342
2018    310097
2017    354529
2016    381544
2015    464248
2014    446446
2013    366952
2012    285206
2011     26232
Name: estimated_delivery_on, dtype: int64

In [ ]:
tmp = reward_df.starts_at.apply(lambda x: x[0:4] if x is not None else None)
tmp.value_counts().sort_index(ascending=False)

2019         20
2018        801
2017        828
2016         55
1970    2898676
Name: starts_at, dtype: int64

In [ ]:
tmp = reward_df.ends_at.apply(lambda x: x[0:4] if x is not None else None)
tmp.value_counts().sort_index(ascending=False)

2018         85
2017         11
1970    2900284
Name: ends_at, dtype: int64

In [ ]:
# Drop attributes "starts_at" and "ends_at", since only a few instances contain actual values
# Most values are either NaN or "1970" (corresponds to int=0)
print("Before: {}".format(reward_df.shape))
reward_df.drop(columns=["starts_at", "ends_at"], inplace=True)
print("After: {}".format(reward_df.shape))

Before: (2914494, 10)
After: (2914494, 8)


In [ ]:
# Drop attribute "updated_at", since I will also not retrieve this attribute for the Webrobots dataset
# Also not very useful for my purpose
print("Before: {}".format(reward_df.shape))
reward_df.drop(columns=["updated_at"], inplace=True)
print("After: {}".format(reward_df.shape))

Before: (2914494, 8)
After: (2914494, 7)


In [ ]:
reward_df[(reward_df.shipping_policy=="no shipping specified")&(reward_df.estimated_delivery_on.isna()==False)].head()

,project_id,title,description,pledge_amount,limit,shipping_policy,estimated_delivery_on
id,,,,,,,
11364,223042566,,"""Yours Truly"" - The above plus: A handwritten thank-you card from the cast/crew\r\nof the Ramen Rockstars! Inside the card: Collectible Guitar Pick with HJ Undead or Ramen \r\nRockstars Logo!!!",10,NaN,no shipping specified,2012-03-01
11365,223042566,,"""Personal Premiere"" - All of the of above plus: Your own personal preview!! One week before each episode, you'll get a download code via email for viewing the each episode!! Have the satisfaction of ruining the story for your friends who won't see it for another week!",15,NaN,no shipping specified,2012-03-01
11366,223042566,,"""Something to Hold"" - All of the above plus: a home-made (but slick!) printed DVD of the enitre \r\nseason. (After Season 001 is complete), you will no longer need to have an Internet connection to see your favorite rockstars!!",30,NaN,no shipping specified,2012-03-01
11368,223042566,,"""Photo Philanthropist"" - All of the above plus: Production Photos signed by the cast/crew!! Warning: All excess fat and acne will be recklessly “photoshop’d”.",40,NaN,no shipping specified,2012-03-01
11369,223042566,,"""Stick to The Script"" - All of the above plus: A script used in the filming of the episodes, with the cover page covered in thank-you notes from the cast and crew!!!! We cannot however, guarantee the absence of wasabi or coffee stains along with the notes inside.",65,NaN,no shipping specified,2012-03-01


##### Combine text attributes into one single attribute:

In [ ]:
reward_df["reward_description"] = reward_df["title"] + ". " + reward_df["description"]
reward_df.drop(columns=["title", "description"], inplace=True)

In [ ]:
reward_df.rename(columns={"estimated_delivery_on":"estimated_delivery"}, inplace=True)
reward_df = reward_df[["project_id", "reward_description", "pledge_amount", "limit", "shipping_policy", "estimated_delivery"]]
reward_df.head()

,project_id,reward_description,pledge_amount,limit,shipping_policy,estimated_delivery
id,,,,,,
3,1860890148,. Grace Jones T Shirt - Limited Edition. (Delivery Included),20,100.0,no shipping specified,None
8,405307432,". Get the behind-the-scenes updates AND an exclusive early download of the record as soon as it's done (~1 month before it's out on CDs, iTunes, etc.)",15,NaN,no shipping specified,None
9,405307432,". Get access to the behind-the-scenes video & audio updates that will document the creation of this album. This will include video clips of recording sessions, audio clips of tunes as they progress, and more!",1,NaN,no shipping specified,None
11,1672335313,. Free LP,15,NaN,no shipping specified,None
12,1672335313,. Free LP and tshirt,40,NaN,no shipping specified,None


#### Preliminary Data Cleaning - Table: Item

In [ ]:
item_df = pd.read_sql_query("SELECT * FROM item", conn)
item_df.set_index("id", inplace=True)
print(item_df.shape)
item_df.head()

(333890, 5)


,project_id,taxable,name,edit_path,amount
id,,,,,
98403,22851610,False,"Hand Written ""Thank You"" Note",/projects/falsegods/editing-false-gods-the-book-of-azure/items/98403/edit,0
98406,22851610,False,"A copy of the Manuscript, signed by the author.",/projects/falsegods/editing-false-gods-the-book-of-azure/items/98406/edit,0
98407,22851610,False,"A copy of the cover art, signed by the Artist",/projects/falsegods/editing-false-gods-the-book-of-azure/items/98407/edit,0
98408,22851610,False,"A copy of the finished work, signed by the author (may be late)",/projects/falsegods/editing-false-gods-the-book-of-azure/items/98408/edit,0
98409,22851610,False,Copy of original manuscript with editing marks,/projects/falsegods/editing-false-gods-the-book-of-azure/items/98409/edit,0


In [ ]:
# Drop all attributes except "name", since only this one is included in the Webrobots dataset
item_df = item_df[["name"]]
item_df.head()

,name
id,
98403,"Hand Written ""Thank You"" Note"
98406,"A copy of the Manuscript, signed by the author."
98407,"A copy of the cover art, signed by the Artist"
98408,"A copy of the finished work, signed by the author (may be late)"
98409,Copy of original manuscript with editing marks


#### Preliminary Data Cleaning - Table: Reward Item

In [ ]:
reward_item_df = pd.read_sql_query("SELECT * FROM reward_item", conn)
reward_item_df.set_index("id", inplace=True)
print(reward_item_df.shape)
reward_item_df.head()

(26101367, 9)


,item_project_id,reward_id,quantity,position,item_taxable,item_amount,item_edit_path,item_id,item_name
id,,,,,,,,,
24,2500107,5145468,1,0.0,False,0.0000,/projects/2101365243/paws-for-a-beer/items/21/edit,21,Many Internet Thanks! You have helped!!
25,2500107,5145469,1,0.0,False,0.0000,/projects/2101365243/paws-for-a-beer/items/22/edit,22,Two One-Day Passes
26,2500107,5145474,1,0.0,False,0.0000,/projects/2101365243/paws-for-a-beer/items/23/edit,23,Two Month Membership
27,2500107,5145473,1,0.0,False,0.0000,/projects/2101365243/paws-for-a-beer/items/24/edit,24,Pet Portrait
28,2500107,5145475,1,0.0,False,0.0000,/projects/2101365243/paws-for-a-beer/items/25/edit,25,6 Month Membership


In [ ]:
# Drop instances that are not available in Reward_DF
print("Before: {}".format(reward_item_df.shape))
reward_item_df = reward_item_df[reward_item_df.reward_id.isin(reward_df.index)] 
print("After: {}".format(reward_item_df.shape))

Before: (26101367, 9)
After: (23542884, 9)


In [ ]:
# Drop some attributes, since they also won't be available in Webrobots dataset
reward_item_df.drop(columns=["quantity", "position", "item_taxable", "item_amount", "item_edit_path"], inplace=True)
reward_item_df.head()

,item_project_id,reward_id,item_id,item_name
id,,,,
24,2500107,5145468,21,Many Internet Thanks! You have helped!!
25,2500107,5145469,22,Two One-Day Passes
26,2500107,5145474,23,Two Month Membership
27,2500107,5145473,24,Pet Portrait
28,2500107,5145475,25,6 Month Membership


In [ ]:
reward_item_df.drop_duplicates(inplace=True)
reward_item_df.shape

(906776, 4)

In [ ]:
# Merge table "reward_item" with "item"
print("Before: {}".format(reward_item_df.shape))
reward_item_df = reward_item_df.join(other=item_df, on="item_id", how="left")
print("After: {}".format(reward_item_df.shape))

Before: (906776, 4)
After: (906776, 5)


In [ ]:
reward_item_df.isna().sum()

item_project_id    0
reward_id          0
item_id            0
item_name          0
name               0
dtype: int64

In [ ]:
reward_item_df.head()

,item_project_id,reward_id,item_id,item_name,name
id,,,,,
24,2500107,5145468,21,Many Internet Thanks! You have helped!!,Many Internet Thanks! You have helped!!
25,2500107,5145469,22,Two One-Day Passes,Two One-Day Passes
26,2500107,5145474,23,Two Month Membership,Two Month Membership
27,2500107,5145473,24,Pet Portrait,Pet Portrait
28,2500107,5145475,25,6 Month Membership,6 Month Membership


In [ ]:
reward_item_df[reward_item_df.item_name!=reward_item_df.name].head(10)

,item_project_id,reward_id,item_id,item_name,name
id,,,,,
437706,2814722,5726253,133764,Freecast Universal,Freestream Universal
437708,2814722,5726253,133748,Freecast Kickstarter T-Shirt,Freestream Kickstarter T-Shirt
451259,2874140,5781152,148914,1x Micro USB Power Adapter,Micro USB Power Adapter
472879,2764330,5688945,156195,1 2013 Original Prototype (Only 3 exist),1x 2013 Original Prototype (Only 3 exist)
472881,2764330,5688945,156197,1 2015 Original Gen Con prototype (Only 4 exist!),1x 2015 Original Gen Con prototype (Only 4 exist!)
489708,2814722,5726253,161675,Freecast Hoodie - Gotta Stay Warm,Freestream Hoodie - Gotta Stay Warm
489716,2814722,5726210,133753,Freecast 5 Panel Hat,Freestream 5 Panel Hat
489718,2814722,5726216,133754,Freecast Standard,Freestream Standard
489810,2814722,5726219,133763,Freecast Professional,Freestream Professional


Result:
- "item_project_id" does not correspond to the real project_id --> drop attribute
- "name" is more accurate than "item_name" --> i.e. drop "item_name"

In [ ]:
# Drop attributes "item_project_id", "item_name", and "item_id", since not useful anymore
print("Before: {}".format(reward_item_df.shape))
reward_item_df.drop(columns=["item_project_id", "item_name", "item_id"], inplace=True)
print("After: {}".format(reward_item_df.shape))

Before: (906776, 5)
After: (906776, 2)


In [ ]:
reward_item_df.rename(columns={"name":"item_names"}, inplace=True)
reward_item_df.head()

,reward_id,item_names
id,,
24,5145468,Many Internet Thanks! You have helped!!
25,5145469,Two One-Day Passes
26,5145474,Two Month Membership
27,5145473,Pet Portrait
28,5145475,6 Month Membership


In [ ]:
len(reward_item_df.reward_id.unique())

264681

In [ ]:
# Aggregate all items per reward
reward_item_df = reward_item_df.groupby(by="reward_id").agg(list)
print(reward_item_df.shape)
reward_item_df.head()

(264681, 1)


,item_names
reward_id,
8852,[Line Art pdf.]
325520,[Psalms 1 & 23 Downloadable Prints]
325523,"[Psalms Softcover Book, Psalms 1 & 23 Downloadable Prints]"
325524,"[Pentateuch Softcover Book, Psalms 1 & 23 Downloadable Prints]"
325527,"[Psalms Coffee Table Book, Psalms 1 & 23 Downloadable Prints]"


In [ ]:
# Merge table reward_df with reward_item_df
print("Before: {}".format(reward_df.shape))
reward_df = reward_df.join(other=reward_item_df, how="left")
print("After: {}".format(reward_df.shape))

Before: (2914494, 6)
After: (2914494, 7)


#### Final Cleaning of Reward_df:

In [ ]:
reward_df.head()

,project_id,reward_description,pledge_amount,limit,shipping_policy,estimated_delivery,item_names
id,,,,,,,
3,1860890148,. Grace Jones T Shirt - Limited Edition. (Delivery Included),20,100.0,no shipping specified,None,NaN
8,405307432,". Get the behind-the-scenes updates AND an exclusive early download of the record as soon as it's done (~1 month before it's out on CDs, iTunes, etc.)",15,NaN,no shipping specified,None,NaN
9,405307432,". Get access to the behind-the-scenes video & audio updates that will document the creation of this album. This will include video clips of recording sessions, audio clips of tunes as they progress, and more!",1,NaN,no shipping specified,None,NaN
11,1672335313,. Free LP,15,NaN,no shipping specified,None,NaN
12,1672335313,. Free LP and tshirt,40,NaN,no shipping specified,None,NaN


In [ ]:
# Convert attribute "item_names" into a string
reward_df["item_names"] = reward_df.item_names.apply(lambda x: ", ".join([str(item_name) for item_name in x]) if type(x) is list else "")
reward_df[reward_df.item_names!=""].item_names.head()

id
893316    #BeingRealComesEasy Mixtape
893317    #BeingRealComesEasy Mixtape
893318    #BeingRealComesEasy Mixtape
893319    #BeingRealComesEasy Mixtape
893320    #BeingRealComesEasy Mixtape
Name: item_names, dtype: object

In [ ]:
# Add attribute "item_names" to "reward_description"
reward_df["reward_description"] = reward_df["reward_description"] + ". Includes: " + reward_df["item_names"]
reward_df[reward_df.item_names!=""].sample(1)

,project_id,reward_description,pledge_amount,limit,shipping_policy,estimated_delivery,item_names
id,,,,,,,
6873946,715125837,"The Ultimate Salsero/a Swag Package. With this donation, you will receive all the swag as well as a signed physical copy of the CD (which will include your name printed on the sleeve), the Happy Birthday/thank you video, and free Salsa Nights for life!. Includes: Website Mention, Salsa del Norte T-Shirt, Salsa del Norte Sticker, Signed Physical CD, Mention on CD Sleeve, Happy Birthday Video, Free Salsa Nights for Life",1000,1.0,restricted,2019-12-01,"Website Mention, Salsa del Norte T-Shirt, Salsa del Norte Sticker, Signed Physical CD, Mention on CD Sleeve, Happy Birthday Video, Free Salsa Nights for Life"


In [ ]:
# Drop attribute "item_names", since not needed anymore
print("Before: {}".format(reward_df.shape))
reward_df.drop(columns=["item_names"], inplace=True)
print("After: {}".format(reward_df.shape))

Before: (2914494, 7)
After: (2914494, 6)


In [ ]:
reward_df.isna().sum()

project_id                  0
reward_description       1997
pledge_amount               0
limit                 1972544
shipping_policy             0
estimated_delivery     183991
dtype: int64

In [ ]:
len(reward_df.project_id.unique())

366833

In [ ]:
# Merge all rewards per project into a list
reward_df = reward_df.groupby(by="project_id").agg(list)
print(reward_df.shape)
reward_df.head()

(366833, 5)


,reward_description,pledge_amount,limit,shipping_policy,estimated_delivery
project_id,,,,,
5971,"[. An easy way to help, for any amount you wish! Give a dollar and feel like you are getting back at the politicians that didn't do their job!. Includes: , . Get back at those politicians that never followed through with their promises! And get your name acknowledged in the back of An American Apocalypse: The Reckoning!. Includes: , . Receive a copy of An American Apocalypse: The Reckoning signed by the artist of the book Jay Peteranetz and the writer Bob Hagedorn!. Includes: , . Receive a pack of four Hoodoo Zombie trading cards from An American Apocalypse: The Reckoning as well as a signed copy of the book.. Includes: , . Receive a copy of An American Apocalypse: The Reckoning signed by the artist of the book Jay Peteranetz and the writer Bob Hagedorn! AND a sketch on the inside cover of the book by Artist Jay Peteranetz.. Includes: , . Become one of the avenging Hoodoo Zombies from An American Apocalypse: The Reckoning and receive a copy of the book signed by the creators! Must send a head shot of yourself to be included.. Includes: , . Become one of the avenging Hoodoo Zombies from An American Apocalypse: The Reckoning, receive a personal sketch of your zombie on 8.5' x 11' Bristol Board and receive a copy of the book signed by the creators! Must send a head shot of yourself to be included.. Includes: , . Receive an original page of your choice* from An American Apocalypse: The Reckoning signed by the artist as well as a signed copy of the book!\r\n*Pages will be given out in order pledgers sign up.. Includes: , . Receive an original commission on 8.5""x11"" Bristol Board by artist Jay Peteranetz as well as a signed copy of An American Apocalypse: The Reckoning. Includes: , . Get three hours of tutoring from Jay Peteranetz, the series artist, on creating your own comic book and comic characters*. \r\n*recipients must be local to the Denver area. Includes: ]","[1, 5, 20, 25, 25, 75, 100, 150, 250, 500]","[nan, nan, 200.0, 100.0, 100.0, 50.0, 50.0, 100.0, 50.0, 7.0]","[no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified]","[2013-05-01, 2013-05-01, 2013-05-01, 2013-05-01, 2013-05-01, 2013-05-01, 2013-05-01, 2013-05-01, 2013-08-01, 2013-07-01]"
8624,"[The fire starter. Thanks for your suport, you receive a magnesium fire starter by Aranud. Includes: Fire starter, early bird. You receive one complete kombuis set with cover bag and save ‎€25. (EMRP: ‎€100). Includes: The kombuis, early bird round two. You receive one complete kombuis set with cover bag and save ‎€15. (EMRP: ‎€100). Includes: The kombuis, One set. You receive one complete kombuis set with cover bag and save ‎€15. (EMRP: ‎€100). Includes: The kombuis, Double set early bird. You receive two complete kombuis sets with cover bag and save ‎€25 on each. (EMRP: ‎€100). Includes: , Double set. You receive two complete kombuis sets with cover bag and save ‎€20 on each. (EMRP: ‎€100). Includes: The kombuis, Family set. You receive four complete kombuis sets with cover bag and save ‎€25 on each. (EMRP: ‎€100). Includes: The kombuis, Big pack. You receive ten complete kombuis sets with cover bag and save ‎€30 on each. (EMRP: ‎€100). Includes: The kombuis, Maxi pack. You receive one hundred complete kombuis sets with cover bag and save ‎€40 on each. (EMRP: ‎€100). Includes: The kombuis]","[31, 92, 99, 105, 185, 197, 370, 863, 7393]","[nan, 200.0, 325.0, nan, 100.0, nan, nan, nan, nan]","[unrestricted, unrestricted, unrestricted, unrestricted, unrestricted, unrestricted, unrestricted, unrestricted, unrestricted]","[2018-07-01, 2018-07-01, 2018-07-01, 2018-07-01, 2018-07-01, 2018-07-01, 2018-07-01, 2018-07-01, 2018-07-01]"
18520,"[P.W.Y.C (Pay What You Can). An Ep i created while on my travels through Toronto

##### Merge kickstarter_berkeley_df with reward_df:

In [ ]:
print(kickstarter_berkeley_df.shape)
kickstarter_berkeley_df.head(1)

(366907, 43)


,url_project,name,blurb,state,goal,pledged,backers_count,comments_count,updates_count,staff_pick,campaign_has_video,kickstarter_country,created_at,launched_at,state_changed_at,successful_at,deadline,updated_at,last_modification,profile_changed_after_success,creator_id,profile_id,subcategory,category,location_name,location_state,location_country,creator_name,creator_url,number_of_creator_links,creator_facebook_links,creator_facebook_linked,creator_twitter_links,creator_twitter_linked,creator_instagram_links,creator_instagram_linked,creator_youtube_links,creator_youtube_linked,creator_linkedin_links,creator_linkedin_linked,creator_location_name,creator_state,creator_country
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22810934,https://www.kickstarter.com/projects/cavinbounce/lights-coma-action,"Lights, Coma, Action!","A book to tell the story of who I was (LIGHTS), how I sustained a serious brain injury (COMA), and my strong recovery (ACTION!).",successful,15000.0,18167.88,284,4,40,False,True,United States,2013-10-01,2013-10-03,2013-11-02,2013-11-02,2013-11-02,2017-03-01,2018-05-24,True,1157451606,716409,Nonfiction,Publishing,"Austin, TX",Texas,United States,Cavin Balaster,https://www.kickstarter.com/profile/cavinbounce,2,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,"Austin, TX",Texas,United States


In [ ]:
print(reward_df.shape)
reward_df.head(1)

(366833, 5)


,reward_description,pledge_amount,limit,shipping_policy,estimated_delivery
project_id,,,,,
5971,"[. An easy way to help, for any amount you wish! Give a dollar and feel like you are getting back at the politicians that didn't do their job!. Includes: , . Get back at those politicians that never followed through with their promises! And get your name acknowledged in the back of An American Apocalypse: The Reckoning!. Includes: , . Receive a copy of An American Apocalypse: The Reckoning signed by the artist of the book Jay Peteranetz and the writer Bob Hagedorn!. Includes: , . Receive a pack of four Hoodoo Zombie trading cards from An American Apocalypse: The Reckoning as well as a signed copy of the book.. Includes: , . Receive a copy of An American Apocalypse: The Reckoning signed by the artist of the book Jay Peteranetz and the writer Bob Hagedorn! AND a sketch on the inside cover of the book by Artist Jay Peteranetz.. Includes: , . Become one of the avenging Hoodoo Zombies from An American Apocalypse: The Reckoning and receive a copy of the book signed by the creators! Must send a head shot of yourself to be included.. Includes: , . Become one of the avenging Hoodoo Zombies from An American Apocalypse: The Reckoning, receive a personal sketch of your zombie on 8.5' x 11' Bristol Board and receive a copy of the book signed by the creators! Must send a head shot of yourself to be included.. Includes: , . Receive an original page of your choice* from An American Apocalypse: The Reckoning signed by the artist as well as a signed copy of the book!\r\n*Pages will be given out in order pledgers sign up.. Includes: , . Receive an original commission on 8.5""x11"" Bristol Board by artist Jay Peteranetz as well as a signed copy of An American Apocalypse: The Reckoning. Includes: , . Get three hours of tutoring from Jay Peteranetz, the series artist, on creating your own comic book and comic characters*. \r\n*recipients must be local to the Denver area. Includes: ]","[1, 5, 20, 25, 25, 75, 100, 150, 250, 500]","[nan, nan, 200.0, 100.0, 100.0, 50.0, 50.0, 100.0, 50.0, 7.0]","[no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified, no shipping specified]","[2013-05-01, 2013-05-01, 2013-05-01, 2013-05-01, 2013-05-01, 2013-05-01, 2013-05-01, 2013-05-01, 2013-08-01, 2013-07-01]"


In [ ]:
print("Before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df = kickstarter_berkeley_df.join(other=reward_df, how="left")
print("After: {}".format(kickstarter_berkeley_df.shape))

Before: (366907, 43)
After: (366907, 48)


In [ ]:
print(kickstarter_berkeley_df.reward_description.isna().sum())
print(kickstarter_berkeley_df.pledge_amount.isna().sum())
print(kickstarter_berkeley_df.limit.isna().sum())
print(kickstarter_berkeley_df.shipping_policy.isna().sum())
print(kickstarter_berkeley_df.estimated_delivery.isna().sum())

74
74
74
74
74


In [ ]:
# Rename reward-based columns
kickstarter_berkeley_df.rename(columns={"pledge_amount":"reward_pledge_amount", "limit":"reward_limit", 
                                      "shipping_policy":"reward_shipping_policy", "estimated_delivery":"reward_estimated_delivery"}, inplace=True)

In [ ]:
kickstarter_berkeley_df.sample(1)

url_project  \
id                                                                                                            
172151425  https://www.kickstarter.com/projects/1583648678/sinbuck-by-joe-vigil-david-barbour-and-tim-vigil   

                                                        name  \
id                                                             
172151425  Sinbuck by Joe Vigil, David Barbour and Tim Vigil   

                                                                                                                                             blurb  \
id                                                                                                                                                   
172151425  SINBUCK is the blood-soaked tale of a woman gunfighter’s journey through the wastelands of Hell battling demons and seeking redemption.   

                state    goal  pledged  backers_count  comments_count  \
id                                                                      
172151425  successful  2999.0   4638.0             46               2   

           updates_count  staff_pick  campaign_has_video kickstarter_country  \
id                                                                             
172151425             26       False                True       United States   

           created_at launched_at state_changed_at successful_at    deadline  \
id                                                                             
172151425  2016-01-19  2016-03-05       2016-04-11    2016-04-11  2016-04-11   

           updated_at last_modification  profile_changed_after_success  \
id                                                                       
172151425  2016-12-20        2018-05-29                           True   

           creator_id  profile_id     subcategory category  location_name  \
id                                                                          
172151425  1583648678     2328985  No Subcategory   Comics  San Diego, CA   

          location_state location_country creator_name  \
id                                                       
172151425     California    United States    Joe Vigil   

                                              creator_url  \
id                                                          
172151425  https://www.kickstarter.com/profile/1583648678   

           number_of_creator_links creator_facebook_links  \
id                                                          
172151425                        1                    NaN   

           creator_facebook_linked creator_twitter_links  \
id                                                         
172151425                    False                   NaN   

           creator_twitter_linked creator_instagram_links  \
id                                                          
172151425                   False                     NaN   

           creator_instagram_linked creator_youtube_links  \
id                                                          
172151425                     False                   NaN   

           creator_youtube_linked creator_linkedin_links  \
id                                                         
172151425                   False                    NaN   

           creator_linkedin_linked creator_location_name creator_state  \
id                                                                       
172151425                    False        Sacramento, CA    California   

          creator_country  \
id                          
172151425   United States   

                                                                                                                                                                                                                                                                                                                                                                                         

#### Consolidate Reward-Based Attributes (i.e. get rid of lists):

##### a) reward_description

In [ ]:
# Merge all reward_descriptions per project into one text
kickstarter_berkeley_df["reward_description"] = kickstarter_berkeley_df.reward_description.apply(
    lambda x: ". ".join([reward for reward in x if pd.isnull(reward)==False]) if type(x) is list else "")
kickstarter_berkeley_df.reward_description.head(1)

id
22810934    . •\tA DIGITAL VERSION OF THE BOOK WHEN IT IS AVAILABLE. Includes: . . •\tA HARD COPY OF THE BOOK WHEN IT IS AVAILABLE and a digital version of the book when it is available. Includes: . . •\tA SIGNED HARD COPY OF THE BOOK WHEN IT IS AVAILABLE and a digital version of the book when it is available. Includes: . . •\tAN OPTIONAL LOCAL SPEAKING ENGAGEMENT, WITH OR WITHOUT A MUSIC PERFORMANCE, GIVEN BY CAVIN BALASTER AT AN AGREED UPON TIME IN AUSTIN, TX, a signed hard copy of the book when it is available, and a digital version of the book when it is available. Includes: . . •\tAN OPTIONAL SPEAKING ENGAGEMENT, WITH OR WITHOUT MUSIC, GIVEN BY CAVIN BALASTER AT ANY APPROPRIATE LOCATION OF YOUR CHOOSING, AT AN AGREED UPON TIME (EXPENSES NOT INCLUDED), 10 signed hard copies of the book when it is available, and a digital version of the book when it is available.. Includes: . . A DIGITAL DOWNLOAD OF MY MUSIC. Includes: . . YOUR NAME IN THE THANK YOU SECTION OF THE BOOK (This appl

##### b) reward_pledge_amount

In [ ]:
# Create attribute "number_of_rewards"
kickstarter_berkeley_df["number_of_rewards"] = kickstarter_berkeley_df.reward_pledge_amount.apply(
    lambda x: len(x) if type(x) is list else 0)
kickstarter_berkeley_df.number_of_rewards.value_counts()

6      34734
7      33991
5      33060
8      31136
4      30032
       ...  
92         1
91         1
86         1
84         1
253        1
Name: number_of_rewards, Length: 125, dtype: int64

In [ ]:
# Create attribute "reward_highest_pledge_amount"
kickstarter_berkeley_df["reward_highest_pledge_amount"] = kickstarter_berkeley_df.reward_pledge_amount.apply(
    lambda x: max(x) if type(x) is list else None)
kickstarter_berkeley_df.reward_highest_pledge_amount.value_counts()

1000.0     40564
500.0      27544
100.0      24522
10000.0    20655
5000.0     19694
           ...  
3853.0         1
9817.0         1
6188.0         1
9815.0         1
7623.0         1
Name: reward_highest_pledge_amount, Length: 7810, dtype: int64

In [ ]:
# Create attribute "lowest_highest_pledge_amount"
kickstarter_berkeley_df["reward_lowest_pledge_amount"] = kickstarter_berkeley_df.reward_pledge_amount.apply(
    lambda x: min(x) if type(x) is list else None)
kickstarter_berkeley_df.reward_lowest_pledge_amount.value_counts()

1.0       98054
5.0       69872
10.0      58760
25.0      21145
2.0       14282
          ...  
1433.0        1
1899.0        1
5717.0        1
357.0         1
568.0         1
Name: reward_lowest_pledge_amount, Length: 1161, dtype: int64

In [ ]:
# Create attribute "reward_average_pledge_amount"
kickstarter_berkeley_df["reward_average_pledge_amount"] = kickstarter_berkeley_df.reward_pledge_amount.apply(
    lambda x: sum(x)/len(x) if type(x) is list else None)
kickstarter_berkeley_df.reward_average_pledge_amount.value_counts()

25.000000      5574
10.000000      3073
20.000000      2812
50.000000      2533
1.000000       2481
               ... 
952.230769        1
504.600000        1
507.400000        1
1031.882353       1
437.733333        1
Name: reward_average_pledge_amount, Length: 72170, dtype: int64

##### c) reward_limit

In [ ]:
# Create attribute "number_of_limited_rewards"
kickstarter_berkeley_df["number_of_limited_rewards"] = kickstarter_berkeley_df.reward_limit.apply(
    lambda x: len([1 for limit in x if pd.isnull(limit)==False]) if type(x) is list else 0)
kickstarter_berkeley_df.number_of_limited_rewards.value_counts()

0      153426
1       46915
2       38516
3       31507
4       24258
        ...  
76          1
74          1
73          1
70          1
245         1
Name: number_of_limited_rewards, Length: 110, dtype: int64

In [ ]:
# Create attribute "reward_limit_sum"
kickstarter_berkeley_df["reward_limit_sum"] = kickstarter_berkeley_df.reward_limit.apply(
    lambda x: 0 + sum([limit for limit in x if pd.isnull(limit)==False]) if type(x) is list else 0)
kickstarter_berkeley_df.reward_limit_sum.value_counts()

0.0        153426
10.0         8523
5.0          6414
20.0         6348
100.0        5835
            ...  
10629.0         1
4175.0          1
10625.0         1
8320.0          1
10079.0         1
Name: reward_limit_sum, Length: 5189, dtype: int64

In [ ]:
# Create attribute "reward_limit_average"
kickstarter_berkeley_df["reward_limit_average"] = kickstarter_berkeley_df["reward_limit_sum"] / kickstarter_berkeley_df["number_of_limited_rewards"]
kickstarter_berkeley_df.loc[kickstarter_berkeley_df.reward_limit_average.isna()==True,"reward_limit_average"] = 0
kickstarter_berkeley_df.reward_limit_average.value_counts()

0.000000         153426
10.000000         10179
5.000000           8128
1.000000           6730
100.000000         5838
                  ...  
380000.000000         1
2355.600000           1
691.333333            1
1869.000000           1
2777.500000           1
Name: reward_limit_average, Length: 14935, dtype: int64

##### d) reward_shipping_policy

In [ ]:
# Create attribute "number_of_shipped_rewards"
kickstarter_berkeley_df["number_of_shipped_rewards"] = kickstarter_berkeley_df.reward_shipping_policy.apply(
    lambda x: len([1 for policy in x if policy!="no shipping specified"]) if type(x) is list else 0)
kickstarter_berkeley_df.number_of_shipped_rewards.value_counts()

0      109411
1       27653
4       27253
5       26124
3       25975
        ...  
84          1
83          1
79          1
75          1
252         1
Name: number_of_shipped_rewards, Length: 111, dtype: int64

In [ ]:
# Create attribute "number_of_restricted_shipping_rewards"
kickstarter_berkeley_df["number_of_restricted_shipping_rewards"] = kickstarter_berkeley_df.reward_shipping_policy.apply(
    lambda x: len([1 for policy in x if policy=="restricted"]) if type(x) is list else 0)
kickstarter_berkeley_df.number_of_restricted_shipping_rewards.value_counts()

0      254335
1       28035
2       16876
3       14541
4       12329
        ...  
55          1
54          1
51          1
45          1
192         1
Name: number_of_restricted_shipping_rewards, Length: 70, dtype: int64

##### e) reward_estimated_delivery

In [ ]:
# Create attribute "reward_earliest_delivery"
from datetime import datetime
kickstarter_berkeley_df["reward_earliest_delivery"] = kickstarter_berkeley_df.reward_estimated_delivery.apply(
    lambda x: min([datetime.strptime(delivery, '%Y-%m-%d') for delivery in x if pd.isnull(delivery)==False], default=None) if type(x) is list else None)

In [ ]:
# Create attribute "reward_latest_delivery"
kickstarter_berkeley_df["reward_latest_delivery"] = kickstarter_berkeley_df.reward_estimated_delivery.apply(
    lambda x: max([datetime.strptime(delivery, '%Y-%m-%d') for delivery in x if pd.isnull(delivery)==False], default=None) if type(x) is list else None)

In [ ]:
# Create attribute "number_of_delivery_specified_rewards"
kickstarter_berkeley_df["number_of_delivery_specified_rewards"] = kickstarter_berkeley_df.reward_estimated_delivery.apply(
    lambda x: len([1 for delivery in x if pd.isnull(delivery)==False]) if type(x) is list else 0)
kickstarter_berkeley_df.number_of_delivery_specified_rewards.value_counts()

6      31500
7      30722
5      30505
4      28152
8      28088
       ...  
92         1
91         1
86         1
84         1
253        1
Name: number_of_delivery_specified_rewards, Length: 125, dtype: int64

In [ ]:
kickstarter_berkeley_df.sample(1)

,url_project,name,blurb,state,goal,pledged,backers_count,comments_count,updates_count,staff_pick,campaign_has_video,kickstarter_country,created_at,launched_at,state_changed_at,successful_at,deadline,updated_at,last_modification,profile_changed_after_success,creator_id,profile_id,subcategory,category,location_name,location_state,location_country,creator_name,creator_url,number_of_creator_links,creator_facebook_links,creator_facebook_linked,creator_twitter_links,creator_twitter_linked,creator_instagram_links,creator_instagram_linked,creator_youtube_links,creator_youtube_linked,creator_linkedin_links,creator_linkedin_linked,creator_location_name,creator_state,creator_country,reward_description,reward_pledge_amount,reward_limit,reward_shipping_policy,reward_estimated_delivery,number_of_rewards,reward_highest_pledge_amount,reward_lowest_pledge_amount,reward_average_pledge_amount,number_of_limited_rewards,reward_limit_sum,reward_limit_average,number_of_shipped_rewards,number_of_restricted_shipping_rewards,reward_earliest_delivery,reward_latest_delivery,number_of_delivery_specified_rewards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
617315258,https://www.kickstarter.com/projects/929488870/ferus,Ferus,Ferus discovers he has telekinetic abilities right out of his dreams and explores his new talent. Challenges and adventures arise!,failed,200.0,55.0,3,0,1,False,False,United States,2014-08-05,2014-09-26,2014-11-25,NaN,2014-11-25,2014-11-25,2017-03-11,False,929488870,1262959,Graphic Novels,Comics,"Albuquerque, NM",New Mexico,United States,Tristan Price,https://www.kickstarter.com/profile/929488870,0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,"Albuquerque, NM",New Mexico,United States,". You will receive high-quality PDF copy of the comic.. Includes: . . You will receive a high-quality PDF copy of the comic, and a printed copy of the comic.. Includes: . . You will receive a high-quality PDF, and a printed copy of the comic with a dedication or custom doodle in the front.. Includes: . . You will receive a high-quality PDF and two printed copies of the comic.. Includes: . . Receive a PDF copy (high-quality, of course), a dedicated printed copy, and one of the original art pages from the comic.. Includes: . . Receive a PDF copy, and a dedicated printed copy of the comic. I will draw and/or paint a piece to send you.. Includes: . . Receive a PDF copy, a dedicated printed copy of the comic, and send me a photo of you or a friend and I will create a painting in the style of the comic.. Includes: . . Receive a PDF copy, a dedicated printed copy of the comic, an original piece from the comic, a custom painting, and the option to have a character based on you in one of the future installments of the comic, or a lunch with the author (lunch only held in Albuquerque).. Includes: . . Feel good about supporting an emerging graphic novelist! I will send a PDF of a random scanned piece of my art.. Includes:","[10, 15, 25, 25, 50, 75, 100, 200, 1]","[nan, nan, nan, nan, 8.0, 10.0, 10.0, 5.0, nan]","[no shipping specified, restricted, restricted, restricted, restricted, restricted, restricted, restricted, no shipping specified]","[2015-01-01, 2015-01-01, 2015-01-01, 2015-01-01, 2015-01-01, 2015-02-01, 2015-02-01, 2015-03-01, 2015-01-01]",9,200.0,1.0,55.666667,4,33.0,8.25,7,7,2015-01-01,2015-03-01,9


##### f) Drop Reward-Based List Attributes, since not needed anymore

In [ ]:
print("Before: {}".format(kickstarter_berkeley_df.shape))
kickstarter_berkeley_df.drop(columns=["reward_pledge_amount","reward_limit","reward_shipping_policy", "reward_estimated_delivery"], inplace=True)
print("After: {}".format(kickstarter_berkeley_df.shape))

Before: (366907, 60)
After: (366907, 56)


In [ ]:
kickstarter_berkeley_df.sample(1)

,url_project,name,blurb,state,goal,pledged,backers_count,comments_count,updates_count,staff_pick,campaign_has_video,kickstarter_country,created_at,launched_at,state_changed_at,successful_at,deadline,updated_at,last_modification,profile_changed_after_success,creator_id,profile_id,subcategory,category,location_name,location_state,location_country,creator_name,creator_url,number_of_creator_links,creator_facebook_links,creator_facebook_linked,creator_twitter_links,creator_twitter_linked,creator_instagram_links,creator_instagram_linked,creator_youtube_links,creator_youtube_linked,creator_linkedin_links,creator_linkedin_linked,creator_location_name,creator_state,creator_country,reward_description,number_of_rewards,reward_highest_pledge_amount,reward_lowest_pledge_amount,reward_average_pledge_amount,number_of_limited_rewards,reward_limit_sum,reward_limit_average,number_of_shipped_rewards,number_of_restricted_shipping_rewards,reward_earliest_delivery,reward_latest_delivery,number_of_delivery_specified_rewards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
429181805,https://www.kickstarter.com/projects/12283819/quickpad-one-device-to-rule-them-all,QuickPad - One Device to Rule Them All.,6 inch lcd touch screen designed to make your work and entertainment quicker than ever. Now every command can be just one key away.,successful,9288.66106,17586.08929,149,8,3,False,True,United Kingdom,2018-05-09,2018-06-20,2018-07-21,2018-07-21,2018-07-21,2018-07-24,2018-08-07,False,12283819,3377506,Gadgets,Technology,"Southampton, UK",England,United Kingdom,Quick Solutions,https://www.kickstarter.com/profile/12283819,0,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,NaN,NaN,"THANK YOU!. Big thank you for your support. Thanks to you we will be closer to our goal to bring QuickPad to live! We will keep you update on the progress.. Includes: . QuickPad Early-Bird. Get your QuickPad £50 lower then the suggested retail price of £129.\r\n\r\nWe will collect your colour preferences after the campaign.\r\n\r\nFree global shipping.. Includes: QuickPad, Silicon keyboards, Sticker sheets, USB cable, USB-C Adapter. QuickPad Kickstarter Special. Get your QuickPad £30 lower then the suggested retail price of £129.\r\n\r\nWe will collect your colour preferences after the campaign.\r\n\r\nFree global shipping.. Includes: QuickPad, Silicon keyboards, Sticker sheets, USB cable, USB-C Adapter. 2x QuickPads Early-Bird. 2 QuickPads £110 lower then the suggested retail price of £258.\r\n\r\nWe will collect your colour preferences after the campaign.\r\n\r\nFree global shipping.. Includes: QuickPad, Silicon keyboards, Sticker sheets, USB cable, USB-C Adapter. 3x QuickPads Early-Bird. 3 QuickPads £189 lower then the suggested retail price of £389.\r\n\r\nWe will collect your colour preferences after the campaign.\r\n\r\nFree global shipping.. Includes: QuickPad, Silicon keyboards, Sticker sheets, USB cable, USB-C Adapter",5,277.0,1.0,142.0,3,1700.0,566.666667,5,0,2018-10-01,2018-11-01,5


#### Save Dataset:

In [ ]:
kickstarter_berkeley_df.to_csv("02_Data Collection/Kickstarter_Berkeley_Cleaned.csv")

In [ ]:
kickstarter_berkeley_df = pd.read_csv("02_Data Collection/Kickstarter_Berkeley_Cleaned.csv", index_col=0)
print(kickstarter_berkeley_df.shape)
kickstarter_berkeley_df.head()